In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

# !pip install validclust

#### PARAMETERS USED:
<b>Dataset and K varies</b> <br>
1. thresh (opt = 0.3)
2. alpha_weight_avg (opt = 0.6)
3. per in LLE (opt = 0.5)
4. percent in GetCutOffDistMod (opt = 0.7)
5. val2 in Correcting_wrong_clustering (opt = 20)
6. val3 in Correcting_wrong_clustering (opt = 15)
7. percent1 in Correcting_wrong_clustering (opt = 0.8)
8. percent2 in Correcting_wrong_clustering (opt = 0.16)
9. percent3 in Correcting_wrong_clustering (opt = 0.10)
10. percent4 in Correcting_wrong_clustering (opt = 0.01)
11. algo for NN = kd_tree
13. KVal in Outlier Postprocessing = 25
14. percent5 in Outlier Postprocessing = 0.1

In [ ]:
import math
import time
import csv
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
from statistics import mean
from operator import itemgetter
from collections import deque
from collections import defaultdict
from validclust import dunn
from sklearn.metrics.pairwise import pairwise_distances
from sklearn .metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score
from sklearn import metrics
import operator
from IPython.display import clear_output
import os
import shelve
import pickle

In [ ]:
start = time.time()

In [ ]:
########################## USER INPUTS ##########################
data_path = r"E:\ISI\ISI PROJECT\DATASETS 28-10-2020\pendigits(data).csv"
# data_path = r"/content/drive/MyDrive/DATASETS/Mnist(data).csv"
Ground_Truth_data_path=r"E:\ISI\ISI PROJECT\DATASETS 28-10-2020\pendigits(gt).csv"
# Ground_Truth_data_path = r"/content/drive/MyDrive/DATASETS/Mnist(gt).csv"
K = 50            ## Actually K-1 neighbors are considered. Remember              #### Imp
#25 #20 #10
Dataset_Name_Only = "pendigits"
# For streaming
train_start = 0
train_end = 1000 #60
test_start = 1000
test_end = 1020 #6870 

####################### Constant Parameters #######################
percent5 = 5

In [ ]:
name_storage_file = "Streaming_Model_" + Dataset_Name_Only + ".db"
SM = shelve.open(name_storage_file)
print("Current Status of Shelve Dictionaries: ", list(SM.items()))

In [ ]:
# Name: Reading Data and Thresholds

if 'Name: Reading Data and Thresholds' not in SM:
    with open(data_path, 'r') as f:
        data_all = np.genfromtxt(f, delimiter=',')   
        ## If we take whole 
        # data1 = data_all
        ## For Streaming
        data1 = data_all[train_start:train_end]

    Concept_Evolution_Thresh = 0.1*len(data_all)   # 0.06

    Dimension = data1.shape[1]
    #data1=data1/np.linalg.norm(data1)
    print("Training Length: ", len(data1))
    print("Dimension of Dataset: ",Dimension)
    thresh = 0.3                                                                       #### Imp
    alpha_weight_avg = 0.6
    algo = 'ball_tree' #'kd_tree' 

    Outlier_Detection = True   # False 
    Clustering = False         # True
    
    ############### SAVING ###############
    SM['data_path'] = data_path
    SM['Ground_Truth_data_path'] = Ground_Truth_data_path
    SM['K'] = K
    SM['Dimension'] = Dimension
    SM['thresh'] = thresh
    SM['alpha_weight_avg'] = alpha_weight_avg
    SM['algo'] = algo
    SM['Outlier_Detection'] = Outlier_Detection
    SM['Clustering'] = Clustering
    SM['data1'] = data1
    SM['train_start'] = train_start
    SM['train_end'] = train_end
    SM['test_start'] = test_start
    SM['test_end'] = test_end
    SM['data_all'] = data_all
    SM['Concept_Evolution_Thresh'] = Concept_Evolution_Thresh

    SM['Name: Reading Data and Thresholds'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Get Incoming Time

if 'Name: Get Incoming Time' not in SM:
    def GetIncomingTime(data_all):
        inc_start_time = time.time()

        for i in range(len(data_all)):
            pass

        total_time = time.time() - inc_start_time
        avg_inc_time = total_time/(len(data_all)-1)
        return avg_inc_time

    data_all = SM['data_all']
    incoming_time_dur = GetIncomingTime(data_all)
    SM['incoming_time_dur'] = incoming_time_dur
    print("Average Incoming Point is {} seconds".format(incoming_time_dur))
    
    SM['Name: Get Incoming Time'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# LLE: Locally linear Embedding
def LLE(data, K): #, data_point_Idx):
    row=len(data)
    #print(row)
    dimension=len(data[0])
    #print(dimension)
    neighbors = K
    #W=np.zeros((row, K)) # W=np.zeros((row, row)) 
    count_pts = 0

    hash_map = {value:np.array([]) for value in range(len(data))}

    for i in range(row):
        D_i=np.array(data-data[i, :])
        #print(D_i)
        distance=(D_i**2).sum(1)
        #print(distance)
        nearest_neighbor=np.argsort(distance)[1:(neighbors+1)]
        #print(nearest_neighbor)
        D_nbrs=D_i[nearest_neighbor, :]
        #print(D_nbrs)
        ##Q=np.dot(D_nbrs, D_nbrs.T)
        Q=np.matmul(D_nbrs, D_nbrs.T)
        #print(Q)
#         t=np.trace(Q)
#         r=0.001*t
        r = 0.001 * float(Q.trace())
        if(neighbors>=dimension):
#             sig2 = (np.linalg.svd(D_i,compute_uv=0))**2
#             r = np.sum(sig2[dimension:])
            Q=Q+(r*np.identity(neighbors))
#             Q.flat[::neighbors+1] += r
        ##w=np.linalg.solve(Q, np.ones(neighbors))
        w=np.linalg.solve(Q, np.ones(neighbors))
        
        w = w/sum(w)
        ##w=w/sum(w)
        #print(i, w)
        #W[i, nearest_neighbor] = w
        
        # 10 2 6 7 3 9 5
        # weight: 10 9 7 6 5 3 2
        # pt:     (1,3,5,6,7)  pt2 pt 4 ::5

        # My code
        temp = []
        for ele in w:
            #print(ele)
            #if ele != 0:
            temp.append(ele)
            temp.sort(reverse = True)
            hash_map[i] = np.array(temp) 
            
#     I=np.identity(row)
#     M=I-W
        if (count_pts+1) % 100 == 0:
            print("****************Computed LLE for {0} datapoints****************".format(count_pts+1))
        count_pts += 1

    per = 5 #0.6 #0.3 #0.5 #0.2 #0.05 #0.1
    store_K = defaultdict()

    for j in range(len(data)):
        count = 0
        for i in range(1, len(hash_map[j])):
            #print(((1/neighbors) - (dec_per*(1/neighbors))), ((1/neighbors) + (dec_per*(1/neighbors))))
            if round((abs(hash_map[j][i-1] - hash_map[j][i])/abs(hash_map[j][i-1]))*100,1) <= per:
                count += 1
        store_K[j] = count
        
    # print(store_K)
    for key in store_K:
        store_K[key] += 1
    return store_K

    # return store_K[data_point_Idx] + 1  # +1 because all neighbouring algorithms consider the datapoint itself to be a part
                                          # of its neighbour

In [ ]:
# Name: LLE_lookup Calculation

if 'Name: LLE_lookup Calculation' not in SM:
    data1 = SM['data1']
    K = SM['K']
    
    start4 = time.time()
    # Optimization: Instead of calling the LLE for each point, we make it constant time
    LLE_lookup = {}
    LLE_lookup = LLE(data1, K)
    # time taken = ",time.time()-start4," sec")
    
    SM['LLE_lookup'] = LLE_lookup

    print("time taken = ",time.time()-start4," sec")
    
    SM['Name: LLE_lookup Calculation'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Distance with one another calculation

if 'Name: Distance with one another calculation' not in SM:
    data1 = SM['data1']
    algo = SM['algo']
    
    ## Calculate the distance of each point with other points in the dataset
    nbrs_ddcal = NearestNeighbors(n_neighbors=len(data1), algorithm=algo).fit(data1)
    distances_ddcal, indices_ddcal = nbrs_ddcal.kneighbors(data1)
    print("****************Distance Between All Datapoints Computed****************")
    # # print(indices_ddcal)

    # Hash_Map = {i:0 for i in range(len(data1))}
    # labels = {i:0 for i in range(len(data1))}
    # density = []
    # for i in range(len(data1)):
    #     density.append([sum(distances_ddcal[i])/(K-1), data1[i], max(distances_ddcal[i]), indices_ddcal[i][1:], i])
    #     Hash_Map[i] = density[i]
    
    # SM['nbrs_ddcal'] = nbrs_ddcal
    
    with open('distances_ddcal.p', 'wb') as pfile1:
        pickle.dump(distances_ddcal, pfile1, protocol=4)
    # SM['distances_ddcal'] = distances_ddcal
    with open('indices_ddcal.p', 'wb') as pfile2:
        pickle.dump(indices_ddcal, pfile2, protocol=4)
    # SM['indices_ddcal'] = indices_ddcal
    
    SM['Name: Distance with one another calculation'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Density Computation of Each

if 'Name: Density Computation of Each' not in SM:
    data1 = SM['data1']
    data_all = SM['data_all']
    LLE_lookup = SM['LLE_lookup']
    with open('distances_ddcal.p', 'rb') as pfile1:
        distances_ddcal = pickle.load(pfile1)
    # distances_ddcal = SM['distances_ddcal']
    with open('indices_ddcal.p', 'rb') as pfile2:
        indices_ddcal = pickle.load(pfile2)
    # indices_ddcal = SM['indices_ddcal']

    # New Method of finding the densest point
    Hash_Map = {i:0 for i in range(len(data1))}
    labels = {i:0 for i in range(len(data1))}
    Store_Self_Density = {i:0 for i in range(len(data_all))}
    Store_Radius_Density = {i:[0,0] for i in range(len(data_all))}
    Store_Parents = {i:[0] for i in range(len(data_all))}
    density = []

    for i in range(len(data1)):
        K = LLE_lookup[i]  # 1 already added to K by the LLE function 
        distances = distances_ddcal[i][:K]
        idxs = indices_ddcal[i][1:K]
        # density.append([sum(distances)/(K-1), data1[i], max(distances), idxs, i])
        Radius = max(distances)
        # print("Sum: ", sum(distances))
        # print("Area: ", (math.pi*Radius*Radius))
        # print("Density: ", sum(distances)/(math.pi*Radius*Radius))
        density.append([(sum(distances)/(math.pi*Radius*Radius)), data1[i], max(distances), idxs, i])
        Hash_Map[i] = density[i]
        if (i+1)%100 == 0: 
            print("****************Density Computed for {} Datapoints****************".format(i+1))
            
    SM['density'] = density
    SM['Hash_Map'] = Hash_Map
    SM['labels'] = labels
    SM['Store_Self_Density'] = Store_Self_Density
    SM['Store_Radius_Density'] = Store_Radius_Density
    SM['Store_Parents'] = Store_Parents
    
    SM['Name: Density Computation of Each'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Sort Density

if 'Sort Density' not in SM:
    density = SM['density']

    # Distance-Density Value; Datapoint; Radius; k-nearest neighbors; idx_number
    # 0: 0.45; (25,5); 0.2; [1,2,3,4,5]; 0

    density.sort(key = itemgetter(0), reverse = True)  #False
    #print(density[:3])
    
    SM['density'] = density
    
    SM['Name: Sort Density'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Function to calculate the weighted moving average
def numpy_ewma_vectorized_v2(values, alpha):
    #values = np.array(values)
    span = (2/alpha) - 1
    df = pd.DataFrame(values)
    return df.ewm(span=span).mean().iloc[-1][0]

In [ ]:
"""
BFS based Approach: 

It takes care of the maximum recursion depth constraint in Python programming Language.
This however may take more time than DFS
"""
def NovelClus_FF(Parent, Hash_Map, dataset, K, list_of_parents_queue, labels, thresh, cluster_num, can_form_cluster, 
                 alpha_weight_avg, Tree_Structure, Roots, Store_Radius_Density, Store_Parents,
                distances_ddcal, indices_ddcal, Store_Self_Density):
    ##print("Cluster =", cluster_num)
    Hold_Root_Density = Parent[0]
    
    # Adding only the root
    Roots[Parent[4]] = [Parent[1],Parent[0]]
    Store_Radius_Density[Parent[4]] = [Parent[2], Parent[0]]
    
    # Add density of the root to Store_Self_Density
    Store_Self_Density[Parent[4]] = Parent[0]
    
    
    # We make use of a Deque for both side operation
    Queue = deque()
    
    # Append from left the Parent Node
    Queue.appendleft(Parent)
    
    
    # print("IP: ", Parent)
    
    
    # While Queue is not Null:
        # Queue extracts the first Node
        # We check if k children of Node satisfies the density criterion
        # if a child satisfies, it is pushed into the Queue from the end; else it is termed -1 and left
    
    while Queue:
        # Pop from the right end
        Parent = Queue.pop()
        # print("P: ", Parent[-1])
            
        # Getting important information from Parent
        Density_Parent = Parent[0]
        Radius = Parent[2]
        #Child_Idx = Parent[3][idx] 
        Child_Idx_array = Parent[3]   # [94,  32, 117, .... 100, 118]
        # print("CI: ", Child_Idx_array)
        #Child = Hash_Map[Child_Idx]
        #Child_Datapoint = Child[1]
        
        # Update the list_of_parents for the weighted moving average
        list_of_parents = list_of_parents_queue.pop() + [Density_Parent] 
        Store_Parents[Parent[4]] = list_of_parents
        #print("LoP =", list_of_parents)
        
        
        Child_array = []
        Child_Datapoint_array = []
        for Child_Idx in Child_Idx_array:
            Child_array.append(Hash_Map[Child_Idx])
            # print("F1: ", Child_Idx, Hash_Map[Child_Idx])
            Child_Datapoint_array.append(Hash_Map[Child_Idx][1])  
        
        # For each Parent find which child satiesfies the density threshold criterion
        ##print("Number of Children Nodes =", len(Child_Idx_array), "\n")
        for i in range(len(Child_Idx_array)):
            
            Child = Child_array[i]
            Child_Idx = Child[-1]             ##Child_Idx_array[i]
            # print("Only CIN: ", Child_Idx)
            Child_Datapoint = Child[1]        ##Child_Datapoint_array[i]
            
            # Base case: if child is already labelled in cluster then ignore
            if labels[Child_Idx] > 0:
                #print("Child is already labelled!!")
                continue
             
            
            # # Old
            # # print("Radius: ", Radius)
            # neigh = NearestNeighbors(radius = Radius)
            # neigh.fit(dataset)
            # rng = neigh.radius_neighbors([Child_Datapoint])
            # print("Prev: ", len(rng[0][0]))
            
            # New
            neigh_dist = []
            neigh_idxs = []
            rng = [[],[]]
            for i in range(1, len(distances_ddcal[Child_Idx])):
                if distances_ddcal[Child_Idx][i] < Radius:
                    neigh_dist.append(distances_ddcal[Child_Idx][i])
                    # print("NIs: ", indices_ddcal[Child_Idx][i])
                    neigh_idxs.append(np.asarray(int(indices_ddcal[Child_Idx][i])))
                else:
                    break
            rng[0].append(np.asarray(neigh_dist))
            rng[1].append(np.asarray(neigh_idxs))
            rng = tuple(rng)
            # print("New RNG:", len(rng[0][0]))
            ####
            
            
            if len(rng[0][0]) == 0:
                Density_Child = 0
            else:
                # Density_Child = sum(rng[0][0])/len(rng[0][0])
                Density_Child = sum(rng[0][0])/(math.pi*Radius*Radius)
                
            #print("No. of Parent =",len(list_of_parents))
            
            data_parent = np.array(list_of_parents) #, dtype=np.float64)
            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
            #print("Density of Parent =", weighted_density)
            #print("Density of Child = ", Density_Child)
            numerator = abs(weighted_density - Density_Child)
            denominator = weighted_density
            #print("Weighted Density =", numerator/denominator)
            
            
            # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
            if numerator/denominator <= thresh:
                
                labels[Child_Idx] = cluster_num
                
                list_of_parents_queue.appendleft(list_of_parents)
                       
                # Update the information for this child
                Child[0] = Density_Child
                if len(rng[0][0]) == 0:
                    Child[2] = 0
                else:
                    Child[2] = max(rng[0][0])
                Child[3] = rng[1][0]
                
                
                # Store Adaptive Radius and New Density of the Parent Node for latter usage
                list_of_parents_1 = list_of_parents + [Density_Child]
                data_parent_1 = np.array(list_of_parents_1) #, dtype=np.float64)
                weighted_density_1 = numpy_ewma_vectorized_v2(data_parent_1, alpha_weight_avg)  #O()??
                Store_Radius_Density[Child[4]] = [Child[2], weighted_density_1]
                
                # Add density of child to Store_Self_Density
                Store_Self_Density[Child[4]] = Density_Child
                
                # Add Child to the Tree Structure
                if Parent[4] not in Tree_Structure:
                    Tree_Structure[Parent[4]] = [Child[4]]
                else:
                    Tree_Structure[Parent[4]].append(Child[4])
                
                Queue.appendleft(Child)
                
                # Update the hashmap OLD
                ##Hash_Map[Child[-1]] = Child
                
                # Update the hashmap NEW
                temp_dict = {Child[-1]:Child}
                Hash_Map.update(temp_dict)
                ##################################
                
            
                # The Parent can form a cluster of its own
                can_form_cluster[0] = [True]
                
                
            else:
                #print("Possible Anomaly!!")
                labels[Child_Idx] = -1
        

In [ ]:
# # Is this function really needed now??
# def update_nbrs(data1, idx, K, algo): # 70
#     nbrs_ddcal = NearestNeighbors(n_neighbors=K, algorithm=algo).fit(data1)
#     distances_ddcal, indices_ddcal = nbrs_ddcal.kneighbors(data1)

#     density_mod = []
#     density_mod.append([sum(distances_ddcal[idx])/(K-1), data1[idx], max(distances_ddcal[idx]), indices_ddcal[idx][1:], idx])
    
#     density_mod.append([sum(distances_ddcal[i])/(K-1), data1[i], max(distances_ddcal[i]), indices_ddcal[i][1:], i])
#     # density_cpy.append([sum(distances_ddcal[i])/(K-1), data1[i], max(distances_ddcal[i]), indices_ddcal[i][1:], i])
    
#     return density_mod[0] #, density_mod[0]


In [ ]:
def CheckWhichNodesHaveNoParentInTree(Tree_Structure):
    SupposedRoots = set()
    keysofDict = Tree_Structure.keys()
    for key_idx in Tree_Structure:
        boolean = False
        for item in keysofDict:
            if key_idx in set(Tree_Structure[item]):
                boolean = True
        if boolean == False:
            SupposedRoots.add(key_idx)
    return list(SupposedRoots)
            

In [ ]:
def AddRootsFromSupposedRoots(Supposed_Roots, Roots, data1, Store_Self_Density):
  # Roots[key]: {[dp, density_value]}
  for idx in Supposed_Roots:
    dp = data1[idx]

    boolean = False
    for root in Roots:
      item1 = Roots[root][0]
      comparison = item1 == dp
      equals_array = comparison.all()
      if equals_array == True:
        boolean = True
        break

    if boolean == False:
      Roots[idx] = [dp, Store_Self_Density[idx]]
  
  return Roots 

In [ ]:
def MakeSupposedRootsandClusternumsamelength(Supposed_Roots, labels, Tree_Structure, Store_Self_Density):
  CurrRootsWithSameLabel = dict()
  for i in range(len(Supposed_Roots)-1):
    for j in range(i+1, len(Supposed_Roots)):
      if labels[Supposed_Roots[i]] == labels[Supposed_Roots[j]]:
        if labels[Supposed_Roots[i]] in CurrRootsWithSameLabel:
          if Supposed_Roots[j] not in set(CurrRootsWithSameLabel[labels[Supposed_Roots[i]]]):
            CurrRootsWithSameLabel[labels[Supposed_Roots[i]]].append(Supposed_Roots[j])
        else:
          CurrRootsWithSameLabel[labels[Supposed_Roots[i]]] = [Supposed_Roots[i]]
          CurrRootsWithSameLabel[labels[Supposed_Roots[i]]].append(Supposed_Roots[j])

  # CurrRootsWithSameLabel = {1: [1,2,3]}
  if len(CurrRootsWithSameLabel) > 0:
    for key_label in CurrRootsWithSameLabel:
      MaxDensityTillNow = float("-inf")
      for idx in CurrRootsWithSameLabel[key_label]:
        if Store_Self_Density[idx] > MaxDensityTillNow:
          MaxDensityTillNow = Store_Self_Density[idx]
          MostDenseIdx = idx
      
      # Add the remaining siblings with same label as children of the most dense sibling
      for idx in CurrRootsWithSameLabel[key_label]:
        if idx != MostDenseIdx:
          Tree_Structure[MostDenseIdx].append(idx)
      
  return Tree_Structure

In [ ]:
# Name: Build Model

if 'Name: Build Model' not in SM:
    
    def BuildModel(data1, density, Hash_Map, labels, thresh, alpha_weight_avg, distances_ddcal, indices_ddcal, start_with, 
                   Store_Self_Density, Store_Radius_Density, Store_Parents, LLE_lookup):

        data_idx = 0
        cluster_num = start_with
        cluster_centers = []
        hold_cluster_val = cluster_num ####

        # For Storing Tree Structure
        Tree_Structure = dict()
        # Only Roots
        Roots = dict()
        # For Storing Radius and the Weighted Density of Ancestors including itself
        # Store_Radius_Density = dict()
        # For Storing parents
        # Store_Parents = dict()
        # For Storing Self Density
        ## Store_Self_Density = dict()


        for data_idx in range(len(data1)):  #O(n)

            if (data_idx + 1) % 100 == 0:
                print("**************************Computed {} Datapoints**************************".format(data_idx+1))
            idx = 0
            list_of_parents_queue = deque()
            list_of_parents_queue.appendleft([])


            if labels[density[data_idx][-1]] == 0: #<= 0:
                cluster_num += 1
                labels[density[data_idx][-1]] = cluster_num
                # print("CHECK1: ", density[data_idx][-1], labels[density[data_idx][-1]])
                can_form_cluster = [False]


                # Update information for the Parent node
                #   0                         1         2        3                    4
                # Distance-Density Value; Datapoint; Radius; k-nearest neighbors; idx_number

                K_mod = LLE_lookup[density[data_idx][-1]]      #O(1) Optimized One
                #print("K modified: ",K_mod) # Can be uncommented for better intuition

                #########################################
                # # New One
                # density[data_idx] = update_nbrs(data1, density[data_idx][-1], K_mod, algo)
                # temp_dict = {density[data_idx][-1]:density[data_idx]}
                # #########################################

                NovelClus_FF(density[data_idx], Hash_Map, data1, K_mod, list_of_parents_queue, labels, thresh, cluster_num, 
                             can_form_cluster, alpha_weight_avg, Tree_Structure, Roots, Store_Radius_Density, Store_Parents,
                            distances_ddcal, indices_ddcal, Store_Self_Density)

                Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

                # Check if cluster_num is same as the Supposed_Roots: if not then this function corrects it.
                # I am considering that len(Supposed_Roots) > or = cluster_num. Is this correct?
                if len(Supposed_Roots) != cluster_num:
                  Tree_Structure = MakeSupposedRootsandClusternumsamelength(Supposed_Roots, labels, Tree_Structure, Store_Self_Density)
                  Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

                Roots = AddRootsFromSupposedRoots(Supposed_Roots, Roots, data1, Store_Self_Density)
                # print("Supposed Roots: ", Supposed_Roots)
                if len(list(Supposed_Roots)) > len(Roots):
                    pass
                    # print("Error at BuildModel")
                    # print("Supposed Roots: ", Supposed_Roots)
                    # print("\nTree_Structure: \n", Tree_Structure)
                    # raise Exception("Number of Supposed Roots has become more than the actual number of roots")

                # # Visualization: For higher dimensions this time complexity can be ignored
                # if Dimension <= 2:
                #     plt.figure(figsize=(8,8))
                #     for i in range(len(data1)):
                #         if labels[i] == cluster_num:
                #             plt.scatter(data1[i][0],data1[i][1],c='pink')
                #         elif labels[i] == -1:
                #             plt.scatter(data1[i][0],data1[i][1], c='yellow')
                #         else:
                #             plt.scatter(data1[i][0], data1[i][1],c='blue')
                #     plt.scatter(density[data_idx][1][0], density[data_idx][1][1], c='red',s=200, marker = '+')
                #     plt.show()



                # If the last Parent Node cannot form a clusterof its own then it is a part of a cluster with only 1 datapoint
                # which means " a possible Anomaly"
                # then change its label to -1 and reduce the cluster num by 1
                if can_form_cluster[0] == False:
                    labels[density[data_idx][-1]] = -1
                    cluster_num = hold_cluster_val  ####
                else:
                    cluster_centers.append([density[data_idx][1].tolist(), cluster_num])
                    hold_cluster_val += 1  ####

                ## print("CC: ", cluster_centers)

        return Roots, cluster_centers, Store_Radius_Density, Store_Parents, Store_Self_Density, Tree_Structure             



    data1 = SM['data1']
    density = SM['density']
    Hash_Map = SM['Hash_Map']
    labels = SM['labels']
    thresh = SM['thresh']
    alpha_weight_avg = SM['alpha_weight_avg']
    with open('distances_ddcal.p', 'rb') as pfile1:
        distances_ddcal = pickle.load(pfile1)
    # distances_ddcal = SM['distances_ddcal']
    with open('indices_ddcal.p', 'rb') as pfile2:
        indices_ddcal = pickle.load(pfile2)
    # indices_ddcal = SM['indices_ddcal']
    Store_Self_Density = SM['Store_Self_Density']
    Store_Radius_Density = SM['Store_Radius_Density']
    Store_Parents = SM['Store_Parents']
    LLE_lookup = SM['LLE_lookup']
    

    
    start_with = 0
    Roots, cluster_centers, Store_Radius_Density, Store_Parents, Store_Self_Density, Tree_Structure = BuildModel(data1, density, 
                        Hash_Map, labels, thresh, alpha_weight_avg, distances_ddcal, indices_ddcal, start_with, 
                        Store_Self_Density, Store_Radius_Density, Store_Parents, LLE_lookup)

    Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

    # Check if cluster_num is same as the Supposed_Roots: if not then this function corrects it.
    # I am considering that len(Supposed_Roots) > or = cluster_num. Is this correct?
    cluster_num = len(cluster_centers)
    if len(Supposed_Roots) != cluster_num:
      Tree_Structure = MakeSupposedRootsandClusternumsamelength(Supposed_Roots, labels, Tree_Structure, Store_Self_Density)
      Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

    Roots = AddRootsFromSupposedRoots(Supposed_Roots, Roots, data1, Store_Self_Density)
    # print("Supposed Roots: ", Supposed_Roots)
    if len(list(Supposed_Roots)) > len(Roots):
        pass
        # print("Error at BuildModel Called Function")
        # print("Supposed Roots: ", Supposed_Roots)
        # print("\nTree_Structure: \n", Tree_Structure)
        # raise Exception("Number of Supposed Roots has become more than the actual number of roots")
        
    SM['Roots'] = Roots
    SM['cluster_centers'] = cluster_centers
    SM['Store_Radius_Density'] = Store_Radius_Density
    SM['Store_Parents'] = Store_Parents
    SM['Store_Self_Density'] = Store_Self_Density
    SM['Tree_Structure'] = Tree_Structure
    SM['Supposed_Roots'] = Supposed_Roots
    SM['cluster_num'] = cluster_num
         
    SM['Name: Build Model'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Display Cluster Centers

if 'Name: Display Cluster Centers' not in SM:
    cluster_centers = SM['cluster_centers']

    # Following print statements can be uncommented for better intuition
    print("Current Cluster Centers: \n")
    # print(cluster_centers)
    print(len(cluster_centers))
    
    SM['Name: Display Cluster Centers'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# print(Tree_Structure)

In [ ]:
def GetIdxFromWholeDataset(whole_dataset, dp):
    for i in range(len(whole_dataset)):
        item = whole_dataset[i]
        comparison = item == dp
        equal_arrays = comparison.all()
        if equal_arrays == True:
            return i

In [ ]:
# Function corrects the wrong clustering
def Correcting_wrong_clustering(labels, cluster_centers, data1, Roots, Tree_Structure):  
    cluster_labels = []
    for key in labels:
        cluster_labels.append(labels[key])
    
    Hash_Map_2 = dict()
    X_label = dict()
    
    for i in range(len(data1)): 
        if labels[i] == -1:
            continue
        if labels[i] in Hash_Map_2:
            Hash_Map_2[labels[i]] += 1
            X_label[labels[i]] = X_label[labels[i]] + [data1[i].tolist()]
        else:
            Hash_Map_2[labels[i]] = 1
            X_label[labels[i]] = [data1[i].tolist()]
            
    # print("\nFF1: HASH MAP\n", Hash_Map_2)

    for key in Hash_Map_2:
        temp = Hash_Map_2[key]
        hold_idx = 0
        for idx in range(len(cluster_centers)):
            if cluster_centers[idx][1] == key:
                hold_idx = idx
                break
        ## print("FF3: ", temp, cluster_centers)
        Hash_Map_2[key] = [temp, cluster_centers[hold_idx][0]]
        
    # print("FF2: Hash_Map_2\n",Hash_Map_2,"\n") ######
        
    # Function to find Intra Cluster Distance    
    def IntraClusterMetric(X):
        Intra_Clus_Dist = 0
        for i in range(len(X)):
            Intra_Clus_Dist += (sum(euclidean_distances(X, [X[i]]))[0])/(len(X)-1)
        return Intra_Clus_Dist/(len(X)-1)
           
    
    for key in X_label:
        X = X_label[key]
        Intra_metric = IntraClusterMetric(X)
        ## print("F: ", Hash_Map_2[key], "F1: ", [Intra_metric])
        Hash_Map_2[key] = Hash_Map_2[key] + [Intra_metric]  
    
    # print("Hash_Map_2\n",Hash_Map_2,"\n")
    
    clusnum_card = sorted(Hash_Map_2.items(), key=lambda kv:(kv[1], kv[0]), reverse = False)
    cluster_center_new = []
    for ele in clusnum_card:
        cluster_center_new.append(np.array(ele[1][1]))   

    ## Following print statements can be uncommented for better intuition
    # print(Hash_Map_2)
    # print("\nAfter Sorting ")
    # print(clusnum_card)
    # print("\nOnly cluster centers")
    # print(cluster_center_new)
    
    
    if len(cluster_center_new) < len(cluster_centers):
        K1 = len(cluster_center_new)
    else:
        K1 = len(cluster_centers)
    nbrs = NearestNeighbors(n_neighbors=K1, algorithm='auto').fit(cluster_center_new)
    distances, indices = nbrs.kneighbors(cluster_center_new)
    #print(indices,"\n")
    #print(distances)

    
    def GetCutoffDistMod(cluster_centers):
        percent = 0.7
        i = 0
        j = 1
        Count = 0
        Sum = 0
        while i < len(cluster_centers)-1:
            while j < len(cluster_centers):
                Sum += np.linalg.norm(cluster_centers[i] - cluster_centers[j]) #math.dist(cluster_centers[i], cluster_centers[j])
                j += 1
                Count += 1
            i += 1
            j = i + 1
        Count = max(Count, 1)
        return math.ceil(Sum/Count)*percent
    
    
    #print(clusnum_card)
    
    #print("\n\n")
    dist_val = GetCutoffDistMod(cluster_center_new)
    val2 = 20
    val3 = 15
    percent1 = 0.8
    percent2 = 0.16
    percent3 = 0.10
    percent4 = 0.02

    # Format of clus_to_change: [curr_clus, clus_to_be]
    clus_to_change = []
    for i in range(len(clusnum_card)-1):
        # We are not considering till the last term since,
        # last term is the most dense and biggest among all 
        # the other clusters. It cannot be merged with others
        
        j = 1
        cardinality_of_self = Hash_Map_2[clusnum_card[i][0]][0]
        Intra_Clus_Self = Hash_Map_2[clusnum_card[i][0]][2]
        cardinality_of_neighbor = Hash_Map_2[clusnum_card[indices[i][j]][0]][0]
        Intra_Clus_Neigh = Hash_Map_2[clusnum_card[indices[i][j]][0]][2]
        
        
        while cardinality_of_self >= cardinality_of_neighbor and j < len(clusnum_card)-1:
            #print("YES")
            j += 1
            #print("j=",j)
            cardinality_of_neighbor = Hash_Map_2[clusnum_card[indices[i][j]][0]][0]  #-1
            Intra_Clus_Neigh = Hash_Map_2[clusnum_card[indices[i][j]][0]][2]

        # cardinality_of_self = Hash_Map_2[clusnum_card[i][0]]
        # cardinality_of_neighbor = Hash_Map_2[clusnum_card[indices[i][j]][0]][0]
        # print(cardinality_of_neighbor)
    
        ## Following print statements can be uncommented for better intuition
        # print("Cluster number", clusnum_card[i][0], "Cardinality ", Hash_Map_2[clusnum_card[i][0]][0])
        # print("Neighbor Cluster number", clusnum_card[indices[i][j]][0], "Cardinality ", Hash_Map_2[clusnum_card[indices[i][j]][0]][0])
        # print("Intra_Clus_Self", Intra_Clus_Self, "Intra_Clus_Neigh ", Intra_Clus_Neigh)
        # print("Intra Val Lower =", (Intra_Clus_Neigh - percent1*Intra_Clus_Neigh), "Intra Val Higher =", (Intra_Clus_Neigh + percent1*Intra_Clus_Neigh))
        # print("Percent1*CNeigh", int(percent1*cardinality_of_neighbor))
        # print("Percent2 =", (cardinality_of_self/cardinality_of_neighbor))
        # print("val3", val3)
        # print("Or condition", abs(cardinality_of_self - cardinality_of_neighbor), "val2", val2)
        # print("Distances between self and neighbor", int(round(distances[i][j],0)), "Dist_val", dist_val)
        
        
        # Using a boolean merge to keep a track if the self cluster is merged or not
        # Unless and until the cluster is merged, it is processed through every if condition
        merged = False
        
        # If cardinality of self and cardinality of neighbor are both less than a given threshold
        # where the threshold is very very small; say 10
        if merged == False:
            # print("Here Card1: ", cardinality_of_self, cardinality_of_neighbor)
            if cardinality_of_self <= val2 and cardinality_of_neighbor <= val2:
                # Distance checking here isn't necessary
                curr_clus = clusnum_card[i][0]
                clus_to_be = clusnum_card[indices[i][j]][0]
                clus_to_change.append([curr_clus, clus_to_be])
                Hash_Map_2[clusnum_card[indices[i][j]][0]][0] += cardinality_of_self
                #print("Merged 1st condition")
                merged = True  # made True so that we donot need to process it over other if conditions
        
        # If cardinality of self is less than 16% of cardinality of neighbor 
        if merged == False:
            # print("Here Card2: ", cardinality_of_self, cardinality_of_neighbor, (cardinality_of_self/cardinality_of_neighbor))
            if (cardinality_of_self/cardinality_of_neighbor) <= percent2 and cardinality_of_self <= val3:
                # print("F1: ", int(round(distances[i][j],0)), dist_val)
                if int(round(distances[i][j],0)) <= dist_val:
                    # print("F")
                    curr_clus = clusnum_card[i][0]
                    clus_to_be = clusnum_card[indices[i][j]][0]
                    clus_to_change.append([curr_clus, clus_to_be])
                    Hash_Map_2[clusnum_card[indices[i][j]][0]][0] += cardinality_of_self
                    #print("Merged 2nd condition")
                    merged = True  # made True so that we donot need to process it over other if conditions
                
        # If Cardinality of self > val3 but is less than percent6 of the cardinality of neighbor
        if merged == False:
            # print("Here Card3: ", cardinality_of_self, cardinality_of_neighbor)
            if val3 < cardinality_of_self <= val3*2.7 and cardinality_of_self/cardinality_of_neighbor <= percent3:
                Intra_Upper_Limit = Intra_Clus_Neigh + percent1*Intra_Clus_Neigh
                Intra_Lower_Limit = (Intra_Clus_Neigh - percent1*Intra_Clus_Neigh)
                if (Intra_Lower_Limit <= Intra_Clus_Self <= Intra_Upper_Limit and int(round(distances[i][j],0)) <= dist_val):
                    curr_clus = clusnum_card[i][0]
                    clus_to_be = clusnum_card[indices[i][j]][0]
                    clus_to_change.append([curr_clus, clus_to_be])
                    Hash_Map_2[clusnum_card[indices[i][j]][0]][0] += cardinality_of_self
                    #print("Merged 3rd condition")
                    merged = True  # made True so that we donot need to process it over other if conditions          
        
        # If Cardinality of self is less than percent4 of the cardinality of neighbor; where percent4 ~ 2 percent
        if merged == False:
            # print("Here Card4: ", cardinality_of_self, cardinality_of_neighbor)
            if cardinality_of_self/cardinality_of_neighbor <= percent4:
                # Here we don't need to check the distance and intra cluster density;
                # The self cluster is very very small compared to the neighbor cluster.
                # So just merge them
                curr_clus = clusnum_card[i][0]
                clus_to_be = clusnum_card[indices[i][j]][0]
                clus_to_change.append([curr_clus, clus_to_be])
                Hash_Map_2[clusnum_card[indices[i][j]][0]][0] += cardinality_of_self
                # print("Merged 4th condition")
                merged = True  # made True so that we donot need to process it over other if conditions      
        
        if merged == False: # Some merges by this logic as well
            # print("Here Card5: ", cardinality_of_self, cardinality_of_neighbor)
            if (cardinality_of_self <= cardinality_of_neighbor and cardinality_of_self <= val3) or (abs(cardinality_of_self - cardinality_of_neighbor) <= val2 and cardinality_of_self <= val3):
                Intra_Upper_Limit = Intra_Clus_Neigh + percent1*Intra_Clus_Neigh
                Intra_Lower_Limit = (Intra_Clus_Neigh - percent1*Intra_Clus_Neigh)
                if (Intra_Lower_Limit <= Intra_Clus_Self <= Intra_Upper_Limit and int(round(distances[i][j],0)) <= dist_val):
                    curr_clus = clusnum_card[i][0]
                    clus_to_be = clusnum_card[indices[i][j]][0]
                    clus_to_change.append([curr_clus, clus_to_be])
                    Hash_Map_2[clusnum_card[indices[i][j]][0]][0] += cardinality_of_self
                    # print("Merged 5th condition")
                    merged = True  # made True so that we donot need to process it over other if conditions      

        # print("\n")
    print("\nChanged Clusters are (If any): [initial cluster --> New Cluster]")
    print(clus_to_change)
    
    
    for pair in clus_to_change:
        # print("F10: ", cluster_centers, pair[0], cluster_centers[pair[0]-1][0])
        # print("F11: ", cluster_centers, pair[1], cluster_centers[pair[1]-1][0])
        # print("Change: ", cluster_centers[pair[0]-1][0], "To: ", cluster_centers[pair[1]-1][0])

        idx_of_changing_cluster = GetIdxFromWholeDataset(data1, cluster_centers[pair[0]-1][0])
        changingtolabel = pair[1]
        # print("Change to label: ", changingtolabel)
        for key in Tree_Structure:
            if labels[key] == changingtolabel:
                # print("Entering: ", idx_of_changing_cluster)
                Tree_Structure[key].append(idx_of_changing_cluster)
                break
                
        keystodel = []
        for key in Roots:
            if labels[key] == pair[0]:
                keystodel.append(key)
        
        for key in keystodel:
            del Roots[key]
        ## print(Roots)
                
                
    for key in labels:
        for item in clus_to_change:
            if labels[key] == item[0]:
                labels[key] = item[1]
    
    return Roots, labels, Tree_Structure



In [ ]:
def RemoveDulplicatePositions(Tree_Structure, Store_Self_Density):
    for parent_idx in Tree_Structure:
        child_idxs = Tree_Structure[parent_idx]
        for child_idx in child_idxs:
            if child_idx in Tree_Structure:
                if parent_idx in set(Tree_Structure[child_idx]):
                    Tree_Structure[child_idx].remove(parent_idx)
                    # print("DUPLICATE FOUND!!!!")
    return Tree_Structure


def CorrectOneChildTwoParents(Tree_Structure, Store_Self_Density):   
    Hash_map = dict()
    # print(Tree_Structure)
    for parent_idx in Tree_Structure:
        for child_idx in Tree_Structure[parent_idx]:
            if child_idx in Hash_map:
                parent1_idx = Hash_map[child_idx]
                parent2_idx = parent_idx
                # print("F", parent1_idx, parent2_idx)
                if Store_Self_Density[parent1_idx] > Store_Self_Density[parent2_idx]:
                    Tree_Structure[parent2_idx].remove(child_idx)
                else:
                    Hash_map[child_idx] = parent2_idx
                    Tree_Structure[parent1_idx].remove(child_idx)
            
            else:
                Hash_map[child_idx] = parent_idx
    return Tree_Structure
            
        
                

In [ ]:
## For Properly Aligning the Tree Structure, 
## i.e., Parents are more denser than child (less density value for parent)

## This function checks whether or not the Tree Structure adheres to our intuition
def CheckTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, K):
    Same_label_points = []
    for i in range(len(data1)):
        if labels[i] == K:
            Same_label_points.append(i)    ## To store the points having similar labels
    # print(Same_label_points)
    Idx_Density = []
    for idx in Same_label_points:   
        Idx_Density.append([idx, Store_Self_Density[idx]])
    # print("Without sort: ", Idx_Density)
    Idx_Density.sort(key=itemgetter(1), reverse=True) #False
    # print("With sort: ", Idx_Density)

    Tree_Structure_Similar_label = {}
    for pair in Idx_Density:
        try:
            Tree_Structure_Similar_label[pair[0]] = Tree_Structure[pair[0]]
        except:
            Tree_Structure_Similar_label[pair[0]] = []
    # print("Tree_Structure_Similar_label: ", Tree_Structure_Similar_label)

    Wrong_child_parent = []
    for parent_idx in Tree_Structure_Similar_label:
        if len(Tree_Structure_Similar_label[parent_idx]) != 0:
            parent_density = Store_Self_Density[parent_idx]             
            for child_idx in Tree_Structure_Similar_label[parent_idx]:
                child_density = Store_Self_Density[child_idx]           
                if child_density > parent_density:   ## Since we are considering density as it really means, so 
                                                     ## child node should be less dense than the parent node
                    Wrong_child_parent.append([child_idx, parent_idx])
    return Wrong_child_parent



## This function swaps Current Parent and New Parent in the Tree Structure
def SwapParentChild(Curr_Parent, New_Parent, Tree_Structure):
    Childs_Curr_Parent = Tree_Structure[Curr_Parent]
    if New_Parent in Tree_Structure:
        Childs_New_Parent = Tree_Structure[New_Parent]
    else:
        Childs_New_Parent = []
    for i in range(len(Childs_Curr_Parent)):
        if Childs_Curr_Parent[i] == New_Parent:
            Childs_Curr_Parent[i] = Curr_Parent
            break
    ## Uncomment for better intuition 
    # print("CCP: ", Childs_Curr_Parent)
    # print("CNP: ", Childs_New_Parent)
    
    Tree_Structure[New_Parent] = Childs_Curr_Parent
    Tree_Structure[Curr_Parent] = Childs_New_Parent
    
    for node in Tree_Structure:
        if node != New_Parent:
            Child_nodes = set(Tree_Structure[node])
            if Curr_Parent in Child_nodes:
                Child_nodes.remove(Curr_Parent)
                Child_nodes.add(New_Parent)
                Tree_Structure[node] = list(Child_nodes)
                ## Uncomment for better intuition 
                # print("Changed:", Tree_Structure[node])
                
    return Tree_Structure


def AdjustRoots(Curr_Parent_idx, New_Parent_idx, Store_Self_Density, Roots, data1, Incoming_points):
    if Curr_Parent_idx in Roots:
        # print("ROOTS PREV: ", Roots)
        del Roots[Curr_Parent_idx]
        try:
            Roots[New_Parent_idx] = [data1[New_Parent_idx], Store_Self_Density[New_Parent_idx]]
        except:
            Roots[New_Parent_idx] = [Incoming_points[New_Parent_idx], Store_Self_Density[New_Parent_idx]]
        print("--> Roots Have Been Adjusted")
        # print("ROOTS NOW: ", Roots)
                
                
                
## This function analyzes the Tree Structure and makes necessary amendments
def RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Incoming_points, Roots):
  K = 1
  while K <= cluster_num:
    hold = []
    while True:     
      Wrong_child_parent_list = CheckTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, K)
      if Wrong_child_parent_list in hold:
        break
      hold.append(Wrong_child_parent_list)
      # print("Wrong_child_parent_list: ", Wrong_child_parent_list, "len(Wrong_child_parent_list): ", len(Wrong_child_parent_list)) ###################
      if len(Wrong_child_parent_list) == 0:
        break
      else:
        Wrong_child_parent = deque() ####################
        for pair1 in Wrong_child_parent_list:
            Wrong_child_parent.appendleft(pair1)
        # print("I am here!!")
        # print("Wrong_child_parent: ", Wrong_child_parent)
        
        # while Wrong_child_parent:
        pts_with_same_parent = []
        temp = Wrong_child_parent.pop()
        pts_with_same_parent.append(temp) #[0]) # If there is only one pair of wrong child and parent
                                                            # that is taken care here.

        # If there are multiple pairs of wrong child and parent
        while Wrong_child_parent and Wrong_child_parent[-1][1] == temp[1]:
            temp1 = Wrong_child_parent.pop()
            pts_with_same_parent.append(temp1)
#                 if len(Wrong_child_parent) > 1:
#                     i = 1
#                     while i < len(Wrong_child_parent) and Wrong_child_parent[i][1] == Wrong_child_parent[i-1][1]:
#                         pts_with_same_parent.append(Wrong_child_parent[i])
#                         i += 1
        # print("Same Parents: ", pts_with_same_parent)
        parent_idx = pts_with_same_parent[0][1]
        # print("Parent: ", parent_idx)
        Hold_Child_Idx_Density = [None, float("-inf")]
        for pair in pts_with_same_parent:
            density_child_node = Store_Self_Density[pair[0]]
            if density_child_node > Hold_Child_Idx_Density[1]:  # <
                Hold_Child_Idx_Density[0] = pair[0] 
                Hold_Child_Idx_Density[1] = density_child_node
        # print("Hold_Child_Idx_Density ", Hold_Child_Idx_Density)
        child_idx = Hold_Child_Idx_Density[0]
        # print("Parent idx: ", parent_idx)
        # print("Child idx: ", child_idx)
        # print("Before: ", Tree_Structure[parent_idx], parent_idx)
        Tree_Structure = SwapParentChild(parent_idx, child_idx, Tree_Structure)
        # print("After: ", Tree_Structure[child_idx], child_idx)
        AdjustRoots(parent_idx, child_idx, Store_Self_Density, Roots, data1, Incoming_points)  ################# 
        # print("Doing all Wrong_child_parent: ", Wrong_child_parent)
    
      Tree_Structure = RemoveDulplicatePositions(Tree_Structure, Store_Self_Density)
      Tree_Structure = CorrectOneChildTwoParents(Tree_Structure, Store_Self_Density)

      Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

      # Check if cluster_num is same as the Supposed_Roots: if not then this function corrects it.
      # I am considering that len(Supposed_Roots) > or = cluster_num. Is this correct?
      if len(Supposed_Roots) != cluster_num:
        Tree_Structure = MakeSupposedRootsandClusternumsamelength(Supposed_Roots, labels, Tree_Structure, Store_Self_Density)
        Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

      Roots = AddRootsFromSupposedRoots(Supposed_Roots, Roots, data1, Store_Self_Density)
      # print("Supposed Roots: ", Supposed_Roots)
      if len(list(Supposed_Roots)) > len(Roots):
          pass
          #print("Error at Rearrange Tree Structure")
          #print("Supposed Roots: ", Supposed_Roots)
          # print("\nTree_Structure: \n", Tree_Structure)
          # raise Exception("Number of Supposed Roots has become more than the actual number of roots")
        
    print("****************Analyzed and Adjusted Datapoints of Cluster {}******************".format(K))

    # Increment K
    K += 1
  
  RemoveNodesWithNoChild(Tree_Structure)

In [ ]:
def RemoveNodesWithNoChild(Tree_Structure):
    keystoremove = []
    
    for key in Tree_Structure:
        if len(Tree_Structure[key]) == 0:
            keystoremove.append(key)
    
    for key in keystoremove:
        del Tree_Structure[key]
        

In [ ]:
# Name: Correcting Wrong Clustering

if 'Name: Correcting Wrong Clustering' not in SM:
    labels = SM['labels']
    cluster_centers = SM['cluster_centers']
    data1 = SM['data1']
    labels = SM['labels']
    Roots = SM['Roots']
    Tree_Structure = SM['Tree_Structure']

    # print(cluster_centers)
    # # Correct the wrong clusters
    Roots, labels, Tree_Structure = Correcting_wrong_clustering(labels, cluster_centers, data1, Roots, Tree_Structure)
    
    SM['Roots'] = Roots
    SM['labels'] = labels
    SM['Tree_Structure'] = Tree_Structure

    SM['Name: Correcting Wrong Clustering'] = True

    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Correcting Labels2

if 'Name: Correcting Labels2' not in SM:


    # This returns cluster numbers in a non-missing sorted order
    # Before this function many clusters may be removed in the previous step
    # So, to keep everything ordered, this function makes sure all clusters have a number
    # and the number starts from 1 all the way to n without any missing number in between
    def get_count_in_label(labels):
        table = {value:0 for value in labels}
        for ele in labels:
            table[ele] += 1
        key_list = table.keys()
        #sorted(key_list, reverse = False)
        for ele in sorted(key_list, reverse = False):
            print(ele,":",table[ele])

    def get_label_list(hash_map):
        label_list = []
        for key in hash_map.keys():
            if hash_map[key] > 0:
                label_list.append(hash_map[key])
            #else:
            #    get_labels.append(hash_map[key])
        #print(get_labels)
        return label_list    


    def find_missing_idx(hash_map):
        max_idx = float("-inf")
        for key in hash_map:
            if hash_map[key] > max_idx:
                max_idx = hash_map[key]
        #max_idx = max(hash_map.keys())

        min_idx = float("inf")
        for key in hash_map:
            if hash_map[key] < min_idx and hash_map[key] != -1:
                min_idx = hash_map[key]
        #min_idx = 0

        ########### print(min_idx, max_idx)                     

        label_list = get_label_list(hash_map)

        missing_idx = []
        count = 1
        while count <= max_idx:
            if count not in label_list:
                missing_idx.append(count)
            count += 1

        ########## print(missing_idx)  
        return missing_idx

    def correct_the_idx(hash_map): #labels):
        missing_idx = find_missing_idx(hash_map) #labels)

        # Can be uncommented for better intuition
        # print(missing_idx)

        while missing_idx:
            for i in missing_idx:
                for key in hash_map:# in range(len(labels)):
                    if hash_map[key] == i + 1: #labels[j] == i+1:
                        hash_map[key] = hash_map[key] - 1 #labels[j] = labels[j] - 1

            #get_count_in_label(labels)

            #print("\n")
            #coorect_the_idx(labels)

            missing_idx = find_missing_idx(hash_map) #labels)


    #missing_idx = correct_the_label_idx(labels)
    #print(missing_idx)

    #get_count_in_label(labels)
    # print("\n\n")
    labels = SM['labels']
    correct_the_idx(labels)
    #get_count_in_label(labels)
    #find_missing_idx(hash_map)
    #print(hash_map)

    #label_l = get_label_list(hash_map)
    #get_count_in_label(label_l)
    
    SM['labels'] = labels
    SM['Name: Correcting Labels2'] = True


else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Before Post Processing Display

if 'Name: Before Post Processing Display' not in SM:
    labels = SM['labels']
    data1 = SM['data1']

    label_list = []
    anomaly_list = []
    max_val = float("-inf")
    for key in labels:
        if max_val < labels[key]:
            max_val = labels[key]
        if labels[key] == -1:
            anomaly_list.append(data1[key])
        else:
            label_list.append(labels[key])
    cluster_num = max_val
    #print(label_list)
    print("\n---------------------Before Outlier Post Processing----------------------")
    print("Total number of datapoints =", len(data1))
    print("Number of Non-Anomalies =", len(label_list))
    print("Number of Anomalies =", len(anomaly_list))
    print("Number of Cluster(s) =", cluster_num)
    
    SM['data1'] = data1
    SM['label_list'] = label_list
    SM['anomaly_list'] = anomaly_list
    SM['cluster_num'] = cluster_num
    
    SM['Name: Before Post Processing Display'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Display1

if 'Name: Display1' not in SM:
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    
    if Dimension <= 2:
        plt.figure(figsize=(8,8))

        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']

        if cluster_num <= len(color):
            for i in range(len(data1)):
                if labels[i] != -1:
                    plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                else:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
        else:
            for i in range(len(data1)):
                if labels[i] == -1:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                else:
                    plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')

        plt.xlabel('X-Coordinates')
        plt.ylabel('Y-Coordinates')
        plt.title('Datapoints Visualization Before Outlier Post Processing')
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
    
    SM['Name: Display1'] = True
    SM.sync()

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Outlier Post Processing

if 'Name: Outlier Post Processing' not in SM:

    def OutlierPostProcessing(LLE_lookup, algo, data1, labels, percent5, alpha_weight_avg, 
                              Store_Radius_Density, distances_ddcal, indices_ddcal, Store_Parents, Store_Self_Density,
                             Tree_Structure):
        # Select a predetermined value of K
        # Find the K nearest Neighbors of an outlier
        # For each of those neighbors:
        #     Find their Intracluster distance with the same K value
        #     If the Intracluster distance of outlier is within +-5% of the intracuster distance of the neighbor,
        #     and the neighbor point is not an outlier, 
        #     then consider the outlier to be a part of the smae cluster that the neighbor point belongs.

        ##nbrs = NearestNeighbors(n_neighbors=K, algorithm=algo).fit(data1)
        ##distances, indices = nbrs_ddcal.kneighbors(data1)

        for i in range(len(data1)):
            if labels[i] == -1:
                # print("Here i: ", i)
                # Predicted Outlier, so find its K nearest neighbors
                #############################################################
                Opt_K = LLE_lookup[i]
                # print("Opt k: ", Opt_K)
                indices = indices_ddcal[i][1:Opt_K]
                # print("Indices: ", indices)
                # nbrs = NearestNeighbors(n_neighbors=Opt_K, algorithm=algo).fit(data1)              ########################
                # _, indices = nbrs.kneighbors(data1)
                #############################################################

                PredOutlierNNidx = indices               ##[i]
                # print("IDXs: ", PredOutlierNNidx)
                PredOutlierNN = []
                for idx in PredOutlierNNidx:
                    PredOutlierNN.append(data1[idx])
                # Find the intracluster distance for the Predicted Outlier
                PredOutlierICD = IntraClusterDistance(PredOutlierNN)
                # print("Outlier", i, PredOutlierICD)


                # For each of the outlier's K neighbors
                for idx in PredOutlierNNidx[1:]:

                    # Check if the neighbor is an outlier or inlier
                    if labels[idx] != -1:
                        # print("Not an Outlier")
                        pruned_k = LLE_lookup[idx]
                        NNeighboridx = indices_ddcal[i][1:pruned_k]
                        NNeighbor = []
                        for idx1 in NNeighboridx:
                            NNeighbor.append(data1[idx1])

                        # Find each neighbor's IntraClusterDistance(ICD)
                        NNICD = IntraClusterDistance(NNeighbor)
                        #print("NNICD: ", NNICD)

                        # Check if outlier's ICD is about +-5% of neighbor's ICD
                        # print("Lower: ",(NNICD-percent5*NNICD), "Upper: ",(NNICD+percent5*NNICD), "Self: ",PredOutlierICD)
                        if (NNICD-percent5*NNICD) <= PredOutlierICD <= (NNICD+percent5*NNICD):
                            # Change the label of the Predicted Outlier 'i' to the 
                            # label of the 1st Nearest Neighbor which satifies the condition
                            labels[i] = labels[idx]
                            # print("Changed\n")
                            if idx not in Tree_Structure:
                                Tree_Structure[idx] = [i]
                            else:
                                Tree_Structure[idx].append(i)

                            # Update its Store_Radius_Density Value
                            # print("IDX:",idx)

                            # # Old
                            # Radius = Store_Radius_Density[idx][0]
                            # neigh = NearestNeighbors(radius = Radius)
                            # neigh.fit(data1)
                            # rng = neigh.radius_neighbors([data1[i]])

                            # New
                            Radius = Store_Radius_Density[idx][0]
                            neigh_dist = []
                            neigh_idxs = []
                            rng = [[],[]]
                            for k in range(1, len(distances_ddcal[i])):
                                if distances_ddcal[i][k] < Radius:
                                    neigh_dist.append(distances_ddcal[i][k])
                                    # print("NIs: ", indices_ddcal[Child_Idx][i])
                                    neigh_idxs.append(np.asarray(int(indices_ddcal[i][k])))
                                else:
                                    break
                            rng[0].append(np.asarray(neigh_dist))
                            rng[1].append(np.asarray(neigh_idxs))
                            rng = tuple(rng)
                            # print("New RNG:", len(rng[0][0]))
                            ####


                            # Check if there is noneighbour within the radius
                            # If yes then the density of the recently changed datapoint is 0
                            if len(rng[0][0]) == 0:
                                Density_Changed_Dp = 0
                            else:
                                #Density_Parent = sum(rng[0][0])/len(rng[0][0])       ########################
                                Radius = max(rng[0][0])
                                Density_Changed_Dp = sum(rng[0][0])/(math.pi*Radius*Radius)

                            list_of_parents = Store_Parents[idx] + [Density_Changed_Dp] 
                            data_parent = np.array(list_of_parents) #, dtype=np.float64)
                            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                            if len(rng[0][0]) == 0:
                                Radius_new = 0
                            else:
                                Radius_new = max(rng[0][0])
                            Store_Radius_Density[i] = [Radius_new, weighted_density]
                            Store_Parents[i] = list_of_parents
                            Store_Self_Density[i] = Density_Changed_Dp

                            break


    def IntraClusterDistance(X):
            Intra_Clus_Dist = 0
            for i in range(len(X)):
                Intra_Clus_Dist += (sum(euclidean_distances(X, [X[i]]))[0])/(len(X)-1)
            return Intra_Clus_Dist/(len(X)-1)



    # KVal = int(0.01*len(data1)) #5 #0.03 
    LLE_lookup = SM['LLE_lookup']
    algo = SM['algo']
    data1 = SM['data1']
    labels = SM['labels']
    alpha_weight_avg = SM['alpha_weight_avg']
    Store_Radius_Density = SM['Store_Radius_Density']
    Tree_Structure = SM['Tree_Structure']
    Store_Parents = SM['Store_Parents']
    with open('distances_ddcal.p', 'rb') as pfile1:
        distances_ddcal = pickle.load(pfile1)
    # distances_ddcal = SM['distances_ddcal']
    with open('indices_ddcal.p', 'rb') as pfile2:
        indices_ddcal = pickle.load(pfile2)
    # indices_ddcal = SM['indices_ddcal']
    Store_Self_Density = SM['Store_Self_Density']
    OutlierPostProcessing(LLE_lookup, algo, data1, labels, percent5, alpha_weight_avg, 
                          Store_Radius_Density, distances_ddcal, indices_ddcal, Store_Parents, Store_Self_Density,
                         Tree_Structure)
    
    SM['Name: Outlier Post Processing'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: After Post Processing Display

if 'Name: After Post Processing Display' not in SM:
    
    labels = SM['labels']
    data1 = SM['data1']
    
    label_list = []
    anomaly_list = []
    max_val = float("-inf")
    for key in labels:
        if max_val < labels[key]:
            max_val = labels[key]
        if labels[key] == -1:
            anomaly_list.append(data1[key])
        else:
            label_list.append(labels[key])
    cluster_num = max_val
    #print(label_list)
    print("\n------------ After Outlier Post Processing ------------")
    print("Total number of datapoints =", len(data1))
    print("Number of Non-Anomalies =", len(label_list))
    print("Number of Anomalies =", len(anomaly_list))
    print("Number of Cluster(s) =", cluster_num)
    
    SM['data1'] = data1
    SM['label_list'] = label_list
    SM['anomaly_list'] = anomaly_list
    SM['cluster_num'] = cluster_num
    
    SM['Name: After Post Processing Display'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Display2

if 'Name: Display2' not in SM:
    
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    
    if Dimension <= 2:
        plt.figure(figsize=(8,8))

        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']

        if cluster_num <= len(color):
            for i in range(len(data1)):
                if labels[i] != -1:
                    plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                else:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
        else:
            for i in range(len(data1)):
                if labels[i] == -1:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                else:
                    plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')

        plt.xlabel('X-Coordinates')
        plt.ylabel('Y-Coordinates')
        plt.title('Datapoints Visualization After Outlier Post Processing')
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
    SM['Name: Display2'] = True
    SM.sync()
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: RearrangeTreeStructure1

if 'Name: RearrangeTreeStructure1' not in SM:
    Tree_Structure = SM['Tree_Structure']
    data1 = SM['data1']
    labels = SM['labels']
    Store_Self_Density = SM['Store_Self_Density']
    cluster_num = SM['cluster_num']
    Roots = SM['Roots']
    
    RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, data1, Roots)
    
    SM['Name: RearrangeTreeStructure1'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# if Clustering:
#     # Performance Evaluation
#     # Ground_Truth_data_path=r"E:\ISI\GROUND TRUTHS CLUSTERING\a2_gt.csv"
#     with open(Ground_Truth_data_path, 'r') as f:
#         reader = csv.reader(f, delimiter=',')
#         data_f = list(reader)

#     # Clusters from Ground Truth Data
#     clustersFromGT = []
#     clustersFromGT.append(0)

#     count = 1
#     temp = []
#     temp.append(int(data_f[0][-1]))

#     for i in range(1, len(data_f)):
#         if int(data_f[i][-1]) != int(data_f[i-1][-1]) and int(data_f[i][-1]) not in temp:
#             temp.append(int(data_f[i][-1]))
#             clustersFromGT.append(i)
#             count += 1
#     clustersFromGT.append(i+1)


#     def EvaluateClustering(labels_PC, n_e_val_0, data_path, clustersFromGT, data1, labels_Idxs):
#         with open(data_path, 'r') as f:
#             reader = csv.reader(f, delimiter=',')
#             data = list(reader)
#             data = np.array(data).astype(float)

#         d=2
#         row=len(data)
#         dimension=len(data[0])
#         neighbors=4
#         W=np.zeros((row, row))

#         for i in range(row):
#             D_i=np.array(data-data[i, :])
#             distance=(D_i**2).sum(1)
#             nearest_neighbor=np.argsort(distance)[1:(neighbors+1)]
#             D_nbrs=D_i[nearest_neighbor, :]
#             Q=np.dot(D_nbrs, D_nbrs.T)
#             t=np.trace(Q)
#             r=0.001*t
#             if(neighbors>=dimension):
#                 Q=Q+(r*np.identity(neighbors))
#             w=np.linalg.solve(Q, np.ones(neighbors))
#             w=w/sum(w)
#             W[i, nearest_neighbor]=w

#         I=np.identity(row)
#         M=I-W

#         U, S, Vt = np.linalg.svd(M)
#         e_val=S**2
#         e_val_0=np.array(e_val<10**-20)


#         uniq=np.array(np.unique(labels_PC))
#         confusion_mat=np.zeros((n_e_val_0,n_e_val_0))
#         status=np.zeros((len(uniq)))
#         # print("\nSTATUS_INIT",status,"\n")                                                                    

#         x = 0
#         y = 1
#         f = 0                       

#         get_cluster_from_ground_truth = clustersFromGT


#         while y < len(get_cluster_from_ground_truth):
#             cm_k=np.zeros((n_e_val_0))
#             start = get_cluster_from_ground_truth[x]
#             end = get_cluster_from_ground_truth[y]
#             pred_labels=labels_PC[start:end]
#             f += 1
#             #print(f)
#             while True:
#                 maj_label=np.argmax(np.bincount(pred_labels))
#                 if(status[maj_label]!=1):
#                     status[maj_label]=1
#                     break
#                 pred_labels=pred_labels[pred_labels!=maj_label]
#                 if len(pred_labels) == 0:
#                     break
#             cl_k_k=np.count_nonzero(np.array(pred_labels==maj_label))  # True Positive
#             cm_k[maj_label]=cl_k_k  
#             for i in range((n_e_val_0)): # n_e_val_0 = no.of clusters in dataset ? Till now 'NO'  
#                 if (i != maj_label):
#                     cl=np.count_nonzero(np.array(pred_labels==uniq[i]))
#                     cm_k[i]=cl
#             confusion_mat[maj_label, :]=cm_k
#             x += 1
#             y += 1

#         # print(status,"\n")

#         def precision(label, confusion_mat):
#             col=confusion_mat[:,label]
#             return confusion_mat[label, label]/col.sum()

#         def recall(label, confusion_mat):
#             row=confusion_mat[label,:]
#             return confusion_mat[label,label]/row.sum()

#         precision_arr=[]
#         recall_arr=[]
#         f_score=[]
#         g_mean=[]
#         for i in range(n_e_val_0):
#             p=precision(i, confusion_mat)
#             r=recall(i, confusion_mat)
#             f=2*p*r/(p+r)
#             g=np.sqrt(p*r)
#             precision_arr=np.append(precision_arr, p)
#             recall_arr=np.append(recall_arr, r)
#             f_score=np.append(f_score, f)
#             g_mean=np.append(g_mean, g)

#         f_score = f_score[np.logical_not(np.isnan(f_score))]
#         g_mean = g_mean[np.logical_not(np.isnan(g_mean))]

#         f_score=np.mean(f_score)
#         g_mean=np.mean(g_mean)
#         accuracy=(np.trace(confusion_mat)/len(data))*100

#         print("\n***************************** PERFORMANCE MEASURE *****************************")
#         print("Accuracy: ",accuracy,"--------- f score:",f_score,"--------- g mean:",g_mean)
#         print("DB Index =", davies_bouldin_score(data1, labels_Idxs))
#         distances = pairwise_distances(data1)
#         print("Dunn Index =", dunn(distances, labels_Idxs))
#         print('Silhouette Score =',silhouette_score(data1, labels_Idxs))
#         print('Calinski-Harabasz Index =',metrics.calinski_harabasz_score(data1, labels_Idxs))




#     # Function to generate labels for getting DB, Dunn, Silhouette and Calinski-Harabasz Indexes
#     def GetLabelsForIdxs(labels):
#         get_labels_DB = []
#         for key in labels:
#             if labels[key] > 0:
#                 get_labels_DB.append(labels[key]-1)
#             else:
#                 get_labels_DB.append(labels[key])
#         return get_labels_DB

#     # Function to generate labels for getting the accurcay, f score and g mean
#     def GetLabelsForPC(labels):
#         get_labels_PC = []
#         for key in labels:
#             if labels[key] > 0:
#                 get_labels_PC.append(labels[key]-1)
#         return get_labels_PC


#     get_labels_PC = GetLabelsForPC(labels)
#     labels_PC = np.array(get_labels_PC)
#     get_labels_DB = GetLabelsForIdxs(labels)
#     labels_Idxs = np.array(get_labels_DB)
#     EvaluateClustering(labels_PC, cluster_num, Ground_Truth_data_path, clustersFromGT, data1, labels_Idxs)

In [ ]:
# Name: Display3

if 'Name: Display3' not in SM:
    
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    Roots = SM['Roots']
    
    if Dimension <= 2:
        plt.figure(figsize=(8,8))

        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']

        if cluster_num <= len(color):
            for i in range(len(data1)):
                if labels[i] != -1:
                    plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                else:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
        else:
            for i in range(len(data1)):
                if labels[i] == -1:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                else:
                    plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')

        for root in Roots:
            plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=200 )

        plt.xlabel('X-Coordinates')
        plt.ylabel('Y-Coordinates')
        plt.title('Datapoints Visualization After Outlier Post Processing')
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
    
    SM['Name: Display3'] = True
    SM.sync()
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Before Restructuring

if 'Name: Before Restructuring' not in SM:
    Roots = SM['Roots']
    print("Before Restructuring: ", Roots)
    
    SM['Name: Before Restructuring'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: RearrangeTreeStructure2

if 'Name: RearrangeTreeStructure2' not in SM:
    Tree_Structure = SM['Tree_Structure']
    data1 = SM['data1']
    labels = SM['labels']
    Store_Self_Density = SM['Store_Self_Density']
    cluster_num = SM['cluster_num']
    Roots = SM['Roots']
    
    RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, [], Roots)
    # print(CheckTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, K=4))
    # for key in Tree_Structure:
    #     for child in Tree_Structure[key]:
    #         if Store_Self_Density[key] > Store_Self_Density[child]:
    #             print("Fault!!")
    
    SM['Name: RearrangeTreeStructure2'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Restructuring1

if 'Name: Restructuring1' not in SM:
    Roots = SM['Roots']
    labels = SM['labels']
    Store_Self_Density = SM['Store_Self_Density']
    cluster_centers = SM['cluster_centers']
    
    toremovepairs1 = []
    idxs_1 = list(Roots.keys())
    for i in range(len(idxs_1)-1):
        for j in range(i+1, len(idxs_1)):
            if labels[idxs_1[i]] == labels[idxs_1[j]]:
                toremovepairs1.append([idxs_1[i], idxs_1[j]])

    for pair in toremovepairs1:
        if Store_Self_Density[pair[0]] < Store_Self_Density[pair[1]]:
          try:
            del Roots[pair[0]]
          except:
            pass
        else:
          try:
            del Roots[pair[1]]
          except:
            pass


    for i in range(len(cluster_centers)):
        for key in Roots:
            if labels[key] == cluster_centers[i][1]:
                cluster_centers[i][0] = Roots[key][0].tolist()
                break


    print("After Restructuring: ", Roots)
    #print(cluster_centers)
    
    SM['Roots'] = Roots
    
    SM['Name: Restructuring1'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Display4

if 'Name: Display4' not in SM:
    
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    Roots = SM['Roots']

    if Dimension <= 2:
        plt.figure(figsize=(8,8))

        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']

        if cluster_num <= len(color):
            for i in range(len(data1)):
                if labels[i] != -1:
                    plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                else:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
        else:
            for i in range(len(data1)):
                if labels[i] == -1:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                else:
                    plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')

        for root in Roots:
            plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=200 )

        plt.xlabel('X-Coordinates')
        plt.ylabel('Y-Coordinates')
        plt.title('Datapoints Visualization After Root Adjustment')
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
        
    SM['Name: Display4'] = True
    SM.sync()
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
def Plot(countimg):
    plt.figure(figsize=(8,8))
    
    color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
    if cluster_num <= len(color):
        for i in labels:
            # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
            if labels[i] != -1:
                try:
                    plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                except IndexError:
                    plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])
                
            else:
                try:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                except IndexError:
                    plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)
                    
                    
    else:
        for i in labels:
            if labels[i] == -1:
                try:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                except:
                    plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)
                
            else:
                try:
                    plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                except:
                    plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')
    
    for root in Roots:
        plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=200 )
                    
    # plt.xlabel('X-Coordinates')
    # plt.ylabel('Y-Coordinates')
    # plt.title("Datapoints Visualization For Incoming Data Points after Processing")
    plt.savefig('E:\ISI\STREAMING RESULTS\RESULTS 29-06-2021\Outlier(F)\{}.png'.format(countimg[0]+1), bbox_inches='tight')
    plt.close()
    ## print("SAVED!!")
    # plt.show()
    return [countimg[0]+1]


In [ ]:
def GetSingleRootIdxs(labels, data1):
    SingleRootIdxs = []
    Hash_Map_F = dict()
    for i in range(len(data1)):
        if labels[i] in Hash_Map_F:
            Hash_Map_F[labels[i]].append(i)
        else:
            Hash_Map_F[labels[i]] = [i]
    
    for key_label in Hash_Map_F:
        if len(Hash_Map_F[key_label]) == 1:
            SingleRootIdxs.append(Hash_Map_F[key_label][0])
    
    return SingleRootIdxs
        

In [ ]:
def ChangeLabelsofSingleNodeTree(SingleRootIdxs, data1, algo, labels):
    nbrs_ddcal = NearestNeighbors(n_neighbors=len(data1), algorithm=algo).fit(data1)
    distances_ddcal, indices_ddcal = nbrs_ddcal.kneighbors(data1)
    
    for item_idx in SingleRootIdxs:
        neighbour_idx = indices_ddcal[item_idx][1]
        # Change the sngle Root's label to the label of its nearest neighbour
        labels[item_idx] = labels[neighbour_idx]
    
    return labels

In [ ]:
def RunConceptEvolution(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
                         Store_Parents, Hash_Map, thresh, anomaly_list, Store_Self_Density, cluster_num,
                        Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
                       algo, alpha_weight_avg):
    
#     # 1
#     data1 = ConceptEvolutionOne(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
#                          Store_Parents, Hash_Map, thresh, anomaly_list, Store_Self_Density, cluster_num,
#                         Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
#                     algo, alpha_weight_avg)


    clear_output(wait=True)
    
    print("TOTAL NUMBER OF DATAPOINTS: ", len(data1))
    
    
    # Change single node points part of some other cluster
    SingleRootIdxs = GetSingleRootIdxs(labels, data1)
    labels = ChangeLabelsofSingleNodeTree(SingleRootIdxs, data1, algo, labels)

    
    # 1
    PlotSeparationAnomalies(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)
    print("1 Done")
    
    
    # 2
    Roots, cluster_centers, anomaly_idxs, anomaly_dps, anomaly_dict = CorrectRootsandClusterCentersandgetCurrentAnomalies(data1, 
                        Roots, cluster_centers, labels, Store_Self_Density)
    print("2 Done")
    
    # 3
    density_1, distances_ddcal_1, indices_ddcal_1, Hash_Map, dense_pt = RunFindMostDensePoint(anomaly_dps, algo, 
                        Incoming_points, data1, Hash_Map, LLE_lookup, anomaly_idxs, anomaly_dict)
    print("3 Done")
    
#     # 4
#     PlotWithDensePoint(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots, dense_pt)
#     print("4 Done")
    
    # 5
    labels, Roots_1, cluster_centers_1, Store_Radius_Density_1, Store_Parents_1, Store_Self_Density_1, Tree_Structure_1, \
    density_1 = ChangelabelsandRunBuildModelStreaming(labels, 
    anomaly_idxs, anomaly_dps, density_1, Hash_Map, thresh, alpha_weight_avg, distances_ddcal_1, indices_ddcal_1,
    data1, LLE_lookup, Store_Self_Density, train_end, train_start, Roots, Store_Radius_Density, Store_Parents)
    print("5 Done")
    
    # 6
    cluster_centers = GetPresentClusterInformation1(cluster_centers_1, cluster_centers)
    print("6 Done")
    
#     # 7
#     Roots_1, labels, Tree_Structure_1 = RunCorrectingwrongclusteringAndcorrectthelabels(labels, cluster_centers, data1, Roots_1)    
    RunCorrectingwrongclusteringAndcorrectthelabels(labels, cluster_centers, data1, Roots_1)
    print("7 Done")
    
    # 8
    Roots, Roots_1, cluster_centers = UpdateRootsandCluster1(cluster_centers, Roots, Roots_1, labels)
    print("8 Done")
    
    # 9
    percent5 = 5
    Tree_Structure, Store_Self_Density, Store_Radius_Density, Store_Parents, Roots, cluster_num = MergeAll(Tree_Structure, 
                            Tree_Structure_1, Store_Self_Density, Store_Self_Density_1, Store_Radius_Density, 
                             Store_Radius_Density_1, Store_Parents, Store_Parents_1, Roots, Roots_1,
                             distances_ddcal_1, indices_ddcal_1, LLE_lookup, algo, anomaly_dps, anomaly_idxs, 
                            labels, percent5, alpha_weight_avg,)
    print("9 Done")
    
    
    Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

    # Check if cluster_num is same as the Supposed_Roots: if not then this function corrects it.
    # I am considering that len(Supposed_Roots) > or = cluster_num. Is this correct?
    if len(Supposed_Roots) != cluster_num:
      Tree_Structure = MakeSupposedRootsandClusternumsamelength(Supposed_Roots, labels, Tree_Structure, Store_Self_Density)
      Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

    Roots = AddRootsFromSupposedRoots(Supposed_Roots, Roots, data1, Store_Self_Density)
    # print("Supposed Roots: ", Supposed_Roots)
    if len(list(Supposed_Roots)) > len(Roots):
        pass
        # print("Error at RunConceptEvolution after 9 done")
        # print("Supposed Roots: ", Supposed_Roots)
        # print("\nTree_Structure: \n", Tree_Structure)
        # raise Exception("Number of Supposed Roots has become more than the actual number of roots")
    
    
    # 10
    Roots, cluster_centers, cluster_num = UpdateRootsAndClusterCentersAgain(Roots, cluster_centers, Store_Self_Density)
    print("10 Done")
    
#     # 11
#     InitalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)
#     print("11 Done")
    
    # 12
    labels = RunRearranging5(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Roots)
    print("12 Done")
    
#     # 13
#     Correct1(labels, data1, algo, LLE_lookup, Tree_Structure, Store_Self_Density, alpha_weight_avg, thresh)    ##### I added this
#     print("13 Done")
    
    # 14
    updated_anomaly_idxs, updated_anomaly_dps = RunCheckandCorrectStreaming(data1, anomaly_idxs, LLE_lookup, anomaly_dps, 
                                Roots, Store_Radius_Density, Store_Parents, Hash_Map, thresh,
                                Store_Self_Density, Tree_Structure, labels, Concept_Evolution_Thresh, cluster_centers,
                                Dimension, train_end, train_start, algo, alpha_weight_avg)
    print("14 Done")
    
#     # 14
#     Correct1(labels, data1, algo, LLE_lookup, Tree_Structure)    ##### I added this
#     print("14 Done")
    
    # 15
    Roots, cluster_centers, cluster_num = UpdateRootsandCluster2(Roots, cluster_centers, Store_Self_Density)
    print("15 Done")
    
    # 16
    labels = RunRearranging7(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Roots)
    print("16 Done")
    
    # 17
    FinalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)
    print("17 Done")
    
    # 18
    DisplayInformation(updated_anomaly_idxs, Roots)
    print("18 Done")
    
#     # 19
#     Correct1(labels, data1, algo, LLE_lookup, Tree_Structure, Store_Self_Density, alpha_weight_avg, thresh)    ##### I added this
#     print("19 Done")

    Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)
    # print("Supposed Roots: ", Supposed_Roots)
    if len(list(Supposed_Roots)) > len(Roots):
        pass
        # print("Error at RunConceptEvolution after 19 done")
        # print("Supposed Roots: ", Supposed_Roots)
        # print("\nTree_Structure: \n", Tree_Structure)
        # raise Exception("Number of Supposed Roots has become more than the actual number of roots")
    
    
    return data1, Roots, cluster_centers, labels, updated_anomaly_dps, updated_anomaly_idxs, \
            Tree_Structure, Store_Self_Density, cluster_num, Store_Radius_Density, Store_Parents, Hash_Map 
    
    
    
    

In [ ]:
## Streaming portion of the model
# Incomming datapoint: Inc_Dp

def Euclidean_Distance(point1, point2):
    # calculating Euclidean distance
    dist = np.linalg.norm(point1 - point2)
    return dist
    
    
    
def GetNearestRoot(Roots, Inc_Dp):
    Dp_Rootidx_Rootdist = []
    for root_idx in Roots:
        dist_with_root = Euclidean_Distance(Roots[root_idx][0], Inc_Dp)
        Dp_Rootidx_Rootdist.append([root_idx, dist_with_root])
        Dp_Rootidx_Rootdist.sort(key=itemgetter(1), reverse=False)
    return Dp_Rootidx_Rootdist



def GetSiblingNode(Inc_Dp, Roots, Root_node_idx, data1, Tree_Structure):
    parentnode = Roots[Root_node_idx][0]
    distance_with_parent = Euclidean_Distance(Inc_Dp, parentnode)
    # print("PN: ",parentnode, "PD: ", distance_with_parent)
    ImmediateAncestorNode_idx = Root_node_idx
    
    if Root_node_idx in Tree_Structure:
        child_nodes_idxs = Tree_Structure[Root_node_idx]
    else:
        child_nodes_idxs = []
    min_dist = float("inf")
    for child_node_idx in child_nodes_idxs:
        curr_dist = Euclidean_Distance(Inc_Dp, data1[child_node_idx])
        # print("CN: ", data1[child_node], "CD: ",curr_dist, "MD: ", min_dist)
        if curr_dist < min_dist:
            min_dist = curr_dist
            curr_sibling_node_idx = child_node_idx
    distance_with_child = min_dist
    
    # Set initial sibling node as the Root node
    sibling_node_idx = Root_node_idx
    
    # Check for the nearest node in the Tree Structure if it exists!!
    while distance_with_parent > distance_with_child: # (3 1)
        # print("DWP: ", distance_with_parent, "DWC: ", distance_with_child, "CSN: ",data1[curr_sibling_node])
        sibling_node_idx = curr_sibling_node_idx
        parent_node_idx = child_node_idx
        
        if parent_node_idx not in Tree_Structure:
            # print("YES")
            break
        else:
            ImmediateAncestorNode_idx = parent_node_idx
            child_nodes_idxs = Tree_Structure[parent_node_idx]
            min_dist = float("inf")
            for child_node_idx in child_nodes_idxs:
                curr_dist = Euclidean_Distance(Inc_Dp, data1[child_node_idx])
                if curr_dist < min_dist:
                    min_dist = curr_dist
                    curr_sibling_node_idx = child_node_idx

            distance_with_parent = Euclidean_Distance(Inc_Dp, data1[parent_node_idx])
            distance_with_child = min_dist
            
            sibling_node_idx = curr_sibling_node_idx
            parent_node_idx = child_node_idx
            

    return sibling_node_idx, ImmediateAncestorNode_idx
        
    
    
    
def IncludeWithinTree(Inc_Dp, Inc_Dp_Idx, Sibling_node, ImmediateAncestorNode, Store_Radius_Density, 
                      Tree_Structure, dataset, Hash_Map, labels, thresh, Store_Parents, Roots):
    # For the point to be included within the same tree, the density of that has to be calculated wrt the tree.
    # Meaning, the neighbours of that new point, should be already members of that Tree.
    # They should not be anomalies or other cluster members
    labelofCurrentTree = labels[Sibling_node]

    rng = [[],[]]
    neigh_dist = []
    neigh_idxs = []        

    # Get density of the Inc_Dp
    Radius = Store_Radius_Density[Sibling_node][0]
    # print("Radius: ", Radius)
    neigh = NearestNeighbors(radius = Radius)
    neigh.fit(dataset)
    rng1 = neigh.radius_neighbors([Inc_Dp])

    for i in range(len(rng1[1][0])):
      if (rng1[1][0][i]) in labels:  
        if labels[rng1[1][0][i]] == labelofCurrentTree:
          neigh_dist.append(rng1[0][0][i])
          neigh_idxs.append(np.asarray(int(rng1[1][0][i])))

    rng[0].append(np.asarray(neigh_dist))
    rng[1].append(np.asarray(neigh_idxs))   
    rng = tuple(rng)


    # Check if there is no neighbour within the radius
    # If yes then the density of the incoming datapoint is 0
    if len(rng[0][0]) == 0:
        Density_Inc_Dp = 0
    else:
        # Density_Inc_Dp = sum(rng[0][0])/len(rng[0][0])
        Density_Inc_Dp = sum(rng[0][0])/(math.pi*Radius*Radius) 
    
    while True:
        # We check for the three possible positions for the new point's inclusion:
        # 1. As a child of the parent of its Sibling
        # 2. As a child of its Sibling
        # 3. As a child of any of its Sibling's childs
        
        # Set as Child of Parent of Sibling
        Parent = ImmediateAncestorNode
        wt_density_ancestors = Store_Radius_Density[Parent][1]
        numerator = abs(wt_density_ancestors - Density_Inc_Dp)
        denominator = wt_density_ancestors
        # print("DIP1: ", Density_Inc_Dp, "N: ", numerator, "D: ", denominator, "Frac: ", numerator/denominator)
        
        # Calculate the distance with parent(d1). Calculate the distance of the parent with its parent(d2).
        # If d1 is within __ % of d2, then include otherwise don't
#         Parent_dp = dataset[Parent]
#         d1 = np.linalg.norm(Parent_dp - Inc_Dp)
        # Getting the Parent's parent idx
#         Parents_parent_dp = None   # If the parent is a root no need to check
#         for key in Tree_Structure:
#             if Parent in set(Tree_Structure[key]):
#                 Parents_parent_idx = key
#                 Parents_parent_dp = dataset[Parents_parent_idx]
#                 break
#         if Parents_parent_dp is not None:
#             d2 = np.linalg.norm(Parent_dp - Parents_parent_dp)
#             # print("d1: ", d1, "d2: ", d2)
                    
        if numerator/denominator <= thresh: ############## and (Parents_parent_dp is None or (d2/d1)>0.7):    
            labels[Inc_Dp_Idx] = labels[ImmediateAncestorNode]
            # Add this newly added nodes information to the existing Hash_Map
            #            0                1         2        3                    4
            # Distance-Density Value; Datapoint; Radius; k-nearest neighbors; idx_number
            if len(rng[0][0]) == 0:
                Radius_new_node = 0
            else:
                Radius_new_node = max(rng[0][0])
            NewNode = [Density_Inc_Dp, Inc_Dp, Radius_new_node, rng[1][0], Inc_Dp_Idx]
            temp_dict = {NewNode[-1]:NewNode}
            Hash_Map.update(temp_dict)

            # Store Adaptive Radius and New Weighted Density of all ansectors including the New Node
            # print("IDX: ", NewNode[4])
            list_of_parents = Store_Parents[Parent] + [Density_Inc_Dp] 
            data_parent = np.array(list_of_parents) #, dtype=np.float64)
            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
            Store_Radius_Density[NewNode[4]] = [NewNode[2], weighted_density]
            Store_Parents[NewNode[4]] = list_of_parents
            Store_Self_Density[NewNode[4]] = Density_Inc_Dp

            # Add the New Node to the Tree Structure
            if Parent not in Tree_Structure:
                Tree_Structure[Parent] = [NewNode[4]]
            else:
                Tree_Structure[Parent].append(NewNode[4])
            # print("CHILD OF PARENT OF SIBLING")
            
            # Check for duplicate poistions
            Tree_Structure = RemoveDulplicatePositions(Tree_Structure, Store_Self_Density)
            Tree_Structure = CorrectOneChildTwoParents(Tree_Structure, Store_Self_Density)
            
            print("----------------------------------Rearranging1----------------------------------")
            cluster_num = len(Roots)
            # dataset.append(Inc_Dp)
            RearrangeTreeStructure(Tree_Structure, dataset, labels, Store_Self_Density, cluster_num, Incoming_points, Roots)
            
            break
        # print("NOT CHILD OF PARENT OF SIBLING")


        # Set as Child of Sibling
        Parent = Sibling_node
        wt_density_ancestors = Store_Radius_Density[Parent][1]
        numerator = abs(wt_density_ancestors - Density_Inc_Dp)
        denominator = wt_density_ancestors
        # print("DIP2: ", Density_Inc_Dp, "N: ", numerator, "D: ", denominator, "Frac: ", numerator/denominator)
        
        # Calculate the distance with parent(d1). Calculate the distance of the parent with its parent(d2).
        # If d1 is within __ % of d2, then include otherwise don't
#         Parent_dp = dataset[Parent]
#         d1 = np.linalg.norm(Parent_dp - Inc_Dp)
#         # Getting the Parent's parent idx
#         Parents_parent_dp = None   # If the parent is a root no need to check
#         for key in Tree_Structure:
#             if Parent in set(Tree_Structure[key]):
#                 Parents_parent_idx = key
#                 Parents_parent_dp = dataset[Parents_parent_idx]
#                 break
#         if Parents_parent_dp is not None:
#             d2 = np.linalg.norm(Parent_dp - Parents_parent_dp)
#             # print("d1: ", d1, "d2: ", d2)
        
        if numerator/denominator <= thresh:  ################## and (Parents_parent_dp is None or (d2/d1)>0.7):
            labels[Inc_Dp_Idx] = labels[Sibling_node]
            # Add this newly added nodes information to the existing Hash_Map
            #            0                1         2        3                    4
            # Distance-Density Value; Datapoint; Radius; k-nearest neighbors; idx_number
            if len(rng[0][0]) == 0:
                Radius_new_node = 0
            else:
                Radius_new_node = max(rng[0][0])
            NewNode = [Density_Inc_Dp, Inc_Dp, Radius_new_node, rng[1][0], Inc_Dp_Idx]
            temp_dict = {NewNode[-1]:NewNode}
            Hash_Map.update(temp_dict)

            # Store Adaptive Radius and New Weighted Density of all ansectors including the New Node
            # print("IDX: ", NewNode[4])
            list_of_parents = Store_Parents[Parent] + [Density_Inc_Dp] 
            data_parent = np.array(list_of_parents) #, dtype=np.float64)
            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
            Store_Radius_Density[NewNode[4]] = [NewNode[2], weighted_density]
            Store_Parents[NewNode[4]] = list_of_parents
            Store_Self_Density[NewNode[4]] = Density_Inc_Dp

            # Add the New Node to the Tree Structure
            if Parent not in Tree_Structure:
                Tree_Structure[Parent] = [NewNode[4]]
            else:
                Tree_Structure[Parent].append(NewNode[4])
            # print("CHILD OF SIBLING")
            
            # Check for duplicate poistions
            Tree_Structure = RemoveDulplicatePositions(Tree_Structure, Store_Self_Density)
            Tree_Structure = CorrectOneChildTwoParents(Tree_Structure, Store_Self_Density)
            
            print("----------------------------------Rearranging2----------------------------------")
            cluster_num = len(Roots)
            # dataset.append(Inc_Dp)
            RearrangeTreeStructure(Tree_Structure, dataset, labels, Store_Self_Density, cluster_num, Incoming_points, Roots)
            
            break
        # print("NOT CHILD OF SIBLING")
        

        # Set as Child of Childs of Sibling
        boolean = False
        Childs_of_Sibling_node = deque()
        if Sibling_node in Tree_Structure:
            for childnode in Tree_Structure[Sibling_node]:
                Childs_of_Sibling_node.appendleft(childnode)  # Leftward addition so that 
                                                              # all childs of one layer are processed
                    
            # Creating a set to prevent multiple checking on the same node
            nodes_visited = set()
            # Check the density criterion with each child
            while Childs_of_Sibling_node:
                childnode = Childs_of_Sibling_node.pop()
                # print("Ch_ND: ", childnode)
                Parent = childnode
                if childnode in Store_Radius_Density:  ## I added this
                    wt_density_ancestors = Store_Radius_Density[childnode][1]
                    numerator = abs(wt_density_ancestors - Density_Inc_Dp)
                    denominator = wt_density_ancestors
                    # print("DIP3: ", Density_Inc_Dp, "N: ", numerator, "D: ", denominator, "Frac: ", numerator/denominator)

                    # Calculate the distance with parent(d1). Calculate the distance of the parent with its parent(d2).
                    # If d1 is within __ % of d2, then include otherwise don't
#                     Parent_dp = dataset[Parent]
#                     d1 = np.linalg.norm(Parent_dp - Inc_Dp)
#                     # Getting the Parent's parent idx
#                     Parents_parent_dp = None   # If the parent is a root no need to check
#                     for key in Tree_Structure:
#                         if Parent in set(Tree_Structure[key]):
#                             Parents_parent_idx = key
#                             Parents_parent_dp = dataset[Parents_parent_idx]
#                             break
#                     if Parents_parent_dp is not None:
#                         d2 = np.linalg.norm(Parent_dp - Parents_parent_dp)
#                         # print("d1: ", d1, "d2: ", d2)

                    if numerator/denominator <= thresh: ################# and (Parents_parent_dp is None or (d2/d1)>0.7):
                        labels[Inc_Dp_Idx] = labels[childnode]
                        # Add this newly added nodes information to the existing Hash_Map
                        #            0                1         2        3                    4
                        # Distance-Density Value; Datapoint; Radius; k-nearest neighbors; idx_number
                        if len(rng[0][0]) == 0:
                            Radius_new_node = 0
                        else:
                            Radius_new_node = max(rng[0][0])
                        NewNode = [Density_Inc_Dp, Inc_Dp, Radius_new_node, rng[1][0], Inc_Dp_Idx]
                        temp_dict = {NewNode[-1]:NewNode}
                        Hash_Map.update(temp_dict)

                        # Store Adaptive Radius and New Weighted Density of all ansectors including the New Node
                        # print("IDX: ", NewNode[4])
                        list_of_parents = Store_Parents[Parent] + [Density_Inc_Dp] 
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        Store_Radius_Density[NewNode[4]] = [NewNode[2], weighted_density]
                        Store_Parents[NewNode[4]] = list_of_parents
                        Store_Self_Density[NewNode[4]] = Density_Inc_Dp

                        # Add the New Node to the Tree Structure
                        if Parent not in Tree_Structure:
                            Tree_Structure[Parent] = [NewNode[4]]
                        else:
                            Tree_Structure[Parent].append(NewNode[4])
                        # print("CHILD OF CHILDS OF SIBLING")

                        # Check for duplicate poistions
                        Tree_Structure = RemoveDulplicatePositions(Tree_Structure, Store_Self_Density)
                        Tree_Structure = CorrectOneChildTwoParents(Tree_Structure, Store_Self_Density)

                        print("----------------------------------Rearranging3----------------------------------")
                        cluster_num = len(Roots)
                        # dataset.append(Inc_Dp)
                        RearrangeTreeStructure(Tree_Structure, dataset, labels, Store_Self_Density, cluster_num, \
                                               Incoming_points, Roots)

                        boolean = True
                        break

                    else:
                        if childnode not in nodes_visited and childnode in Tree_Structure:
                            for new_child_node in Tree_Structure[childnode]:
                                Childs_of_Sibling_node.appendleft(childnode)  # Leftward addition so that all
                                                                              # childs of one layer are processed
                            # print("CSN: ", Childs_of_Sibling_node)
                            nodes_visited.add(childnode)  

                            
        ##  We have already checked whether it can be its sibling's child. So no need to check again
        
                            
                            
            if boolean == True:
                break
        #     else:
        #         print("NOT CHILD OF CHILDS OF SIBLING")
        # else:
        #     print("NOT CHILD OF CHILDS OF SIBLING")
            
        # If we reach here, it means the incoming point is a possible anomaly
        # Assign its label as -1 and terminate the loop
        labels[Inc_Dp_Idx] = -1
        break
    
    return Store_Radius_Density, Store_Self_Density#, dataset
    
    
    
    
def Updatedistance_ddcalandindices_ddcal(data1, algo):
    nbrs_ddcal = NearestNeighbors(n_neighbors=len(data1), algorithm=algo).fit(data1)
    distances_ddcal, indices_ddcal = nbrs_ddcal.kneighbors(data1)
    return distances_ddcal, indices_ddcal



# LLE: Locally linear Embedding
def LLE_Individual_Datapoint(data_pt, K, dp_idx, total_num_pts, data): #, data_point_Idx):
    row=total_num_pts
    #print(row)
    dimension=len(data_pt)
    #print(dimension)
    neighbors= K

    hash_map = {value:np.array([]) for value in range(0, 1)}

    # for i in range(row):
    D_i=np.array(data-data[dp_idx, :])
    #print(D_i)
    distance=(D_i**2).sum(1)
    #print(distance)
    nearest_neighbor=np.argsort(distance)[1:(neighbors+1)]
    #print(nearest_neighbor)
    D_nbrs=D_i[nearest_neighbor, :]
    #print(D_nbrs)
    ##Q=np.dot(D_nbrs, D_nbrs.T)
    Q=np.matmul(D_nbrs, D_nbrs.T)
    #print(Q)
#         t=np.trace(Q)
#         r=0.001*t
    r = 0.001 * float(Q.trace())
    if(neighbors>=dimension):
#             sig2 = (np.linalg.svd(D_i,compute_uv=0))**2
#             r = np.sum(sig2[dimension:])
        Q=Q+(r*np.identity(neighbors))
#             Q.flat[::neighbors+1] += r
    ##w=np.linalg.solve(Q, np.ones(neighbors))
    w=np.linalg.solve(Q, np.ones(neighbors))

    w = w/sum(w)
    
    temp = []
    for ele in w:
        temp.append(ele)
    temp.sort(reverse=True)
    # print("w: ", temp)
    
    hash_map[0] = np.array(temp) 

    

    per = 5 #0.6 #0.3 #0.5 #0.2 #0.05 #0.1

    for j in range(1):
        count = 0
        for i in range(1, len(hash_map[j])):
            #print(((1/neighbors) - (dec_per*(1/neighbors))), ((1/neighbors) + (dec_per*(1/neighbors))))
            if round((abs(hash_map[j][i-1] - hash_map[j][i])/abs(hash_map[j][i-1]))*100,1) <= per:
                count += 1


    return count+1              # +1 because all neighbouring algorithms consider the datapoint itself to be a part
                                # of its neighbour


    

    
def GetNeighbours(inlier, data1, K):
    dist_idx = []
    for i in range(len(data1)):
        comparison = data1[i] == inlier
        equal_arrays = comparison.all()
        if equal_arrays == False: #True:
            # print(data1[i], inlier)
            dist_idx.append([np.linalg.norm(inlier - data1[i]), i])   #math.dist(inlier, data1[i]), i])
    dist_idx.sort(key=operator.itemgetter(0))
    idxs = [row[1] for row in dist_idx][:K]
    return idxs
    
def GetNeighbourAnomalies(data_pt_idx, LLE_lookup, data1):
    opt_k = LLE_lookup[data_pt_idx]
    nearest_neighbour_idxs = GetNeighbours(data1[data_pt_idx], data1, opt_k)
    
    neighbour_anomalies = []
    idxs = []
    for neighbour_idx in nearest_neighbour_idxs:
        if labels[neighbour_idx] == -1:
            idxs.append(neighbour_idx)
            neighbour_anomalies.append(data1[neighbour_idx])
    # print("Neighbour Anomaly indexes: ", idxs)
    return neighbour_anomalies


def getanomalyindexes(anomaly_list, data1):
    anomaly_idxs = []
    for anomaly in anomaly_list:
        for i in range(len(data1)):
            comparison = data1[i] == anomaly
            equal_arrays = comparison.all()
            if equal_arrays == True:
                anomaly_idxs.append(i)
                break
    return anomaly_idxs


def GetParentfromTree_F1(Tree_Structure, idx):
    send_key = None
    for key in Tree_Structure:
        if idx in set(Tree_Structure[key]):
            send_key = key
            break
    if send_key is None:
        return send_key
    else:
        return int(send_key)
        

def GetCardinality_F(labels, idx):
    cardinality = 0
    label_idx = labels[idx]
    for key in labels:
        if labels[key] == label_idx:
            cardinality += 1
    return cardinality


def ChangeAllLabels_F(labels, Tree_Structure, idx, changetolabel):
    ToChange = deque()
    ToChange.append(idx)
    visited = {key:False for key in labels}
    while True:
        # print("ToChange: ", ToChange)
        if len(ToChange) == 0:
            break
        curr_idx = ToChange.pop()
        labels[curr_idx] = changetolabel
        if curr_idx in Tree_Structure:
            for child_idx in Tree_Structure[curr_idx]:
                if child_idx not in set(ToChange) and visited[child_idx] == False:
                    ToChange.appendleft(child_idx)
                    visited[child_idx] = True
                    
    # return labels


def GetNeighboursofDifferentCluster(Inc_Dp_Idx, LLE_lookup, labels, data1):
    opt_k = LLE_lookup[Inc_Dp_Idx]
    nearest_neighbour_idxs = GetNeighbours(data1[Inc_Dp_Idx], data1, opt_k)
    
    neighbour_idx_of_different_clusters = []
    for neighbour_idx in nearest_neighbour_idxs:
        if labels[neighbour_idx] != -1 and labels[neighbour_idx] != labels[Inc_Dp_Idx]:
            neighbour_idx_of_different_clusters.append(neighbour_idx)
            # neighbour_anomalies.append(data1[neighbour_idx])
    # print("Neighbour Anomaly indexes: ", idxs)
    return neighbour_idx_of_different_clusters
    

    
def GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx, data1):
    parentsofIncDpincludingself = deque()
    parentsofIncDpincludingself.appendleft(Store_Self_Density[Inc_Dp_Idx])
    parentlist = deque()
    parentlist.append(Inc_Dp_Idx)
    visited = {i:False for i in range(len(data1))}
    while parentlist:
        # print("PL: ", parentlist)
        curr_dp_idx = parentlist.pop()
        if visited[curr_dp_idx] == True:
          continue
        else:
          visited[curr_dp_idx] = True
          for key in Tree_Structure:
              if curr_dp_idx in set(Tree_Structure[key]):
                  parentlist.append(key)
                  parentsofIncDpincludingself.appendleft(Store_Self_Density[key])
                  break
    # print("RETURNING PARENTLIST")
    return list(parentsofIncDpincludingself)

    

def RunStreamingData(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
                     Store_Parents, Hash_Map, thresh, anomaly_list, countimg, Store_Self_Density, cluster_num,
                    Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
                    algo, alpha_weight_avg, incoming_time_dur):
     
    thresh_5 = 10 #(for corners)
    percent7 = 0.05  #0.5 #0.3
    Sibling_nodes = []
    total_num_pts = len(data1)
    # Check if Incoming point was a potential anomaly and now an inlier
    # In such a scenario just skip that data point
    
    # Initialize curr_waiting_time as 0 as there is no waiting for the very first incoming datapoint
    curr_waiting_time = 0
    # Initialize the Total_Time as 0
    Total_Time = 0
    
    
    for Inc_Dp in Incoming_points:
        print("********************************DATAPOINT INCOMING*****************************")
        Bool = False
        
        # print("Here F100: ", Tree_Structure)
        
        for i in range(len(data1)):
            item = data1[i]
            comparison = item == Inc_Dp
            equal_arrays = comparison.all()
            if equal_arrays == True:
                Bool = True
                hold_idx = i
                break
        if Bool == False:
            # This is a new data point so calculate its processing time
            pro_start_time = time.time()
            
            data1 = np.append(data1, [Inc_Dp], axis=0) # Adding Data Point to the dataset
            Inc_Dp_Idx = len(data1)-1                  # Assigning index number of new Data point  
            labels[Inc_Dp_Idx] = 0                     # Assigning new data point with label 0
            
        else:
            Inc_Dp_Idx = hold_idx                             # Assigning index number of already seen Data point
        
        if (Inc_Dp_Idx in labels and labels[Inc_Dp_Idx] <= 0):
            labels[Inc_Dp_Idx] = 0                            # Assigning anmalous/new data point with label 0

            # print("Incoming Data point Idx: ", Inc_Dp_Idx)

            # Get all roots in increasing order of their distance with the incoming data point
            idx_dist_with_roots = GetNearestRoot(Roots, Inc_Dp)
            # print("idx_dist_with_roots: ", idx_dist_with_roots)

            i = 0
            limit = 1
            # print("i: ", i, "idx_dist_with_roots: ", idx_dist_with_roots)

#             while i < len(idx_dist_with_roots) and i < limit and labels[Inc_Dp_Idx] <= 0:

                # print("i: ", i)
            Root_node_idx = idx_dist_with_roots[i][0]
            # print("Closest Root of", Inc_Dp, "is:", Roots[Root_node_idx][0])
            sibling_node_idx, ImmediateAncestorNode_idx = GetSiblingNode(Inc_Dp, Roots, Root_node_idx, data1, Tree_Structure)
            # print("Sibling Node of ", Inc_Dp, "is:", data1[sibling_node_idx])
            Sibling_nodes.append(data1[sibling_node_idx])

            # print("ImmediateAncestorNode of Sibling is: ", data1[ImmediateAncestorNode_idx])
            Store_Radius_Density, Store_Self_Density = IncludeWithinTree(Inc_Dp, Inc_Dp_Idx, sibling_node_idx, ImmediateAncestorNode_idx, Store_Radius_Density,
                              Tree_Structure, data1, Hash_Map, labels, thresh, Store_Parents, Roots)
#                 i += 1
                # print("\n")
                ## print("i: ", i, "idx_dist_with_roots: ", idx_dist_with_roots)

        
        
            ### I Commented this - Since these parameters are not used in this function
            ## Update distances_ddcal and indices_ddcal
            #distances_ddcal, indices_ddcal = Updatedistance_ddcalandindices_ddcal(data1, algo)
            ###print(len(distances_ddcal), len(indices_ddcal))
            
            
            
            # Get Pruned k for the new data point
            opt_k = LLE_Individual_Datapoint(Inc_Dp, K, Inc_Dp_Idx, total_num_pts, data1)
            ## print("Opt K: ", opt_k)
            LLE_lookup[Inc_Dp_Idx] = opt_k
            ## print(len(LLE_lookup))
            

            ## For saving Individual plots
            # if labels[Inc_Dp_Idx] == -1:
            #     countimg = Plot(countimg)
            

            if labels[Inc_Dp_Idx] != -1:    
                
                # For the point that is just labelled, compare its distance with its present parent and its neighbours 
                # which belong to some different cluster. If the distance and density criteria satisfies,
                # include it to that neighbour cluster and break
                parent_of_curr_pt_idx = GetParentfromTree_F1(Tree_Structure, Inc_Dp_Idx)
                neighbour_idx_of_different_clusters = GetNeighboursofDifferentCluster(Inc_Dp_Idx, LLE_lookup, labels, data1)
                print("--> ASSESSING WITH ITS NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
                for neighbour_idx in neighbour_idx_of_different_clusters:

                    neighbour_dp = data1[neighbour_idx]
                    # If the current point has become a root node
                    if parent_of_curr_pt_idx == None: 
                        # print("--> ROOT OBTAINED!! with idx: ", neighbour_idx, "and label:", labels[neighbour_idx])
                        
                        # If root is obtained, check the density of the root with the probable parent.
                        # It is to be done similar to the method of inclusion of a data point (same threshold)
                        # If the root satisfies the density criterion, include it otherwise don't
                        
                        Density_neighbour = Store_Self_Density[neighbour_idx]
                        Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                        list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, neighbour_idx, data1)
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        numerator = abs(weighted_density - Density_Inc_Dp)
                        denominator = weighted_density
                        # print("Weighted Density =", numerator/denominator)
                        # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                        
                        # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
                        cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
                        if cardinality_curr_cluster <= 8 and numerator/denominator <= thresh:
                            # cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                            # cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
                            # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
                            # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
                            #     # print(neighbour_idx, "INCLUDED")
                            ChangeAllLabels_F(labels, Tree_Structure, Inc_Dp_Idx, labels[neighbour_idx])
                            RemoveNodesWithNoChild(Tree_Structure)
                            if neighbour_idx in Tree_Structure:
                                Tree_Structure[neighbour_idx].append(Inc_Dp_Idx)
                            else:
                                Tree_Structure[neighbour_idx] = [Inc_Dp_Idx]
                            break
                    
                    
                    else:
                        # If the current point is not a root Node
                        
                        if type(parent_of_curr_pt_idx) != int:
                            raise Exception("More than 1 idx present(1)")
                        parent_of_curr_pt = data1[parent_of_curr_pt_idx]
                        # parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, Inc_Dp_Idx)
                        # parent_of_curr_dp = data1[parent_of_curr_dp_idx]
                        # Not the child's parent but check with the child itself.

                        Density_neighbour = Store_Self_Density[neighbour_idx]
                        Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                        list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, neighbour_idx, data1)
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        numerator = abs(weighted_density - Density_Inc_Dp)
                        denominator = weighted_density
                        # print("Weighted Density =", numerator/denominator)
                        # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)

                        # Find distances with the present parent and the probable parent.
                        dist_orig_parent = np.linalg.norm(Inc_Dp - parent_of_curr_pt)
                        dist_prob_parent = np.linalg.norm(Inc_Dp - neighbour_dp)
                        
                        # Find density differences with the present parent and the probable parent.
                        # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
                        # density_diff_with_prob_parent = Store_Self_Density[Inc_Dp_Idx] - Store_Self_Density[neighbour_idx]
                        
                        ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                        
                        ## print("Idx: ", neighbour_idx, "Label: ", labels[neighbour_idx], "dist_orig_parent: ", dist_orig_parent, "dist_prob_parent: ", dist_prob_parent)
                        if (dist_orig_parent)*0.3 <= (dist_prob_parent) and numerator/denominator > thresh: ## and (dist_orig_parent) <= thresh_5:
                        ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                            # Everything is fine. Nothing needs to be changed
                            # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
                            # print("--> Everything is fine. Nothing needs to be changed")
                            continue

                        else:
                            # 1. Change the neighbour's parent as the parent of the current data point
                            # 2. If there are children of the neigbour, include them all into this cluster
                            #    and under the current data point in the Tree_Structure

                            # Plot to check whats happening
                            # print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", Inc_Dp)
                            # PlotF2(Inc_Dp, parent_of_curr_pt, neighbour_dp, Dimension, cluster_num, data1, \
                            #       labels, Incoming_points, train_end, train_start, Roots)

                            if neighbour_idx in Tree_Structure:
                                Tree_Structure[neighbour_idx].append(Inc_Dp_Idx)
                            else:
                                Tree_Structure[neighbour_idx] = [Inc_Dp_Idx]
                            if parent_of_curr_pt_idx != None:
                                Tree_Structure[parent_of_curr_pt_idx].remove(Inc_Dp_Idx)

                            ChangeAllLabels_F(labels, Tree_Structure, Inc_Dp_Idx, labels[neighbour_idx])
                            RemoveNodesWithNoChild(Tree_Structure)
                            break

                print("--> COMPLETED ASSESSING WITH ITS NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
                
                
                
                # Similar to above, find neighbours which are assigned to some other cluster
                # Check the distance between them and their parent and them and probable parent
                # Assign point to the closer parent
                # Change labels of that point and its child nodes and add them to the Tree Structure of the curr_pt
                # What to do if neighbour is a Root --> Check Cardinality. If very small include it
                neighbour_idx_of_different_clusters = GetNeighboursofDifferentCluster(Inc_Dp_Idx, LLE_lookup, labels, data1)
                print("--> ASSESSING NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
                for neighbour_idx in neighbour_idx_of_different_clusters:

                    neighbour_dp = data1[neighbour_idx]
                    parent_of_neighbour_idx = GetParentfromTree_F1(Tree_Structure, neighbour_idx)
                    
                    # If the neighbour is a root node
                    if parent_of_neighbour_idx == None: # or parent_of_curr_dp_idx == None:
                        # print("--> ROOT OBTAINED!! with idx: ", neighbour_idx, "and label:", labels[neighbour_idx])
                        
                        # If root is obtained, check the density of the root with the probable parent.
                        # It is to be done similar to the method of inclusion of a data point (same threshold)
                        # If the root satisfies the density criterion, include it otherwise don't
                        
                        Density_neighbour = Store_Self_Density[neighbour_idx]
                        Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                        list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx, data1)
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        numerator = abs(weighted_density - Density_neighbour)
                        denominator = weighted_density
                        # print("Weighted Density =", numerator/denominator)
                        # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                        
                        # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
                        cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                        if cardinality_neighbour_cluster <= 8 and numerator/denominator <= thresh:
                            # cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                            # cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
                            # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
                            # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
                            #     # print(neighbour_idx, "INCLUDED")
                            ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[Inc_Dp_Idx])
                            RemoveNodesWithNoChild(Tree_Structure)
                            if Inc_Dp_Idx in Tree_Structure:
                                Tree_Structure[Inc_Dp_Idx].append(neighbour_idx)
                            else:
                                Tree_Structure[Inc_Dp_Idx] = [neighbour_idx]
                    
                    
                    else:
                        # If the neighbour is not a root Node
                        
                        if type(parent_of_neighbour_idx) != int:
                            raise Exception("More than 1 idx present(1)")
                        parent_of_neighbour = data1[parent_of_neighbour_idx]
                        # parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, Inc_Dp_Idx)
                        # parent_of_curr_dp = data1[parent_of_curr_dp_idx]
                        # Not the child's parent but check with the child itself.

                        Density_neighbour = Store_Self_Density[neighbour_idx]
                        Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                        list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx, data1)
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        numerator = abs(weighted_density - Density_neighbour)
                        denominator = weighted_density
                        # print("Weighted Density =", numerator/denominator)
                        # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)

                        # Find distances with the present parent and the probable parent.
                        dist_orig_parent = np.linalg.norm(neighbour_dp - parent_of_neighbour)
                        dist_prob_parent = np.linalg.norm(neighbour_dp - Inc_Dp)
                        
                        # Find density differences with the present parent and the probable parent.
                        # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
                        # density_diff_with_prob_parent = Store_Self_Density[Inc_Dp_Idx] - Store_Self_Density[neighbour_idx]
                        
                        ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                        
                        ## print("Idx: ", neighbour_idx, "Label: ", labels[neighbour_idx], "dist_orig_parent: ", dist_orig_parent, "dist_prob_parent: ", dist_prob_parent)
                        if (dist_orig_parent) <= (dist_prob_parent) or numerator/denominator > thresh: ## and (dist_orig_parent) <= thresh_5:
                        ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                            # Everything is fine. Nothing needs to be changed
                            # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
                            # print("--> Everything is fine. Nothing needs to be changed")
                            continue

                        else:
                            # 1. Change the neighbour's parent as the parent of the current data point
                            # 2. If there are children of the neigbour, include them all into this cluster
                            #    and under the current data point in the Tree_Structure

                            # Plot to check whats happening
                            # print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", Inc_Dp)
                            # PlotF2(neighbour_dp, parent_of_neighbour, Inc_Dp, Dimension, cluster_num, data1, \
                            #       labels, Incoming_points, train_end, train_start, Roots)

                            if Inc_Dp_Idx in Tree_Structure:
                                Tree_Structure[Inc_Dp_Idx].append(neighbour_idx)
                            else:
                                Tree_Structure[Inc_Dp_Idx] = [neighbour_idx]
                            if parent_of_neighbour_idx != None:
                                Tree_Structure[parent_of_neighbour_idx].remove(neighbour_idx)

                            ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[Inc_Dp_Idx])
                            RemoveNodesWithNoChild(Tree_Structure)

                print("--> COMPLETED ASSESSING NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
                
                
                ## Assess the Anomaly neighbour points
                neighbour_anomalies = GetNeighbourAnomalies(Inc_Dp_Idx, LLE_lookup, data1)
                print("--> ASSESSING POTENTIAL ANOMALIES")
                # data1, Roots
                data1, Roots, Tree_Structure, Store_Radius_Density, Store_Parents, Store_Self_Density,  cluster_centers, _ = RunStreamingData(data1, 
                         neighbour_anomalies, Roots, labels, Tree_Structure, Store_Radius_Density, 
                         Store_Parents, Hash_Map, thresh, anomaly_list, countimg, Store_Self_Density, cluster_num,
                                        Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, 
                                         cluster_centers, algo, alpha_weight_avg, incoming_time_dur)
                print("--> COMPLETED ASSESSING POTENTIAL ANOMALIES")
                print("----------------------------------Rearranging4----------------------------------")
                RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Incoming_points, Roots)
                
                
                
                
                
                
                
                
            

        else:
            pass
            # print("Incoming Data point Idx: ", Inc_Dp_Idx)
            # print("ALREADY LABELLED AS INLIER SO SKIPPED!!!!")
            
        
        ## For saving Individual plots
        # if labels[Inc_Dp_Idx] == -1:
        #     countimg = Plot(countimg)


        # total_num_pts += 1
        # Update the data1 as new point enters
        # print("Label of", Inc_Dp, Inc_Dp_Idx, "is: ", labels[Inc_Dp_Idx])
        
        
        anomaly_idxs = getanomalyindexes(anomaly_list, data1)
        for label in labels:
            if labels[label] == -1 and (label not in set(anomaly_idxs)):
                anomaly_idxs.append(label)
        for anomaly_idx in anomaly_idxs:
            if labels[anomaly_idx] != -1:
                anomaly_idxs.remove(anomaly_idx)
                
                
                
        print("Current Number of Anomalies: ", len(anomaly_idxs))
        if len(anomaly_idxs) >= Concept_Evolution_Thresh:
            print("*************************PERFORMING CONCEPT EVOLUTION**************************")
            ## data1, Roots, cluster_centers, labels, updated_anomaly_dps, updated_anomaly_idxs, Tree_Structure, \
            ## Store_Self_Density, cluster_num, Store_Radius_Density, Store_Parents, Hash_Map = 
            data1, Roots, cluster_centers, labels, updated_anomaly_dps, updated_anomaly_idxs, Tree_Structure, \
            Store_Self_Density, cluster_num, Store_Radius_Density, Store_Parents, Hash_Map = \
            RunConceptEvolution(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
                         Store_Parents, Hash_Map, thresh, anomaly_list, Store_Self_Density, cluster_num,
                        Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
                       algo, alpha_weight_avg)
            print("********************COMPLETED PERFORMING CONCEPT EVOLUTION*********************")
            
            
        # clear_output(wait=True)    
        
        # Plotting just to see
        #### FinalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)
        
        # Check which are the nodes having no parents. Should be only the roots, if any more then that's wrong
        Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

        # Check if cluster_num is same as the Supposed_Roots: if not then this function corrects it.
        # I am considering that len(Supposed_Roots) > or = cluster_num. Is this correct?
        if len(Supposed_Roots) != cluster_num:
          Tree_Structure = MakeSupposedRootsandClusternumsamelength(Supposed_Roots, labels, Tree_Structure, Store_Self_Density)
          Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)

        Roots = AddRootsFromSupposedRoots(Supposed_Roots, Roots, data1, Store_Self_Density)
        # print("Supposed Roots: ", Supposed_Roots)
        if len(list(Supposed_Roots)) > len(Roots):
            pass
            # print("Error at RunStreamingData")
            # print("\nTree_Structure: \n", Tree_Structure)
            # raise Exception("Number of Supposed Roots has become more than the actual number of roots")
        
        # PlotF1(Inc_Dp, Inc_Dp_Idx, Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)
        
        
        print("******************************DATAPOINT PROCESSED******************************")
        if Bool == False:
            print("Total Datapoints now: ", len(data1))
            
            # Processing Time:
            # This was a new data point and we need to calculate its processing time
            processing_time = time.time() - pro_start_time
            
            # Waiting time:
            # For the very first incoming point the waiting time is 0
            # Once processed, we check the processing time --> If that Processing time > Incoming_time of the new
            # point, then that additional time is going to become the waiting time of the new point
            if processing_time > incoming_time_dur:
                waiting_time = processing_time - incoming_time_dur
            
            # Total Time: Update the total time as processing_time + waiting_time
            Curr_Total = processing_time + curr_waiting_time
            Total_Time += Curr_Total
            
            # Update the waiting time --> as this is for the next incoming data point
            curr_waiting_time = waiting_time
            
            # Display
            print("-----------------------------Time Taken: {} seconds----------------------------".format(Curr_Total))
            print("\n")

     
    # try:
    #     if len(anomaly_idxs) >= Concept_Evolution_Thresh:
    #         print("*************************PERFORMING CONCEPT EVOLUTION**************************")
    #         data1, Roots, cluster_centers, labels, updated_anomaly_dps, updated_anomaly_idxs, Tree_Structure, \
    #         Store_Self_Density, cluster_num, Store_Radius_Density, Store_Parents, Hash_Map = RunConceptEvolution(data1, 
    #         Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
    #                      Store_Parents, Hash_Map, thresh, anomaly_list, Store_Self_Density, cluster_num,
    #                     Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
    #                    algo, alpha_weight_avg)
    #         print("********************COMPLETED PERFORMING CONCEPT EVOLUTION*********************")

    # except:
    #     pass
        
        
    
    # print("!!!!!!!!!!CURRENT CLUSTER LENGTH: ", cluster_num)
    # print("!!!!!!!!!!CURRENT ROOT LENGTH: ", len(Roots))
    # return data1, Roots
    return data1, Roots, Tree_Structure, Store_Radius_Density, Store_Parents, Store_Self_Density,  cluster_centers, Total_Time
#     try:
#         return data1, anomaly_idxs
#     except:
#         return data1, [] 
  

In [ ]:
# # def ConceptEvolutionOne(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
# #                          Store_Parents, Hash_Map, thresh, anomaly_list, Store_Self_Density, cluster_num,
# #                         Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
# #                     algo, alpha_weight_avg):
#     countimg = [0]
#     # data1, anomaly_idxs = RunStreamingData(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
#     #                      Store_Parents, Hash_Map, thresh, anomaly_list, countimg, Store_Self_Density, cluster_num)

#     data1 = RunStreamingData(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
#                          Store_Parents, Hash_Map, thresh, anomaly_list, countimg, Store_Self_Density, cluster_num,
#                             Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
#                     algo, alpha_weight_avg)
    
# #     return data1


In [ ]:
# Get current Anomaly_idxs and Anomaly_dps
def GetCurrentAnomalyIdxsandDps(labels, data1):
    anomaly_idxs = []
    anomaly_dps = []
    for key in labels:
        if labels[key] == -1:
            anomaly_idxs.append(key)
            anomaly_dps.append(data1[key])
    
    anomaly_dict = dict() # Anomaly_dict --> Anomaly_idx:actual_idx
    for i in range(len(anomaly_idxs)):
        anomaly_dict[i] = anomaly_idxs[i]
        
    return anomaly_idxs, anomaly_dps, anomaly_dict


In [ ]:
def PlotSeparationAnomalies(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots):
    
    if Dimension <= 2:

        f = plt.figure(figsize=(16,8))
        plt1 = f.add_subplot(121)
        plt2 = f.add_subplot(122)


        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in range(len(data1)):
                if labels[i] != -1:
                    plt1.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                else:
                    plt1.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
        else:
            for i in range(len(data1)):
                if labels[i] == -1:
                    plt1.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                else:
                    plt1.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')

        for j in range(len(Incoming_points)):
            plt1.scatter(Incoming_points[j][0],Incoming_points[j][1],c='black',marker='*',s=150)    
        plt1.set_title("Datapoints Visualization For Incoming Data Points before Processing")


        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt2.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt2.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt2.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt2.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt2.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt2.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt2.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt2.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        for root in Roots:
            plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=200 )


        plt2.set_title("Datapoints Visualization For Incoming Data Points after Processing")


        plt.show()
        plt.clf()
        plt.cla()
        plt.close()


In [ ]:
def CorrectRootsandClusterCentersandgetCurrentAnomalies(data1, Roots, cluster_centers, labels, Store_Self_Density):
    # Correct Roots and Cluster_Centers and get Current Anomalies (both idxs and dps)

    # print("Data1 length: ", len(data1))

    toremovepairs1 = []
    idxs_1 = list(Roots.keys())
    for i in range(len(idxs_1)-1):
        for j in range(i+1, len(idxs_1)):
            if labels[idxs_1[i]] == labels[idxs_1[j]]:
                toremovepairs1.append([idxs_1[i], idxs_1[j]])

    for pair in toremovepairs1:
        if Store_Self_Density[pair[0]] < Store_Self_Density[pair[1]]:
          try:
            del Roots[pair[0]]
          except:
            pass
        else:
          try:
            del Roots[pair[1]]
          except:
            pass


    for i in range(len(cluster_centers)):
        for key in Roots:
            if labels[key] == cluster_centers[i][1]:
                cluster_centers[i][0] = Roots[key][0].tolist()
                break

                
    def CheckIfSame_1(item1, item2):
        comparison = np.array(item1) == np.array(item2)
        equals_array = comparison.all()
        if equals_array == True:
            return True
        else:
            return False
    
    
    # Remove duplicate values from cluster_centers
    def Removeduplicatesfromclustercenters_1(cluster_centers):
        new_cluster_centers = []
        duplicate_at_idx = set()
        for i in range(len(cluster_centers)-1):
            boolean_f = False
            for j in range(i+1, len(cluster_centers)):
                if CheckIfSame_1(cluster_centers[i][0], cluster_centers[j][0]) == True:
                    duplicate_at_idx.add(j)
        for i in range(len(cluster_centers)):
            if i not in duplicate_at_idx:
                new_cluster_centers.append(cluster_centers[i])
        cluster_centers = new_cluster_centers
        return cluster_centers

        
    if len(cluster_centers) > 1:
        cluster_centers = Removeduplicatesfromclustercenters_1(cluster_centers)
    
    anomaly_idxs, anomaly_dps, anomaly_dict = GetCurrentAnomalyIdxsandDps(labels, data1)


    # print("Updated Roots: ", Roots)
    # print("Cluster_Centers: ", cluster_centers)
    # print("Number of Anomalies: ", len(anomaly_idxs))
    
    return Roots, cluster_centers, anomaly_idxs, anomaly_dps, anomaly_dict

In [ ]:
# print(labels)
# print(Tree_Structure)
# print(CheckTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, K=0))

In [ ]:
# len(data1)

In [ ]:
# When the total number of amonalies reach a certain threshold, we initiate the new cluster formation algorithm
# How it works:
# 1. Select the most dense point from the present anomalies and label it as the new root
# 2. Build a Tree Structure similar to what we have done before

def ReturnIdxs(data1, set_idxs, anomaly_dps, anomaly_dict):
    # idx 1 --> anomaly_dps
    # Need to convert it to idx in data1
    changed_idxs = []
    for idx in set_idxs:
        changed_idxs.append(anomaly_dict[idx])
        
#         item = anomaly_dps[idx]
#         for i in range(len(data1)):
#             comparison = data1[i] == item
#             equal_arrays = comparison.all()
#             if equal_arrays == True:
#                 corresponding_idx = i
#                 break
#         changed_idxs.append(corresponding_idx)
    return changed_idxs
        
    


def FindMostDensePoint(anomaly_dps, algo, Incoming_points, data1, Hash_Map, LLE_lookup, anomaly_idxs, anomaly_dict):
    # print(len(anomaly_idxs))
    # print(anomaly_dps)
    
    # anomaly_dps = []
    # for idx in anomaly_idxs:
    #     try:
    #         anomaly_dps.append(data1[idx])
    #     except:
    #         anomaly_dps.append(Incoming_points[idx-(train_end-train_start)])
    # anomaly_dps = np.asarray(anomaly_dps)
    
    
    # Anomaly_dict--> Anomaly_idx: actual_idx
    
    
    # # Calculate the distance of each point with other points in the dataset
    nbrs_ddcal_1 = NearestNeighbors(n_neighbors=len(anomaly_dps), algorithm=algo).fit(anomaly_dps)
    distances_ddcal_1, indices_ddcal_1 = nbrs_ddcal_1.kneighbors(anomaly_dps)
    print("****************Distance Between All Anomalous Datapoints Computed****************")
    # print("\n") 
    
    ## print("DD_2: ", distances_ddcal_2)
    ## print("ID_2: ", indices_ddcal_2)
    ## distances_ddcal_1 = distances_ddcal_2
    count_5 = 0
    for i in range(len(indices_ddcal_1)):
        count_5 += 1
        send_idxs = indices_ddcal_1[i]
        # print("Count: ", count_5)
        # print("SI: ", send_idxs)
        indices_ddcal_1[i] = ReturnIdxs(data1, send_idxs, anomaly_dps, anomaly_dict)
        if count_5%50 == 0:
            print("****************Indexes Changed for {} Anomalous Datapoints****************".format(count_5))
    # print("\n")    
    ## indices_ddcal_1 = ReturnIdxs(data1, indices_ddcal_2, anomaly_dps)
    ## print(indices_ddcal_1)

    # New Method of finding the densest point
    # Hash_Map_1 = dict() #{i:0 for i in range(len(anomaly_dps))}
    # labels_1 = {i:0 for i in range(len(anomaly_dps))}
    density_1 = []
    
    
    count = 0
    for i in range(len(anomaly_idxs)): #in range(len(data1)):
        count += 1
        opt_K = LLE_lookup[anomaly_idxs[i]]  # 1 already added to K by the LLE function 
        distances_1 = distances_ddcal_1[i][:opt_K]
        idxs_1 = indices_ddcal_1[i][1:opt_K]
        # idxs_1 = ReturnIdxs(data1, idxs_2, anomaly_dps)
        # print("ID2 :", idxs_1)
        # density.append([sum(distances)/(K-1), data1[i], max(distances), idxs, i])
        Radius_1 = max(distances_1)
        # print("Sum: ", sum(distances))
        # print("Area: ", (math.pi*Radius*Radius))
        # print("Density: ", sum(distances)/(math.pi*Radius*Radius))
        temp = [(sum(distances_1)/(math.pi*Radius_1*Radius_1)), anomaly_dps[i], Radius_1, idxs_1, anomaly_idxs[i]]
        density_1.append(temp)
        Hash_Map[anomaly_idxs[i]] = temp
        if (count)%100 == 0:
            print("****************Density Computed for {} Anomalous Datapoints****************".format(count))
            
            
    density_1.sort(key = itemgetter(0), reverse = True)
    
    # print(density_1)
    return density_1, distances_ddcal_1, indices_ddcal_1, Hash_Map



def GetDPsfromIdxs(anomaly_idxs, data1, Incoming_points, train_end, train_start):
    anomaly_dps = []
    for idx in anomaly_idxs:
        try:
            anomaly_dps.append(data1[idx])
        except:
            anomaly_dps.append(Incoming_points[idx-(train_end-train_start)])
    return np.asarray(anomaly_dps)

def ReLabelAnomaliestoZero(labels, anomaly_idxs):
    for idx in anomaly_idxs:
        labels[idx] = 0
    return labels


def RunFindMostDensePoint(anomaly_dps, algo, Incoming_points, data1, Hash_Map, LLE_lookup, anomaly_idxs, anomaly_dict):
    
    # anomaly_dps = GetDPsfromIdxs(anomaly_idxs, data1, Incoming_points, train_end, train_start)
    # print("Length of anomaly_dps: ", len(anomaly_dps), "\n")
    density_1, distances_ddcal_1, indices_ddcal_1, Hash_Map = FindMostDensePoint(anomaly_dps, algo, 
                                        Incoming_points, data1, Hash_Map, LLE_lookup, anomaly_idxs, anomaly_dict)
    dense_pt = density_1[0][1]
    # print(dense_pt)
    # print(density_1[0][-1])
    
    return density_1, distances_ddcal_1, indices_ddcal_1, Hash_Map, dense_pt

In [ ]:
def PlotWithDensePoint(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots, dense_pt):
    if Dimension <= 2:

        plt.figure(figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        for root in Roots:
            plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=100 )

        plt.scatter(dense_pt[0], dense_pt[1], c='gold', marker = 'o', s=100 )


        plt.title("Datapoints Visualization For Incoming Data Points after Processing")

        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

In [ ]:
def ChangeidxLargerdatasettoSmallerdataset(data1, idx, anomaly_dps):
    # idx 1 --> anomaly_dps
    # Need to convert it to idx in data1
    for i in range(len(anomaly_dps)):
        comparison = data1[idx] == anomaly_dps[i]
        equal_arrays = comparison.all()
        if equal_arrays == True:
            corresponding_idx = i
            return corresponding_idx
                

In [ ]:
"""
BFS based Approach: 

It takes care of the maximum recursion depth constraint in Python programming Language.
This however may take more time than DFS
"""
def NovelClus_FF_Streaming(Parent, Hash_Map, dataset, K, list_of_parents_queue, labels, thresh, cluster_num, can_form_cluster, 
                 alpha_weight_avg, Tree_Structure, Roots, Store_Radius_Density, Store_Parents,
                distances_ddcal, indices_ddcal, Store_Self_Density, whole_dataset, anomaly_dps, Incoming_points, 
                   train_end, train_start, LLE_lookup):
    ##print("Cluster =", cluster_num)
    Hold_Root_Density = Parent[0]
    
    # Adding only the root
    Roots[Parent[4]] = [Parent[1],Parent[0]]
    Store_Radius_Density[Parent[4]] = [Parent[2], Parent[0]]
    
    # Add density of the root to Store_Self_Density
    Store_Self_Density[Parent[4]] = Parent[0]
    
    
    # We make use of a Deque for both side operation
    Queue = deque()
    
    # Append from left the Parent Node
    Queue.appendleft(Parent)
    
    
    # print("IP: ", Parent)
    
    
    # While Queue is not Null:
        # Queue extracts the first Node
        # We check if k children of Node satisfies the density criterion
        # if a child satisfies, it is pushed into the Queue from the end; else it is termed -1 and left
    
    while Queue:
        # Pop from the right end
        Parent = Queue.pop()
        # print("P: ", Parent[-1])
            
        # Getting important information from Parent
        Density_Parent = Parent[0]
        Radius = Parent[2]
        #Child_Idx = Parent[3][idx] 
        Child_Idx_array = Parent[3]   # [94,  32, 117, .... 100, 118]
        # print("CI: ", Child_Idx_array)
        #Child = Hash_Map[Child_Idx]
        #Child_Datapoint = Child[1]
        
        # Update the list_of_parents for the weighted moving average
        list_of_parents = list_of_parents_queue.pop() + [Density_Parent] 
        Store_Parents[Parent[4]] = list_of_parents
        #print("LoP =", list_of_parents)
        
        
        Child_array = []
        Child_Datapoint_array = []
        for Child_Idx in Child_Idx_array:
            Child_array.append(Hash_Map[Child_Idx])
            # print("F1: ", Child_Idx, Hash_Map[Child_Idx])
            Child_Datapoint_array.append(Hash_Map[Child_Idx][1])  
        
        # For each Parent find which child satiesfies the density threshold criterion
        ##print("Number of Children Nodes =", len(Child_Idx_array), "\n")
        for i in range(len(Child_Idx_array)):
            
            Child = Child_array[i]
            Child_Idx = Child[-1]             ##Child_Idx_array[i]
            # print("Only CIN: ", Child_Idx)
            Child_Datapoint = Child[1]        ##Child_Datapoint_array[i]
            
            # Base case: if child is already labelled in cluster then ignore
            if labels[Child_Idx] > 0:
                #print("Child is already labelled!!")
                continue
             
            
            # # Old
            # # print("Radius: ", Radius)
            # neigh = NearestNeighbors(radius = Radius)
            # neigh.fit(dataset)
            # rng = neigh.radius_neighbors([Child_Datapoint])
            # print("Prev: ", len(rng[0][0]))
            
            
            Corresponding_Child_Idx = ChangeidxLargerdatasettoSmallerdataset(whole_dataset, Child_Idx, anomaly_dps)
            # print("Changed: ", Corresponding_Child_Idx)
            
            
            # New
            neigh_dist = []
            neigh_idxs = []
            rng = [[],[]]
            for i in range(1, len(distances_ddcal[Corresponding_Child_Idx])):
                if distances_ddcal[Corresponding_Child_Idx][i] < Radius and \
                (labels[indices_ddcal[Corresponding_Child_Idx][i]] == -1 or \
                 labels[indices_ddcal[Corresponding_Child_Idx][i]] == labels[Parent[-1]]):
                    
                    neigh_dist.append(distances_ddcal[Corresponding_Child_Idx][i])
                    # print("NIs: ", indices_ddcal[Child_Idx][i])
                    neigh_idxs.append(np.asarray(int(indices_ddcal[Corresponding_Child_Idx][i])))
                    
                    ## print("Added: ", labels[indices_ddcal[Corresponding_Child_Idx][i]], "to: ", labels[Parent[-1]])
                    
                else:
                    break
            rng[0].append(np.asarray(neigh_dist))
            rng[1].append(np.asarray(neigh_idxs))
            rng = tuple(rng)
            # print("New RNG:", len(rng[0][0]))
            ####
            
            
            if len(rng[0][0]) == 0:
                Density_Child = 0
            else:
                # Density_Child = sum(rng[0][0])/len(rng[0][0])
                Density_Child = sum(rng[0][0])/(math.pi*Radius*Radius)
                
            #print("No. of Parent =",len(list_of_parents))
            
            data_parent = np.array(list_of_parents) #, dtype=np.float64)
            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
            #print("Density of Parent =", weighted_density)
            #print("Density of Child = ", Density_Child)
            numerator = abs(weighted_density - Density_Child)
            denominator = weighted_density
            #print("Weighted Density =", numerator/denominator)
            
            
            # print("CIN (Streaming): ", Child_Idx, "N: ", numerator, "D: ", denominator, "Frac: ", numerator/denominator)
            if numerator/denominator <= thresh:
                # print("Included!!")
                labels[Child_Idx] = cluster_num
                
                list_of_parents_queue.appendleft(list_of_parents)
                       
                # Update the information for this child
                Child[0] = Density_Child
                if len(rng[0][0]) == 0:
                    Child[2] = 0
                else:
                    Child[2] = max(rng[0][0])
                Child[3] = rng[1][0]
                
                
                # Store Adaptive Radius and New Density of the Parent Node for latter usage
                list_of_parents_1 = list_of_parents + [Density_Child]
                data_parent_1 = np.array(list_of_parents_1) #, dtype=np.float64)
                weighted_density_1 = numpy_ewma_vectorized_v2(data_parent_1, alpha_weight_avg)  #O()??
                Store_Radius_Density[Child[4]] = [Child[2], weighted_density_1]
                
                # Add density of child to Store_Self_Density
                Store_Self_Density[Child[4]] = Density_Child
                
                # Add Child to the Tree Structure
                if Parent[4] not in Tree_Structure:
                    Tree_Structure[Parent[4]] = [Child[4]]
                else:
                    Tree_Structure[Parent[4]].append(Child[4])
                    
                    
                #################################################################################################    
                Inc_Dp_Idx = Child_Idx
                Inc_Dp = Child[1]
                # For the point that is just labelled, compare its distance with its present parent and its neighbours 
                # which belong to some different cluster. If the distance and density criteria satisfies,
                # include it to that neighbour cluster and break
                parent_of_curr_pt_idx = GetParentfromTree_F1(Tree_Structure, Inc_Dp_Idx)
                neighbour_idx_of_different_clusters = GetNeighboursofDifferentCluster(Inc_Dp_Idx, LLE_lookup, labels, whole_dataset)
                print("--> ASSESSING WITH ITS NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
                for neighbour_idx in neighbour_idx_of_different_clusters:

                    neighbour_dp = whole_dataset[neighbour_idx]
                    # If the current point has become a root node
                    if parent_of_curr_pt_idx == None: 
                        # print("--> ROOT OBTAINED!! with idx: ", neighbour_idx, "and label:", labels[neighbour_idx])
                        
                        # If root is obtained, check the density of the root with the probable parent.
                        # It is to be done similar to the method of inclusion of a data point (same threshold)
                        # If the root satisfies the density criterion, include it otherwise don't
                        
                        Density_neighbour = Store_Self_Density[neighbour_idx]
                        Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                        list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, neighbour_idx, whole_dataset)
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        numerator = abs(weighted_density - Density_Inc_Dp)
                        denominator = weighted_density
                        # print("Weighted Density =", numerator/denominator)
                        # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                        
                        # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
                        cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
                        if cardinality_curr_cluster <= 8 or numerator/denominator <= thresh:
                            # cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                            # cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
                            # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
                            # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
                            #     # print(neighbour_idx, "INCLUDED")
                            ChangeAllLabels_F(labels, Tree_Structure, Inc_Dp_Idx, labels[neighbour_idx])
                            RemoveNodesWithNoChild(Tree_Structure)
                            if neighbour_idx in Tree_Structure:
                                Tree_Structure[neighbour_idx].append(Inc_Dp_Idx)
                            else:
                                Tree_Structure[neighbour_idx] = [Inc_Dp_Idx]
                            break
                    
                    
                    else:
                        # If the current point is not a root Node
                        
                        if type(parent_of_curr_pt_idx) != int:
                            raise Exception("More than 1 idx present(1)")
                        parent_of_curr_pt = whole_dataset[parent_of_curr_pt_idx]
                        # parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, Inc_Dp_Idx)
                        # parent_of_curr_dp = whole_dataset[parent_of_curr_dp_idx]
                        # Not the child's parent but check with the child itself.

                        Density_neighbour = Store_Self_Density[neighbour_idx]
                        Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                        list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, neighbour_idx, whole_dataset)
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        numerator = abs(weighted_density - Density_Inc_Dp)
                        denominator = weighted_density
                        # print("Weighted Density =", numerator/denominator)
                        # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)

                        # Find distances with the present parent and the probable parent.
                        dist_orig_parent = np.linalg.norm(Inc_Dp - parent_of_curr_pt)
                        dist_prob_parent = np.linalg.norm(Inc_Dp - neighbour_dp)
                        
                        # Find density differences with the present parent and the probable parent.
                        # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
                        # density_diff_with_prob_parent = Store_Self_Density[Inc_Dp_Idx] - Store_Self_Density[neighbour_idx]
                        
                        ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                        
                        ## print("Idx: ", neighbour_idx, "Label: ", labels[neighbour_idx], "dist_orig_parent: ", dist_orig_parent, "dist_prob_parent: ", dist_prob_parent)
                        
                        
                        if (dist_orig_parent)*0.3 <= (dist_prob_parent) and numerator/denominator > thresh: # Changed or to and
                        ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                            # Everything is fine. Nothing needs to be changed
                            # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
                            # print("--> Everything is fine. Nothing needs to be changed")
                            # print("dist_orig_parent", dist_orig_parent, "dist_prob_parent", dist_prob_parent, "frac" , numerator/denominator)
                            continue

                        else:
                            # 1. Change the neighbour's parent as the parent of the current data point
                            # 2. If there are children of the neigbour, include them all into this cluster
                            #    and under the current data point in the Tree_Structure

                            # Plot to check whats happening
                            # print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", Inc_Dp)
                            # PlotF2(Inc_Dp, parent_of_curr_pt, neighbour_dp, Dimension, cluster_num, whole_dataset, \
                            #       labels, Incoming_points, train_end, train_start, Roots)

                            if neighbour_idx in Tree_Structure:
                                Tree_Structure[neighbour_idx].append(Inc_Dp_Idx)
                            else:
                                Tree_Structure[neighbour_idx] = [Inc_Dp_Idx]
                            if parent_of_curr_pt_idx != None:
                                Tree_Structure[parent_of_curr_pt_idx].remove(Inc_Dp_Idx)

                            ChangeAllLabels_F(labels, Tree_Structure, Inc_Dp_Idx, labels[neighbour_idx])
                            RemoveNodesWithNoChild(Tree_Structure)
                            break

                print("--> COMPLETED ASSESSING WITH ITS NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
                #################################################################################################    
                       
                
                Queue.appendleft(Child)
                
                # Update the hashmap OLD
                ##Hash_Map[Child[-1]] = Child
                
                # Update the hashmap NEW
                temp_dict = {Child[-1]:Child}
                Hash_Map.update(temp_dict)
                ##################################
                
            
                # The Parent can form a cluster of its own
                can_form_cluster[0] = [True]
                
                
            else:
                #print("Possible Anomaly!!")
                labels[Child_Idx] = -1
                
            
            # Comment this later
            Dimension = 2
            # PlotF1(Child_Datapoint, Child_Idx, Dimension, cluster_num, whole_dataset, labels, Incoming_points, 
            #        train_end, train_start, Roots)
        

In [ ]:
def BuildModel_Streaming(data1, density, Hash_Map, labels, thresh, alpha_weight_avg, distances_ddcal, indices_ddcal, 
                         start_with, whole_dataset, anomaly_dps, LLE_lookup, Store_Self_Density, train_end, train_start, 
                         Store_Radius_Density, Store_Parents):
    
    data_idx = 0
    cluster_num = start_with
    cluster_centers = []
    hold_cluster_val = cluster_num ####

    # For Storing Tree Structure
    Tree_Structure = dict()
    # Only Roots
    Roots = dict()
    # For Storing Radius and the Weighted Density of Ancestors including itself
    # Store_Radius_Density = dict()
    # For Storing parents
    # Store_Parents = dict()
    # For Storing Self Density
    ## Store_Self_Density = dict()


    for data_idx in range(len(data1)):  #O(n)

        if (data_idx + 1) % 100 == 0:
            print("**************************Computed {} Datapoints**************************".format(data_idx+1))
        idx = 0
        list_of_parents_queue = deque()
        list_of_parents_queue.appendleft([])
        
        
        if labels[density[data_idx][-1]] == 0: #<= 0:
            cluster_num += 1
            labels[density[data_idx][-1]] = cluster_num
            # print("CHECK1: ", density[data_idx][-1], labels[density[data_idx][-1]])
            can_form_cluster = [False]


            # Update information for the Parent node
            #   0                         1         2        3                    4
            # Distance-Density Value; Datapoint; Radius; k-nearest neighbors; idx_number

            K_mod = LLE_lookup[density[data_idx][-1]]      #O(1) Optimized One
            #print("K modified: ",K_mod) # Can be uncommented for better intuition

            #########################################
            # # New One
            # density[data_idx] = update_nbrs(data1, density[data_idx][-1], K_mod, algo)
            # temp_dict = {density[data_idx][-1]:density[data_idx]}
            # #########################################
        
            NovelClus_FF_Streaming(density[data_idx], Hash_Map, data1, K_mod, list_of_parents_queue, labels, thresh, 
                                   cluster_num, can_form_cluster, alpha_weight_avg, Tree_Structure, Roots, 
                                   Store_Radius_Density, Store_Parents, distances_ddcal, indices_ddcal, Store_Self_Density,
                                  whole_dataset, anomaly_dps, data1, train_end, train_start, LLE_lookup)
            
                
            # # Visualization: For higher dimensions this time complexity can be ignored
            # if Dimension <= 2:
            #     plt.figure(figsize=(8,8))
            #     for i in range(len(data1)):
            #         if labels[i] == cluster_num:
            #             plt.scatter(data1[i][0],data1[i][1],c='pink')
            #         elif labels[i] == -1:
            #             plt.scatter(data1[i][0],data1[i][1], c='yellow')
            #         else:
            #             plt.scatter(data1[i][0], data1[i][1],c='blue')
            #     plt.scatter(density[data_idx][1][0], density[data_idx][1][1], c='red',s=200, marker = '+')
            #     plt.show()



            # If the last Parent Node cannot form a clusterof its own then it is a part of a cluster with only 1 datapoint
            # which means " a possible Anomaly"
            # then change its label to -1 and reduce the cluster num by 1
            if can_form_cluster[0] == False:
                labels[density[data_idx][-1]] = -1
                cluster_num = hold_cluster_val  ####
            else:
                cluster_centers.append([density[data_idx][1].tolist(), cluster_num])
                hold_cluster_val += 1  ####
                
            ## print("CC: ", cluster_centers)

    return Roots, cluster_centers, Store_Radius_Density, Store_Parents, Store_Self_Density, Tree_Structure, density             
                

In [ ]:
def OutlierPostProcessing_Streaming(LLE_lookup, algo, anomaly_dps, anomaly_idxs, labels, percent5, alpha_weight_avg, 
                          Store_Radius_Density, distances_ddcal_1, indices_ddcal_1, whole_dataset, 
                                    Store_Parents, Store_Self_Density):
    # Select a predetermined value of K
    # Find the K nearest Neighbors of an outlier
    # For each of those neighbors:
    #     Find their Intracluster distance with the same K value
    #     If the Intracluster distance of outlier is within +-5% of the intracuster distance of the neighbor,
    #     and the neighbor point is not an outlier, 
    #     then consider the outlier to be a part of the smae cluster that the neighbor point belongs.
    
    ##nbrs = NearestNeighbors(n_neighbors=K, algorithm=algo).fit(data1)
    ##distances, indices = nbrs_ddcal.kneighbors(data1)
    
    for idx in anomaly_idxs:
        if labels[idx] == -1:
            # print("Here i: ", i)
            # Predicted Outlier, so find its K nearest neighbors
            #############################################################
            Opt_K = LLE_lookup[idx]
            # print("Opt k: ", Opt_K)
            
            Corresponding_Idx = ChangeidxLargerdatasettoSmallerdataset(whole_dataset, idx, anomaly_dps)
            ## print("Changed: ", Corresponding_Idx)
            
            indices = indices_ddcal_1[Corresponding_Idx][1:Opt_K]
            # print("Indices: ", indices)
            # nbrs = NearestNeighbors(n_neighbors=Opt_K, algorithm=algo).fit(data1)              ########################
            # _, indices = nbrs.kneighbors(data1)
            #############################################################
            
            PredOutlierNNidx = indices               ##[i]
            # print("IDXs: ", PredOutlierNNidx)
            PredOutlierNN = []
            for idx in PredOutlierNNidx:
                PredOutlierNN.append(whole_dataset[idx])
            # Find the intracluster distance for the Predicted Outlier
            PredOutlierICD = IntraClusterDistance(PredOutlierNN)
            # print("Outlier", i, PredOutlierICD)
            
            
            # For each of the outlier's K neighbors
            for idx1 in PredOutlierNNidx[1:]:
                
                # Check if the neighbor is an outlier or inlier
                if labels[idx1] != -1:
                    # print("Not an Outlier")
                    pruned_k = LLE_lookup[idx1]
                    Corresponding_Idx1 = ChangeidxLargerdatasettoSmallerdataset(whole_dataset, idx, anomaly_dps)
                    # print("Changed: ", Corresponding_Idx)
                    NNeighboridx = indices_ddcal_1[Corresponding_Idx1][1:pruned_k]
                    NNeighbor = []
                    for idx2 in NNeighboridx:
                        NNeighbor.append(whole_dataset[idx2])
                        
                    # Find each neighbor's IntraClusterDistance(ICD)
                    NNICD = IntraClusterDistance(NNeighbor)
                    #print("NNICD: ", NNICD)
                    
                    # Check if outlier's ICD is about +-5% of neighbor's ICD
                    # print("Lower: ",(NNICD-percent5*NNICD), "Upper: ",(NNICD+percent5*NNICD), "Self: ",PredOutlierICD)
                    if (NNICD-percent5*NNICD) <= PredOutlierICD <= (NNICD+percent5*NNICD):
                        # Change the label of the Predicted Outlier 'i' to the 
                        # label of the 1st Nearest Neighbor which satifies the condition
                        labels[idx] = labels[idx1]
                        # print("Changed\n")
                        if idx1 not in Tree_Structure:
                            Tree_Structure[idx1] = [idx]
                        else:
                            Tree_Structure[idx1].append(idx)
                            
                        # Update its Store_Radius_Density Value
                        # print("IDX:",idx)
                        
                        # # Old
                        # Radius = Store_Radius_Density[idx][0]
                        # neigh = NearestNeighbors(radius = Radius)
                        # neigh.fit(data1)
                        # rng = neigh.radius_neighbors([data1[i]])
                        
                        # New
                        Radius = Store_Radius_Density[idx1][0]
                        neigh_dist = []
                        neigh_idxs = []
                        rng = [[],[]]
                        for k in range(1, len(distances_ddcal_1[Corresponding_Idx])):
                            if distances_ddcal_1[Corresponding_Idx][k] < Radius:
                                neigh_dist.append(distances_ddcal_1[Corresponding_Idx][k])
                                # print("NIs: ", indices_ddcal[Child_Idx][i])
                                neigh_idxs.append(np.asarray(int(indices_ddcal_1[Corresponding_Idx][k])))
                            else:
                                break
                        rng[0].append(np.asarray(neigh_dist))
                        rng[1].append(np.asarray(neigh_idxs))
                        rng = tuple(rng)
                        # print("New RNG:", len(rng[0][0]))
                        ####
                        
                        
                        # Check if there is no neighbour within the radius
                        # If yes then the density of the recently changed datapoint is 0
                        if len(rng[0][0]) == 0:
                            Density_Changed_Dp = 0
                        else:
                            #Density_Parent = sum(rng[0][0])/len(rng[0][0])       ########################
                            Radius = max(rng[0][0])
                            Density_Changed_Dp = sum(rng[0][0])/(math.pi*Radius*Radius)
                            
                        list_of_parents = Store_Parents[idx1] + [Density_Changed_Dp] 
                        data_parent = np.array(list_of_parents) #, dtype=np.float64)
                        weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                        if len(rng[0][0]) == 0:
                            Radius_new = 0
                        else:
                            Radius_new = max(rng[0][0])
                        Store_Radius_Density[idx] = [Radius_new, weighted_density]
                        Store_Parents[idx] = list_of_parents
                        Store_Self_Density[idx] = Density_Changed_Dp
                        
                        break

            
def IntraClusterDistance(X):
        Intra_Clus_Dist = 0
        for i in range(len(X)):
            Intra_Clus_Dist += (sum(euclidean_distances(X, [X[i]]))[0])/(len(X)-1)
        return Intra_Clus_Dist/(len(X)-1)
    


In [ ]:
def ChangelabelsandRunBuildModelStreaming(labels, anomaly_idxs, anomaly_dps, density_1, Hash_Map, thresh, 
            alpha_weight_avg, distances_ddcal_1, indices_ddcal_1, data1, LLE_lookup, Store_Self_Density, 
                                          train_end, train_start, Roots, Store_Radius_Density, Store_Parents):

    # Changed labels of anomalies to 0 for remodeling
    labels = ReLabelAnomaliestoZero(labels, anomaly_idxs)
    # print(labels)

    # print(len(Hash_Map))
    # print(Hash_Map)
    ## print(len(distances_ddcal_1), len(indices_ddcal_1), len(anomaly_dps))

    # distances_ddcal = distances_ddcal + distances_ddcal_1
    # indices_ddcal = indices_ddcal + indices_ddcal_1
    # print(len(distances_ddcal), len(indices_ddcal), len(anomaly_dps))

    listofkeys = []
    for key in labels:
        if labels[key] != -1 and labels[key] not in set(listofkeys):
            listofkeys.append(labels[key])
        
    start_with = len(Roots) # len(listofkeys)

    


    Roots_1, cluster_centers_1, Store_Radius_Density_1, Store_Parents_1, Store_Self_Density_1, Tree_Structure_1, density_1 = BuildModel_Streaming(anomaly_dps, 
    density_1, Hash_Map, labels, thresh, alpha_weight_avg, distances_ddcal_1, indices_ddcal_1, start_with, data1, 
    anomaly_dps, LLE_lookup, Store_Self_Density, train_end, train_start, Store_Radius_Density, Store_Parents)


    # print(cluster_centers_1)
    # print("\n", len(cluster_centers_1))
    
    return labels, Roots_1, cluster_centers_1, Store_Radius_Density_1, Store_Parents_1, Store_Self_Density_1, Tree_Structure_1, density_1



In [ ]:
def CheckIfSame(item1, item2):
        comparison = np.array(item1) == np.array(item2)
        equals_array = comparison.all()
        if equals_array == True:
            return True
        else:
            return False
    
def CheckIfPresent1(cluster_centers, clus_cen):
    for item in cluster_centers:
        # print("F1: ", item[0], clus_cen[0])
        comparison = np.array(item[0]) == np.array(clus_cen[0])
        equals_array = comparison.all()
        if equals_array == True:
            return True
    return False
    

def GetPresentClusterInformation1(cluster_centers_1, cluster_centers):
    
    if cluster_centers is not None:
        # Remove duplicate values from cluster_centers
        new_cluster_centers = []
        duplicate_at_idx = set()
        for i in range(len(cluster_centers)-1):
            boolean_f = False
            for j in range(i+1, len(cluster_centers)):
                if CheckIfSame(cluster_centers[i][0], cluster_centers[j][0]) == True:
                    duplicate_at_idx.add(j)

        for i in range(len(cluster_centers)):
            if i not in duplicate_at_idx:
                new_cluster_centers.append(cluster_centers[i])

        cluster_centers = new_cluster_centers


    for clus_cen in cluster_centers_1:
        # print(clus_cen[1])
        boolean = False
        for present_clus_cen in cluster_centers:
            # print(present_clus_cen[1], clus_cen[1])
            if present_clus_cen[1] == clus_cen[1]:
                boolean = True
                break
        present = CheckIfPresent1(cluster_centers, clus_cen)
        # print("Present: ", present)
        if boolean == False and present == False:
            cluster_centers.append(clus_cen)


    # print("Present Cluster Centers: \n")
    # print(cluster_centers)
    
    return cluster_centers


In [ ]:
def AdjustRoot_Streaming(Roots, labels):
    seen = set()
    idxs_to_remove = []
    for idx in Roots:
        if labels[idx] in seen or labels[idx] == -1:
            idxs_to_remove.append(idx)
        else:
            seen.add(labels[idx])

    for idx in idxs_to_remove:
        del Roots[idx]
    
    return Roots
    

# print(Roots)
# for key in Roots:
#     print(labels[key])

In [ ]:
def RunCorrectingwrongclusteringAndcorrectthelabels(labels, cluster_centers, data1, Roots_1):
    #######Roots_1, labels, Tree_Structure_1 = Correcting_wrong_clustering(labels, cluster_centers, data1, Roots_1)
    #### For the time being I have commented the above function, so no need of returning anything
    correct_the_idx(labels)
    
    ##return Roots_1, labels, Tree_Structure_1


In [ ]:
def UpdateRootsandCluster1(cluster_centers, Roots, Roots_1, labels):
    
    # Updating the Cluster_Centers

    toremove = []
    for item in cluster_centers:
        boolean = False
        item1 = np.array(item[0])
        for key in Roots:
            item2 = np.array(Roots[key][0])
            comparison1 = item1 == item2
            equals_array1 = comparison1.all()
            if equals_array1 == True:
                boolean = True
                break
        if boolean == False:
            for key1 in Roots_1:
                item3 = np.array(Roots_1[key1][0])
                comparison2 = item1 == item3
                equals_array2 = comparison2.all()
                if equals_array2 == True:
                    boolean = True
                    break
        if boolean == False:
            toremove.append(item)

    new_cluster_centers = []
    for i in range(len(cluster_centers)):
        boolean1 = False
        for j in range(len(toremove)):
            item4 = np.array(cluster_centers[i][0])
            item5 = np.array(toremove[j][0])
            comparison4 = item4 == item5
            equals_array4 = comparison4.all()
            if equals_array4 == True:
                boolean1 = True
                break
        if boolean1 == False:
            new_cluster_centers.append(cluster_centers[i])

    cluster_centers = new_cluster_centers

    # toadd = []
    for key2 in Roots_1:
        boolean5 = False
        item6 = Roots_1[key2][0]
        for item in cluster_centers:
            comparison5 = item[0] == np.array(item6)
            equals_array5 = comparison5.all()
            if equals_array5 == True:
                boolean5 = True
                break
        if boolean5 == False and labels[key2] != -1:
            cluster_centers.append([Roots_1[key2][0].tolist(), labels[key2]])


    # print("Roots: ", Roots)
    # print("Roots_1: ", Roots_1)
    # print("Cluster Centers: ", cluster_centers)
    
    return Roots, Roots_1, cluster_centers

In [ ]:
# labels[271]

In [ ]:
# for idx in anomaly_idxs:
#     print(labels[idx])

In [ ]:
def MergeAll(Tree_Structure, Tree_Structure_1, Store_Self_Density, Store_Self_Density_1, Store_Radius_Density, 
             Store_Radius_Density_1, Store_Parents, Store_Parents_1, Roots, Roots_1, distances_ddcal_1, indices_ddcal_1,
            LLE_lookup, algo, anomaly_dps, anomaly_idxs, labels, percent5, alpha_weight_avg):

    # print("Merging Tree Structures")
    # print(len(Tree_Structure))
    # print(len(Tree_Structure_1))
    Tree_Structure = {**Tree_Structure, **Tree_Structure_1}
    # print(len(Tree_Structure))

    # print("\nMerging Store_Self_Density")
    # print(len(Store_Self_Density))
    # print(len(Store_Self_Density_1))
    Store_Self_Density = {**Store_Self_Density, **Store_Self_Density_1}
    # print(len(Store_Self_Density))

    # print("\nMerging Store_Radius_Density")
    # print(len(Store_Radius_Density))
    # print(len(Store_Radius_Density_1))
    Store_Radius_Density = {**Store_Radius_Density, **Store_Radius_Density_1}
    # print(len(Store_Radius_Density))

    # print("\nMerging Store_Parents")
    # print(len(Store_Parents))
    # print(len(Store_Parents_1))
    Store_Parents = {**Store_Parents, **Store_Parents_1}
    # print(len(Store_Parents))

    # print("\nMerging Roots")
    # print(len(Roots))
    # print(len(Roots_1))
    Roots = {**Roots, **Roots_1}
    # print(len(Roots))
    # Roots = AdjustRoot_Streaming(Roots, labels)
    # print("Current Roots (will be updated again): ", len(Roots))

    # print("\n")
    cluster_num = len(Roots)
    # print("Cluster Num: ", cluster_num)

    ######### I have uncommented this
#     OutlierPostProcessing_Streaming(LLE_lookup, algo, anomaly_dps, anomaly_idxs, labels, percent5, alpha_weight_avg, 
#                               Store_Radius_Density, distances_ddcal_1, indices_ddcal_1, data1, 
#                                     Store_Parents, Store_Self_Density)

    
    return Tree_Structure, Store_Self_Density, Store_Radius_Density, Store_Parents, Roots, cluster_num



In [ ]:
def UPDATEROOTS(Roots, cluster_centers, Store_Self_Density):
    toremovepairs1 = []
    idxs_1 = list(Roots.keys())
    for i in range(len(idxs_1)-1):
        for j in range(i+1, len(idxs_1)):
            if labels[idxs_1[i]] == labels[idxs_1[j]]:
                toremovepairs1.append([idxs_1[i], idxs_1[j]])
    # print("To Remove: ", toremovepairs1, "Roots: ", Roots)
    for pair in toremovepairs1:
        if Store_Self_Density[pair[0]] < Store_Self_Density[pair[1]]:
            if pair[0] in Roots:
                del Roots[pair[0]]
        else:
            if pair[1] in Roots:
                del Roots[pair[1]]
            
    # If one root has become an anomaly
    toremove2 = []
    for key in Roots:
        if labels[key] == -1:
            toremove2.append(key)
    for key_item in toremove2:
        del Roots[key_item]
        
            
    for i in range(len(cluster_centers)):
        for key in Roots:
            if labels[key] == cluster_centers[i][1]:
                cluster_centers[i][0] = Roots[key][0].tolist()
                break
            
                
    return Roots, cluster_centers

def UpdateRootsAndClusterCentersAgain(Roots, cluster_centers, Store_Self_Density):
    Roots, cluster_centers = UPDATEROOTS(Roots, cluster_centers, Store_Self_Density)
    
    def CheckIfSame_2(item1, item2):
        comparison = np.array(item1) == np.array(item2)
        equals_array = comparison.all()
        if equals_array == True:
            return True
        else:
            return False
    
    # Remove duplicate values from cluster_centers
    def Removeduplicatesfromclustercenters_2(cluster_centers):
        new_cluster_centers = []
        duplicate_at_idx = set()
        for i in range(len(cluster_centers)-1):
            boolean_f = False
            for j in range(i+1, len(cluster_centers)):
                if CheckIfSame_2(cluster_centers[i][0], cluster_centers[j][0]) == True:
                    duplicate_at_idx.add(j)
        for i in range(len(cluster_centers)):
            if i not in duplicate_at_idx:
                new_cluster_centers.append(cluster_centers[i])
        cluster_centers = new_cluster_centers
        return cluster_centers

        
    if len(cluster_centers) > 1:
        cluster_centers = Removeduplicatesfromclustercenters_2(cluster_centers)
        
    
    # Make Cluster Center and Roots same
    new_cluster_center2 = []
    for item in cluster_centers:
        boolean = False
        item1 = item[0]
        for root in Roots:
            item2 = Roots[root][0]
            comparison = np.array(item1) == np.array(item2)
            equals_array = comparison.all()
            if equals_array == True:
                boolean = True
                break
        if boolean == True:
            new_cluster_center2.append(item)
    
    cluster_centers = new_cluster_center2
    
    
    # print("Current Updated Roots: ", Roots)
    # print("Current Updated Cluster Centers: ", cluster_centers)
    cluster_num = len(Roots)
    # print("Current Updated cluster_num: ", cluster_num)
    
    return Roots, cluster_centers, cluster_num


In [ ]:
# print("Current Updated Roots: ", Roots)
# print("Current Updated Cluster Centers: ", cluster_centers)
# cluster_num = len(Roots)
# print("Current Updated cluster_num: ", cluster_num)

In [ ]:
def InitalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots):
    
    if Dimension <= 2:
        plt.figure(figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        for root in Roots:
            plt.scatter(Roots[root][0][0], Roots[root][0][1], c='black', marker = 'o', s=200 )

        # plt.scatter(dense_pt[0], dense_pt[1], c='gold', marker = 'o', s=100 )


        plt.title("Datapoints Visualization For Incoming Data Points after Concept Evolution")

        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

In [ ]:
def RunRearranging5(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Roots):
    print("----------------------------------Rearranging5----------------------------------")
    RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, data1, Roots)
    correct_the_idx(labels)
    return labels

In [ ]:
# For all the anomaly points:
# 1. Find their neighbours using pruned k
# 2. If all neighbours belong to the same cluster as itself, then do nothing
# 3. If neighbour is an anomaly then do the same thing while building Tree in Streaming for anomaly
# 4. If neighbour belongs to a different cluster, check its distance with its parent and the parent of the point
#    under consideration. Whichever is lesser insert it into that.


# When a neighbour is the root of a cluster, check its cardinality.
# If that is less than a percent include it with the curr_idx label along with its children nodes
#


# Function to return the set of labels for all neighbour points
def ReturnSetofLabels(labels, neighbour_idxs):
    return_labels = set()
    for idx in neighbour_idxs:
        if labels[idx] not in return_labels:
            return_labels.add(labels[idx])
    return return_labels


def GetParentfromTree(Tree_Structure, idx):
    send_key = None
    for key in Tree_Structure:
        if idx in set(Tree_Structure[key]):
            send_key = key
            break
    if send_key is None:
        return send_key
    else:
        return int(send_key)
    

def GetCardinality(labels, idx):
    cardinality = 0
    label_idx = labels[idx]
    for key in labels:
        if labels[key] == label_idx:
            cardinality += 1
    return cardinality


# def ChangeAllLabels(labels, Tree_Structure, idx, changetolabel):
#     ToChange = deque()
#     ToChange.append(idx)
#     while ToChange:
#         curr_idx = ToChange.pop()
#         labels[curr_idx] = changetolabel
#         if curr_idx in Tree_Structure:
#             for child_idx in Tree_Structure[curr_idx]:
#                 ToChange.append(child_idx)
#     return labels



def CheckandCorrectStreaming(data1, anomaly_idxs, LLE_lookup, anomaly_dps, Roots, Store_Radius_Density, 
                             Store_Parents, Hash_Map, thresh, Store_Self_Density, Tree_Structure, labels, 
                             Concept_Evolution_Thresh, cluster_centers, Dimension, train_end, train_start,
                              algo, alpha_weight_avg):
    
    thresh_5 = 10 #(for corners)
    percent7 = 0.05 # 0.5  #0.05(For Correct One till now)
    
    for i in range(len(anomaly_dps)):
        curr_idx = anomaly_idxs[i]
        curr_dp = anomaly_dps[i]
        
        if labels[curr_idx] != -1:
            ## 1. Find their neighbours using pruned k
            opt_k = LLE_lookup[anomaly_idxs[i]]
            nearest_neighbour_idxs = GetNeighbours(anomaly_dps[i], data1, opt_k)
            # print("NNI: ", nearest_neighbour_idxs)


            SetofLabels = ReturnSetofLabels(labels, nearest_neighbour_idxs)
            ## 2. If all neighbours belong to the same cluster as itself, then do nothing
            if len(SetofLabels) == 1 and SetofLabels.pop() != -1:
                # print("--> All neighbours belong to the same cluster and are not anomalies")
                continue


            ## 3. If neighbour belongs to a different cluster, check its distance with its parent and the parent of the point
            ##    under consideration. Whichever is lesser insert it into that.
            elif len(SetofLabels) > 1:
                # print("--> Neighbours belong to different clusters")
                # To do
                # Neighbours can either belong to different cluster or can be anomalies
                # For neighbours belonging to other clusters:
                for neighbour_idx in nearest_neighbour_idxs:
                    # print("Label of ", neighbour_idx, "is ", labels[neighbour_idx], "Curr idx label: ", labels[curr_idx])
                    if labels[curr_idx] == labels[neighbour_idx]:
                        # Same label as the point under consideration so do nothing
                        continue

                    elif labels[curr_idx] != labels[neighbour_idx] and labels[neighbour_idx] != -1:
                        # print("Label of neighbour: ", labels[neighbour_idx], neighbour_idx)
                        # neighbour belongs to some other cluster
                        # print("NI: ", neighbour_idx)
                        neighbour_dp = data1[neighbour_idx]
                        parent_of_neighbour_idx = GetParentfromTree(Tree_Structure, neighbour_idx)
                        
                        
                        # When a neighbour is the root of a cluster, check its cardinality.
                        # If that is less than a percent include it with the curr_idx label along with its children nodes
                        if parent_of_neighbour_idx == None: # or parent_of_curr_dp_idx == None:
                            # print("--> ROOT OBTAINED!!")
                            
                            Density_neighbour = Store_Self_Density[neighbour_idx]
                            Density_Inc_Dp = Store_Self_Density[curr_idx]
                            list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, curr_idx, data1)
                            data_parent = np.array(list_of_parents) #, dtype=np.float64)
                            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                            numerator = abs(weighted_density - Density_neighbour)
                            denominator = weighted_density
                            # print("Weighted Density =", numerator/denominator)
                            # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                            
                            # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
                            cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                            if cardinality_neighbour_cluster <= 8 or numerator/denominator <= thresh:
                                # cardinality_neighbour_cluster = GetCardinality(labels, neighbour_idx)
                                # cardinality_curr_cluster = GetCardinality(labels, curr_idx)
                                # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
                                # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
                                #     # print(neighbour_idx, "INCLUDED")
                                ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[curr_idx])
                                RemoveNodesWithNoChild(Tree_Structure)
                                if curr_idx in Tree_Structure:
                                    Tree_Structure[curr_idx].append(neighbour_idx)
                                else:
                                    Tree_Structure[curr_idx] = [neighbour_idx]
                                # labels[neighbour_idx] = labels[curr_idx]
                                # if neighbour_idx in Tree_Structure:
                                #     for child_idx in Tree_Structure[neighbour_idx]:
                                #         labels[child_idx] = labels[curr_idx]
                        
                        
                        else:
                            # If the neighbour is not a root node
                            # print("PNI: ", parent_of_neighbour_idx)
                            if type(parent_of_neighbour_idx) != int:
                                raise Exception("More than 1 idx present(2)")
                            parent_of_neighbour = data1[parent_of_neighbour_idx]
                            # parent_of_curr_dp_idx = GetParentfromTree(Tree_Structure, curr_idx)
                            # parent_of_curr_dp = data1[parent_of_curr_dp_idx]

                            # print("PI: ", parent_of_neighbour_idx)

                            Density_neighbour = Store_Self_Density[neighbour_idx]
                            Density_Inc_Dp = Store_Self_Density[curr_idx]
                            list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, curr_idx, data1)
                            data_parent = np.array(list_of_parents) #, dtype=np.float64)
                            weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                            numerator = abs(weighted_density - Density_neighbour)
                            denominator = weighted_density
                            # print("Weighted Density =", numerator/denominator)
                            # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                                
                            # Find distances with the present parent and the probable parent.
                            dist_orig_parent = np.linalg.norm(neighbour_dp - parent_of_neighbour)
                            dist_prob_parent = np.linalg.norm(neighbour_dp - curr_dp)
                            
                            # Find density differences with the present parent and the probable parent.
                            # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
                            # density_diff_with_prob_parent = Store_Self_Density[curr_idx] - Store_Self_Density[neighbour_idx]
                        
                            ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                        
                            if (dist_orig_parent)*0.3 <= (dist_prob_parent) or numerator/denominator > thresh: ## and (dist_orig_parent) <= thresh_5:
                            ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                                # Everything is fine. Nothing needs to be changed
                                # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
                                # print("--> Everything is fine. Nothing needs to be changed")
                                continue

                            else:
                                # 1. Change the neighbour's parent as the parent of the current data point
                                # 2. If there are children of the neigbour, include them all into this cluster
                                #    and under the current data point in the Tree_Structure

                                # Plot to check whats happening
                                # Plot to check whats happening
                                # print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", curr_dp)
                                # PlotF2(neighbour_dp, parent_of_neighbour, curr_dp, Dimension, cluster_num, data1, \
                                #       labels, Incoming_points, train_end, train_start, Roots)
                                
                                if curr_idx in Tree_Structure:
                                    Tree_Structure[curr_idx].append(neighbour_idx)
                                else:
                                    Tree_Structure[curr_idx] = [neighbour_idx]
                                if parent_of_neighbour_idx != None:
                                    Tree_Structure[parent_of_neighbour_idx].remove(neighbour_idx)

                                ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[curr_idx])
                                RemoveNodesWithNoChild(Tree_Structure)
                                # labels[neighbour_idx] = labels[curr_idx]
                                # if neighbour_idx in Tree_Structure:
                                #     for child_idx in Tree_Structure[neighbour_idx]:
                                #         labels[child_idx] = labels[curr_idx]

                            
                    else:
                        ## 3. If neighbour is an anomaly then do the same thing while building Tree in Streaming for anomaly
                        # print("--> Anomaly neighbour faced!!")
                        pass
                        # To do

    
    # clear_output(wait=True)
    
    # Get the remaining anomalies
    updated_anomaly_idxs = []
    updated_anomaly_dps = [] 
    ########### I changed this
#     for key in labels:
#         if labels[key] == -1:
#             updated_anomaly_idxs.append(key)
#             updated_anomaly_dps.append(data1[key])
#     updated_anomaly_dps = np.asarray(updated_anomaly_dps)
    
    for idx5 in anomaly_idxs:
        if labels[idx5] == -1:
            updated_anomaly_idxs.append(idx5)
            updated_anomaly_dps.append(data1[idx5])
    updated_anomaly_dps = np.asarray(updated_anomaly_dps)
            
    
    
    
    print("--> ASSESSING POTENTIAL ANOMALIES")
    correct_the_idx(labels)
    Roots = AdjustRoot_Streaming(Roots, labels)
    ## Roots, cluster_centers = UPDATEROOTS(Roots, cluster_centers, Store_Self_Density)
    countimg = [0]   ## Not interest in plotting now. Afterwards it can be uncommented
    cluster_num_1 = len(Roots)
    # print("Cluster Num2: ", cluster_num_1)
    
    # data1, Roots
    data1, Roots, Tree_Structure, Store_Radius_Density, Store_Parents, Store_Self_Density,  cluster_centers, _ = RunStreamingData(data1, updated_anomaly_dps, Roots, labels, Tree_Structure, 
                                           Store_Radius_Density, Store_Parents, Hash_Map, thresh, 
                                           updated_anomaly_dps, countimg, Store_Self_Density, cluster_num_1,
                                          Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start,
                                         cluster_centers, algo, alpha_weight_avg, incoming_time_dur)
    
    
    print("--> COMPLETED ASSESSING POTENTIAL ANOMALIES")
    print("----------------------------------Rearranging6----------------------------------")
    RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num_1, data1, Roots)
    
    
    
    
    # Get the remaining anomalies
    updated_anomaly_idxs = []
    updated_anomaly_dps = []
    ########### I changed this
#     for key in labels:
#         if labels[key] == -1:
#             updated_anomaly_idxs.append(key)
#             updated_anomaly_dps.append(data1[key])
    for idx6 in anomaly_idxs:
        if labels[idx6] == -1:
            updated_anomaly_idxs.append(idx6)
            updated_anomaly_dps.append(data1[idx6])
    updated_anomaly_dps = np.asarray(updated_anomaly_dps)

    return updated_anomaly_idxs, updated_anomaly_dps




In [ ]:
def RunCheckandCorrectStreaming(data1, anomaly_idxs, LLE_lookup, anomaly_dps, 
                                Roots, Store_Radius_Density, Store_Parents, Hash_Map, thresh,
                                Store_Self_Density, Tree_Structure, labels, Concept_Evolution_Thresh, cluster_centers,
                               Dimension, train_end, train_start, algo, alpha_weight_avg):
    
    updated_anomaly_idxs, updated_anomaly_dps = CheckandCorrectStreaming(data1, anomaly_idxs, LLE_lookup, anomaly_dps, 
                                Roots, Store_Radius_Density, Store_Parents, Hash_Map, thresh,
                                Store_Self_Density, Tree_Structure, labels, Concept_Evolution_Thresh, cluster_centers,
                                Dimension, train_end, train_start, algo, alpha_weight_avg)
    
    return updated_anomaly_idxs, updated_anomaly_dps


In [ ]:
def UpdateRootsandCluster2(Roots, cluster_centers, Store_Self_Density):
    # Roots = AdjustRoot_Streaming(Roots, labels)
    Roots, cluster_centers = UPDATEROOTS(Roots, cluster_centers, Store_Self_Density)
    # print("Current Updated Roots: ", Roots)

    # print("\n")
    cluster_num = len(Roots)
    # print("Current Cluster Num: ", cluster_num)
    
    return Roots, cluster_centers, cluster_num

In [ ]:
def RunRearranging7(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Roots):
    print("----------------------------------Rearranging7----------------------------------")
    RearrangeTreeStructure(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, data1, Roots)
    correct_the_idx(labels)
    return labels

In [ ]:
def PlotF1(Inc_Dp, Inc_Dp_Idx, Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots):
    
    if Dimension <= 2:
        
        fig, ax = plt.subplots()
        
        # ax = plt.figure(figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        ax.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        ax.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        ax.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        ax.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        ax.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        ax.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        ax.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        ax.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        ax.scatter(Inc_Dp[0], Inc_Dp[1], c='black', marker = 'o')
        ax.annotate('(%.1f)'%(Inc_Dp_Idx), xy=(Inc_Dp[0], Inc_Dp[1]))



        ax.set_title("Datapoints Visualization For Incoming Data Points after Processing")

        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

In [ ]:
def PlotF2(Child_Dp, Orig_Parent_Dp, Prob_Parent_Dp, Dimension, cluster_num, data1, \
           labels, Incoming_points, train_end, train_start, Roots):
    
    if Dimension <= 2:
                
        plt.figure() #figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        plt.scatter(Child_Dp[0], Child_Dp[1], c='black', marker = 'X', label = 'Child')
        plt.scatter(Orig_Parent_Dp[0], Orig_Parent_Dp[1], c='black', marker = 'o', label = 'Orig_Parent')
        plt.scatter(Prob_Parent_Dp[0], Prob_Parent_Dp[1], c='black', marker = '^', label = 'Prob Parent')

        plt.title("Datapoints Visualization For Incoming Data Points after Processing")
        plt.legend()
        
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

In [ ]:
def FinalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots):
    
    if Dimension <= 2:
        plt.figure(figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        # for root in Roots:
        #     plt.scatter(Roots[root][0][0], Roots[root][0][1], c='black', marker = 'o', s=200 )



        plt.title("Datapoints Visualization For Incoming Data Points after Processing")

        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

In [ ]:
def DisplayInformation(updated_anomaly_idxs, Roots):
    print("Number of current Anomalies: ", len(updated_anomaly_idxs))
    # Roots = AdjustRoot_Streaming(Roots, labels)
    # print("Roots: ", Roots)

In [ ]:
def ComputeDistIdxs(data1, algo):
    nbrs_ddcal = NearestNeighbors(n_neighbors=len(data1), algorithm=algo).fit(data1)
    distances_ddcal, indices_ddcal = nbrs_ddcal.kneighbors(data1)
    print("****************Completed Computing Distance Between All Datapoints****************")
    return distances_ddcal, indices_ddcal

def GetCardinalityofIdx(labels, label_idx):
    Cardinality_idx = 0
    for key in labels:
        if labels[key] == label_idx:
            Cardinality_idx += 1
    return Cardinality_idx


def IdxofDp(dp, data1):
    for i in range(len(data1)):
        comparison = dp == data1[i]
        equals_array = comparison.all()
        if equals_array == True:
            return i
        
        
def GetParentfromTree_F1(Tree_Structure, idx):
    send_key = None
    for key in Tree_Structure:
        if idx in set(Tree_Structure[key]):
            send_key = key
            break
    # print("Sending: ", send_key)
    if send_key is None:
        return send_key
    else:
        return int(send_key)


# This function is already defined as ChangeAllLabels_F
# def ChangeAllLabels_F1(labels, Tree_Structure, idx, changetolabel):
#     ToChange = deque()
#     ToChange.append(idx)
#     visited = {key:False for key in labels}
#     while True:
#         # print("ToChange: ", ToChange)
#         if len(ToChange) == 0:
#             break
#         curr_idx = ToChange.pop()
#         labels[curr_idx] = changetolabel
#         if curr_idx in Tree_Structure:
#             for child_idx in Tree_Structure[curr_idx]:
#                 if child_idx not in set(ToChange) and visited[child_idx] == False:
#                     ToChange.appendleft(child_idx)
#                     visited[child_idx] = True
                    
#     return labels


def CorrectSpecificLabel(dps, data1, labels, indices_ddcal, LLE_lookup, Tree_Structure):
    
    Tree_Structure = RemoveDulplicatePositions(Tree_Structure, Store_Self_Density)
    Tree_Structure = CorrectOneChildTwoParents(Tree_Structure, Store_Self_Density)
    
    percent7 = 0.05 #0.05
    
    thresh_5 = 10 #(for corners)
    
    for i in range(len(dps)):
        curr_dp = dps[i]
        idxofdp = IdxofDp(dps[i], data1)

        # Check only if curr data point is not an anomaly
        if labels[idxofdp] != -1:
            
          opt_K = LLE_lookup[idxofdp]
          neighbour_idxs = indices_ddcal[i][1:opt_K]

          for neighbour_idx in neighbour_idxs:
              if labels[neighbour_idx] != labels[idxofdp] and labels[neighbour_idx] != -1:
                  # print("NEED WORK!!")
                  # To do
                  
                  # print("NI: ", neighbour_idx)
                  neighbour_dp = data1[neighbour_idx]
                  parent_of_neighbour_idx = GetParentfromTree_F1(Tree_Structure, neighbour_idx)
                  
                  
                  if parent_of_neighbour_idx == None: # or parent_of_curr_dp_idx == None:
                      # print("--> ROOT OBTAINED!!")
                      
                      Density_neighbour = Store_Self_Density[neighbour_idx]
                      Density_Inc_Dp = Store_Self_Density[idxofdp]
                      list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, idxofdp, data1)
                      data_parent = np.array(list_of_parents) #, dtype=np.float64)
                      weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                      numerator = abs(weighted_density - Density_neighbour)
                      denominator = weighted_density
                      # print("Weighted Density =", numerator/denominator)
                      # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                      
                      cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                      if cardinality_neighbour_cluster <= 10 or numerator/denominator <= thresh:
                          # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
                          # cardinality_neighbour_cluster = GetCardinalityofIdx(labels, labels[neighbour_idx])
                          # cardinality_curr_cluster = GetCardinalityofIdx(labels, labels[idxofdp])
                          # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
                          # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
                          #     # print(neighbour_idx, "INCLUDED")
                          ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[idxofdp])
                          RemoveNodesWithNoChild(Tree_Structure)
                          if idxofdp in Tree_Structure:
                              Tree_Structure[idxofdp].append(neighbour_idx)
                          else:
                              Tree_Structure[idxofdp] = [neighbour_idx]
                      
                  
                  else:
                      # If neighbour is not a root node
                      if type(parent_of_neighbour_idx) != int:
                          raise Exception("More than 1 idx present(3)")
                      parent_of_neighbour = data1[parent_of_neighbour_idx]
                      # parent_of_curr_dp_idx = GetParentfromTree_F1(Tree_Structure, idxofdp)
                      # parent_of_curr_dp = data1[parent_of_curr_dp_idx]


                      Density_neighbour = Store_Self_Density[neighbour_idx]
                      Density_Inc_Dp = Store_Self_Density[idxofdp]
                      list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, idxofdp, data1)
                      data_parent = np.array(list_of_parents) #, dtype=np.float64)
                      weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                      numerator = abs(weighted_density - Density_neighbour)
                      denominator = weighted_density
                      # print("Weighted Density =", numerator/denominator)
                      # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
      
                      # Find distances with the present parent and the probable parent.
                      dist_orig_parent = np.linalg.norm(neighbour_dp - parent_of_neighbour)
                      dist_prob_parent = np.linalg.norm(neighbour_dp - curr_dp)
                      
                      # Find density differences with the present parent and the probable parent.
                      # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
                      # density_diff_with_prob_parent = Store_Self_Density[idxofdp] - Store_Self_Density[neighbour_idx]
          
                      ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                          
                      if (dist_orig_parent)*0.3 <= (dist_prob_parent) and numerator/denominator > thresh: # and (dist_orig_parent) <= thresh_5:
                      ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                          # Everything is fine. Nothing needs to be changed
                          # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
                          # print("--> Everything is fine. Nothing needs to be changed")
                        
                          continue

                      else:
                          # 1. Change the neighbour's parent as the parent of the current data point
                          # 2. If there are children of the neigbour, include them all into this cluster
                          #    and under the current data point in the Tree_Structure

                          # Plot to check whats happening
                          # print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", curr_dp)
                          # PlotF2(neighbour_dp, parent_of_neighbour, curr_dp, Dimension, cluster_num, data1, \
                          #       labels, Incoming_points, train_end, train_start, Roots)
                            
                          if idxofdp in Tree_Structure:
                              Tree_Structure[idxofdp].append(neighbour_idx)
                          else:
                              Tree_Structure[idxofdp] = [neighbour_idx]
                          if parent_of_neighbour_idx != None:
                              Tree_Structure[parent_of_neighbour_idx].remove(neighbour_idx)
                          
                          ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[idxofdp])
                          RemoveNodesWithNoChild(Tree_Structure)
                    

                  
                  
              else:
                  # Same label so do nothing
                  continue

    # clear_output(wait=True)



In [ ]:
def Correct1(labels, data1, algo, LLE_lookup, Tree_Structure, Store_Self_Density, alpha_weight_avg, thresh):

  # Check neighbours of all datapoints. If there exists members from other clusters, check there distance with their original parent 
  # and the curr point under consideration. Whichever is smaller, include in that.

  for Inc_Dp_Idx in range(len(data1)):
    Inc_Dp = data1[Inc_Dp_Idx]

    # Check only if curr data point is not an anomaly
    if labels[Inc_Dp_Idx] != -1:
      neighbour_idx_of_different_clusters = GetNeighboursofDifferentCluster(Inc_Dp_Idx, LLE_lookup, labels, data1)

      if len(neighbour_idx_of_different_clusters) > 0:
        # There are neighbours belonging to different cluster, so need to check
        print("--> ASSESSING NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")

        for neighbour_idx in neighbour_idx_of_different_clusters:

            neighbour_dp = data1[neighbour_idx]
            parent_of_neighbour_idx = GetParentfromTree_F1(Tree_Structure, neighbour_idx)
            
            
            # If the neighbour is a root node
            if parent_of_neighbour_idx == None: # or parent_of_curr_dp_idx == None:
                # print("--> ROOT OBTAINED!! with idx: ", neighbour_idx, "and label:", labels[neighbour_idx])
                
                # If root is obtained, check the density of the root with the probable parent.
                # It is to be done similar to the method of inclusion of a data point (same threshold)
                # If the root satisfies the density criterion, include it otherwise don't
                
                Density_neighbour = Store_Self_Density[neighbour_idx]
                Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
                list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx, data1)
                data_parent = np.array(list_of_parents) #, dtype=np.float64)
                weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
                numerator = abs(weighted_density - Density_neighbour)
                denominator = weighted_density
                # print("Weighted Density =", numerator/denominator)
                # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                
                # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
                cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                if cardinality_neighbour_cluster <= 8 and numerator/denominator <= thresh:
                    # cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
                    # cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
                    # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
                    # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
                    #     # print(neighbour_idx, "INCLUDED")
                    ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[Inc_Dp_Idx])
                    RemoveNodesWithNoChild(Tree_Structure)
                    if Inc_Dp_Idx in Tree_Structure:
                        Tree_Structure[Inc_Dp_Idx].append(neighbour_idx)
                    else:
                        Tree_Structure[Inc_Dp_Idx] = [neighbour_idx]
            
            
            else:
              # If the neighbour is not a root Node
              
              if type(parent_of_neighbour_idx) != int:
                raise Exception("More than 1 idx present(1)")
              parent_of_neighbour = data1[parent_of_neighbour_idx]
              # parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, Inc_Dp_Idx)
              # parent_of_curr_dp = data1[parent_of_curr_dp_idx]
              # Not the child's parent but check with the child itself.

              Density_neighbour = Store_Self_Density[neighbour_idx]
              Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
              list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx, data1)
              data_parent = np.array(list_of_parents) #, dtype=np.float64)
              weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
              numerator = abs(weighted_density - Density_neighbour)
              denominator = weighted_density
              # print("Weighted Density =", numerator/denominator)
              # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)

              # Find distances with the present parent and the probable parent.
              dist_orig_parent = np.linalg.norm(neighbour_dp - parent_of_neighbour)
              dist_prob_parent = np.linalg.norm(neighbour_dp - Inc_Dp)
              
              # Find density differences with the present parent and the probable parent.
              # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
              # density_diff_with_prob_parent = Store_Self_Density[Inc_Dp_Idx] - Store_Self_Density[neighbour_idx]
              
              ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
              
              ## print("Idx: ", neighbour_idx, "Label: ", labels[neighbour_idx], "dist_orig_parent: ", dist_orig_parent, "dist_prob_parent: ", dist_prob_parent)
              ## For the Correct1 I am not including the density criteria
              if (dist_orig_parent)*0.3 <= (dist_prob_parent) and numerator/denominator > thresh: ## and (dist_orig_parent) <= thresh_5:
              ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
                # print("HERE1")
                # Everything is fine. Nothing needs to be changed
                # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
                # print("--> Everything is fine. Nothing needs to be changed")
                # print("HERE2")
                continue

              else:
                # print("HERE3")
                # 1. Change the neighbour's parent as the parent of the current data point
                # 2. If there are children of the neigbour, include them all into this cluster
                #    and under the current data point in the Tree_Structure

                # Plot to check whats happening
                # print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", Inc_Dp)
                # PlotF2(neighbour_dp, parent_of_neighbour, Inc_Dp, Dimension, cluster_num, data1, \
                #       labels, Incoming_points, train_end, train_start, Roots)

                if Inc_Dp_Idx in Tree_Structure:
                  Tree_Structure[Inc_Dp_Idx].append(neighbour_idx)
                else:
                  Tree_Structure[Inc_Dp_Idx] = [neighbour_idx]
                if parent_of_neighbour_idx != None:
                  Tree_Structure[parent_of_neighbour_idx].remove(neighbour_idx)

                # print("HERE4")
                ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[Inc_Dp_Idx])
                RemoveNodesWithNoChild(Tree_Structure)
                # print("HERE5")

        print("--> COMPLETED ASSESSING NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
        
  # clear_output(wait=True)  







    # Seen_1 = set()
    # for key in labels:
    #     if labels[key] not in Seen_1 and labels[key] != -1:
    #         Seen_1.add(labels[key])
    # print("Labels: ", Seen_1)

    # See_1_list = list(Seen_1)


    # Cardinality_list = []
    # for idx in Seen_1:
    #     Cardinality_idx = GetCardinalityofIdx(labels, idx)
    #     Cardinality_list.append(Cardinality_idx)

    # print("Cardinality: ", Cardinality_list)

    # Joined = []
    # for i in range(len(Cardinality_list)):
    #     Joined.append([Cardinality_list[i], See_1_list[i]])

    # # print(Joined)

    # Joined.sort(key=itemgetter(0), reverse = True)

    # print("Combined: ", Joined)


    # distances_ddcal, indices_ddcal = ComputeDistIdxs(data1, algo)

    # for pair in Joined:
    #     curr_label = pair[1]
    #     send_dps = []
    #     for dp in data1:
    #         idxofdp = IdxofDp(dp, data1)
    #         if labels[idxofdp] == curr_label:
    #             send_dps.append(dp)

    #     CorrectSpecificLabel(send_dps, data1, labels, indices_ddcal, LLE_lookup, Tree_Structure)


# Roots, cluster_centers, cluster_num = UpdateRootsandCluster2(Roots, cluster_centers, Store_Self_Density)
# labels = RunRearranging7(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Roots)
# FinalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)


In [ ]:
# def Correct2(labels, data1, algo, LLE_lookup, Tree_Structure, Store_Self_Density, alpha_weight_avg, thresh):

#   # Check neighbours of all datapoints. If there exists members from other clusters, check there distance with their original parent 
#   # and the curr point under consideration. Whichever is smaller, include in that.

#   for Inc_Dp_Idx in range(len(data1)):
#     Inc_Dp = data1[Inc_Dp_Idx]

#     # Check only if curr data point is not an anomaly
#     if labels[Inc_Dp_Idx] != -1:
#       neighbour_idx_of_different_clusters = GetNeighboursofDifferentCluster(Inc_Dp_Idx, LLE_lookup, labels, data1)

#       if len(neighbour_idx_of_different_clusters) > 0:
#         # There are neighbours belonging to different cluster, so need to check
#         print("--> ASSESSING NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")

#         for neighbour_idx in neighbour_idx_of_different_clusters:

#             neighbour_dp = data1[neighbour_idx]
#             parent_of_neighbour_idx = GetParentfromTree_F1(Tree_Structure, neighbour_idx)
            
            
#             # If the neighbour is a root node
#             if parent_of_neighbour_idx == None: # or parent_of_curr_dp_idx == None:
#                 # print("--> ROOT OBTAINED!! with idx: ", neighbour_idx, "and label:", labels[neighbour_idx])
                
#                 # If root is obtained, check the density of the root with the probable parent.
#                 # It is to be done similar to the method of inclusion of a data point (same threshold)
#                 # If the root satisfies the density criterion, include it otherwise don't
                
#                 Density_neighbour = Store_Self_Density[neighbour_idx]
#                 Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
#                 list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx)
#                 data_parent = np.array(list_of_parents) #, dtype=np.float64)
#                 weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
#                 numerator = abs(weighted_density - Density_neighbour)
#                 denominator = weighted_density
#                 # print("Weighted Density =", numerator/denominator)
#                 # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)
                
#                 # print("++++Numerator: ", numerator, "Denominator: ", denominator, "Frac: ", numerator/denominator)
#                 cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
#                 if cardinality_neighbour_cluster <= 8 and numerator/denominator <= thresh:
#                     # cardinality_neighbour_cluster = GetCardinality_F(labels, neighbour_idx)
#                     # cardinality_curr_cluster = GetCardinality_F(labels, Inc_Dp_Idx)
#                     # # print("Percent required: ", cardinality_neighbour_cluster/cardinality_curr_cluster)
#                     # if cardinality_neighbour_cluster/cardinality_curr_cluster <= percent7:
#                     #     # print(neighbour_idx, "INCLUDED")
#                     ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[Inc_Dp_Idx])
#                     RemoveNodesWithNoChild(Tree_Structure)
#                     if Inc_Dp_Idx in Tree_Structure:
#                         Tree_Structure[Inc_Dp_Idx].append(neighbour_idx)
#                     else:
#                         Tree_Structure[Inc_Dp_Idx] = [neighbour_idx]
            
            
#             else:
#               # If the neighbour is not a root Node
              
#               if type(parent_of_neighbour_idx) != int:
#                 raise Exception("More than 1 idx present(1)")
#               parent_of_neighbour = data1[parent_of_neighbour_idx]
#               # parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, Inc_Dp_Idx)
#               # parent_of_curr_dp = data1[parent_of_curr_dp_idx]
#               # Not the child's parent but check with the child itself.

#               Density_neighbour = Store_Self_Density[neighbour_idx]
#               Density_Inc_Dp = Store_Self_Density[Inc_Dp_Idx]
#               list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, Inc_Dp_Idx)
#               data_parent = np.array(list_of_parents) #, dtype=np.float64)
#               weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
#               numerator = abs(weighted_density - Density_neighbour)
#               denominator = weighted_density
#               # print("Weighted Density =", numerator/denominator)
#               # print("CIN: ", Child_Idx, "N: ", numerator, "D: ", denominator)

#               # Find distances with the present parent and the probable parent.
#               dist_orig_parent = np.linalg.norm(neighbour_dp - parent_of_neighbour)
#               dist_prob_parent = np.linalg.norm(neighbour_dp - Inc_Dp)
              
#               # Find density differences with the present parent and the probable parent.
#               # density_diff_with_orig_parent = Store_Self_Density[parent_of_neighbour_idx] - Store_Self_Density[neighbour_idx]
#               # density_diff_with_prob_parent = Store_Self_Density[Inc_Dp_Idx] - Store_Self_Density[neighbour_idx]
              
#               ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
              
#               ## print("Idx: ", neighbour_idx, "Label: ", labels[neighbour_idx], "dist_orig_parent: ", dist_orig_parent, "dist_prob_parent: ", dist_prob_parent)
#               ## For the Correct1 I am not including the density criteria
#               if (dist_orig_parent)*0.3 <= (dist_prob_parent): # or numerator/denominator > thresh: ## and (dist_orig_parent) <= thresh_5:
#               ## if (dist_orig_parent+density_diff_with_orig_parent) <= (dist_prob_parent+density_diff_with_prob_parent):
#                 # print("HERE1")
#                 # Everything is fine. Nothing needs to be changed
#                 # print("F: ", dist_orig_parent, "\t", dist_prob_parent)
#                 # print("--> Everything is fine. Nothing needs to be changed")
#                 # print("HERE2")
#                 continue

#               else:
#                 # print("HERE3")
#                 # 1. Change the neighbour's parent as the parent of the current data point
#                 # 2. If there are children of the neigbour, include them all into this cluster
#                 #    and under the current data point in the Tree_Structure

#                 # Plot to check whats happening
#                 print("NDp: ", neighbour_dp, "PN: ", parent_of_neighbour, "ProP: ", Inc_Dp)
#                 PlotF2(neighbour_dp, parent_of_neighbour, Inc_Dp, Dimension, cluster_num, data1, \
#                       labels, Incoming_points, train_end, train_start, Roots)

#                 if Inc_Dp_Idx in Tree_Structure:
#                   Tree_Structure[Inc_Dp_Idx].append(neighbour_idx)
#                 else:
#                   Tree_Structure[Inc_Dp_Idx] = [neighbour_idx]
#                 if parent_of_neighbour_idx != None:
#                   Tree_Structure[parent_of_neighbour_idx].remove(neighbour_idx)

#                 # print("HERE4")
#                 ChangeAllLabels_F(labels, Tree_Structure, neighbour_idx, labels[Inc_Dp_Idx])
#                 RemoveNodesWithNoChild(Tree_Structure)
#                 # print("HERE5")

#         print("--> COMPLETED ASSESSING NEIGHBOURS BELONGING TO DIFFERENT CLUSTER")
        
#   # clear_output(wait=True)  







#     # Seen_1 = set()
#     # for key in labels:
#     #     if labels[key] not in Seen_1 and labels[key] != -1:
#     #         Seen_1.add(labels[key])
#     # print("Labels: ", Seen_1)

#     # See_1_list = list(Seen_1)


#     # Cardinality_list = []
#     # for idx in Seen_1:
#     #     Cardinality_idx = GetCardinalityofIdx(labels, idx)
#     #     Cardinality_list.append(Cardinality_idx)

#     # print("Cardinality: ", Cardinality_list)

#     # Joined = []
#     # for i in range(len(Cardinality_list)):
#     #     Joined.append([Cardinality_list[i], See_1_list[i]])

#     # # print(Joined)

#     # Joined.sort(key=itemgetter(0), reverse = True)

#     # print("Combined: ", Joined)


#     # distances_ddcal, indices_ddcal = ComputeDistIdxs(data1, algo)

#     # for pair in Joined:
#     #     curr_label = pair[1]
#     #     send_dps = []
#     #     for dp in data1:
#     #         idxofdp = IdxofDp(dp, data1)
#     #         if labels[idxofdp] == curr_label:
#     #             send_dps.append(dp)

#     #     CorrectSpecificLabel(send_dps, data1, labels, indices_ddcal, LLE_lookup, Tree_Structure)


# # Roots, cluster_centers, cluster_num = UpdateRootsandCluster2(Roots, cluster_centers, Store_Self_Density)
# # labels = RunRearranging7(Tree_Structure, data1, labels, Store_Self_Density, cluster_num, Roots)
# # FinalPlot(Dimension, cluster_num, data1, labels, Incoming_points, train_end, train_start, Roots)


In [ ]:
# Name: Get Incoming Points

if 'Name: Get Incoming Points' not in SM:
    data_all = SM['data_all']
    test_start = SM['test_start']
    test_end = SM['test_end']

    # """
    # print(Roots)
    # Incoming_points = np.array([[5,5],
    #                    [5,-5],
    #                    [-10,5],
    #                    [-6,-6],
    #                    [1,1],
    #                    [1,-1],
    #                    [-1,1],
    #                    [-1,-1],
    #                    [-12,-12],
    #                    [3,3],
    #                    [10,-3],
    #                    [-2,-10],
    #                    [-2,12],
    #                    [-10,2],
    #                    [-5,-5]])

    # """

    Incoming_points = data_all[test_start:test_end+1]
    print("Testing Length: ", len(Incoming_points))

    SM['Incoming_points'] = Incoming_points
    SM['Name: Get Incoming Points'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Display5

if 'Name: Display5' not in SM:
    
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    Incoming_points = SM['Incoming_points']
    Roots = SM['Roots']
    
    # """
    if Dimension <= 2:
        plt.figure(figsize=(8,8))

        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']

        if cluster_num <= len(color):
            for i in range(len(data1)):
                if labels[i] != -1:
                    plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                else:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
        else:
            for i in range(len(data1)):
                if labels[i] == -1:
                    plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                else:
                    plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')

        for j in range(len(Incoming_points)):
            plt.scatter(Incoming_points[j][0],Incoming_points[j][1],c='black',marker='*',s=200)

        for root in Roots:
            plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=200 )



        plt.xlabel('X-Coordinates')
        plt.ylabel('Y-Coordinates')
        plt.title('Datapoints Visualization After Outlier Post Processing')
        plt.show()
        plt.clf()
        plt.cla()
        plt.close()
    # """
    
    SM['Name: Display5'] = True
    SM.sync()
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Run Streaming Data

if 'Name: Run Streaming Data' not in SM:
    data1 = SM['data1']
    Incoming_points = SM['Incoming_points']
    Roots = SM['Roots']
    labels = SM['labels']
    Tree_Structure = SM['Tree_Structure']
    Store_Radius_Density = SM['Store_Radius_Density']
    Store_Parents = SM['Store_Parents']
    Hash_Map = SM['Hash_Map']
    thresh = SM['thresh']
    anomaly_list = SM['anomaly_list']
    Store_Self_Density = SM['Store_Self_Density']
    cluster_num = SM['cluster_num']
    Concept_Evolution_Thresh = SM['Concept_Evolution_Thresh']
    LLE_lookup = SM['LLE_lookup']
    Dimension = SM['Dimension']
    train_end = SM['train_end']
    train_start = SM['train_start']
    cluster_centers = SM['cluster_centers']
    algo = SM['algo']
    alpha_weight_avg = SM['alpha_weight_avg']
    incoming_time_dur = SM['incoming_time_dur']
    

    countimg = [0]

    # print(Tree_Structure)
    # Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)
    # print(Supposed_Roots)


    data1, Roots, Tree_Structure, Store_Radius_Density, Store_Parents, Store_Self_Density,  cluster_centers, Total_Time = \
                        RunStreamingData(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
                             Store_Parents, Hash_Map, thresh, anomaly_list, countimg, Store_Self_Density, cluster_num,
                                Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
                        algo, alpha_weight_avg, incoming_time_dur)

    # Correct1(labels, data1, algo, LLE_lookup, Tree_Structure, Store_Self_Density, alpha_weight_avg, thresh)
    # Correct2(labels, data1, algo, LLE_lookup, Tree_Structure, Store_Self_Density, alpha_weight_avg, thresh)

    SM['data1'] = data1
    SM['Roots'] = Roots
    SM['Tree_Structure'] = Tree_Structure
    SM['Store_Radius_Density'] = Store_Radius_Density
    SM['Store_Self_Density'] = Store_Self_Density
    SM['cluster_centers'] = cluster_centers
    SM['Total_Time'] = Total_Time
    SM['labels'] = labels
    SM['LLE_lookup'] = LLE_lookup
    SM['cluster_num'] = cluster_num
    SM['anomaly_list'] = anomaly_list
    SM['Hash_Map'] = Hash_Map
    SM['Store_Parents'] = Store_Parents

    SM['Name: Run Streaming Data'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: RunConceptEvolution Last Time

if 'Name: RunConceptEvolution Last Time' not in SM:
    
    data1 = SM['data1']
    Incoming_points = SM['Incoming_points']
    Roots = SM['Roots']
    labels = SM['labels']
    Tree_Structure = SM['Tree_Structure']
    Store_Radius_Density = SM['Store_Radius_Density']
    Store_Parents = SM['Store_Parents']
    Hash_Map = SM['Hash_Map']
    thresh = SM['thresh']
    anomaly_list = SM['anomaly_list']
    Store_Self_Density = SM['Store_Self_Density']
    cluster_num = SM['cluster_num']
    Concept_Evolution_Thresh = SM['Concept_Evolution_Thresh']
    LLE_lookup = SM['LLE_lookup']
    Dimension = SM['Dimension']
    train_end = SM['train_end']
    train_start = SM['train_start']
    cluster_centers = SM['cluster_centers']
    algo = SM['algo']
    alpha_weight_avg = SM['alpha_weight_avg']
    incoming_time_dur = SM['incoming_time_dur']
    
    # print("Inc:", incoming_time_dur)

    ## Run the Concept Evolution for the last time
    data1, Roots, cluster_centers, labels, updated_anomaly_dps, updated_anomaly_idxs, \
    Tree_Structure, Store_Self_Density, cluster_num, Store_Radius_Density, Store_Parents, Hash_Map = \
    RunConceptEvolution(data1, Incoming_points, Roots, labels, Tree_Structure, Store_Radius_Density, 
                         Store_Parents, Hash_Map, thresh, anomaly_list, Store_Self_Density, cluster_num,
                        Concept_Evolution_Thresh, LLE_lookup, Dimension, train_end, train_start, cluster_centers,
                       algo, alpha_weight_avg)

    SM['data1'] = data1
    SM['Roots'] = Roots
    SM['Tree_Structure'] = Tree_Structure
    SM['Store_Radius_Density'] = Store_Radius_Density
    SM['Store_Parents'] = Store_Parents
    SM['Store_Self_Density'] = Store_Self_Density
    SM['cluster_centers'] = cluster_centers
    SM['labels'] = labels
    SM['LLE_lookup'] = LLE_lookup
    SM['cluster_num'] = cluster_num
    SM['anomaly_list'] = anomaly_list
    SM['Hash_Map'] = Hash_Map

    SM['Name: RunConceptEvolution Last Time'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# for item in Tree_Structure.keys():
#     print("item: ", item, "labels: ", labels[item])

In [ ]:
# Name: Display6

if 'Name: Display6' not in SM:
    
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    Incoming_points = SM['Incoming_points']
    Roots = SM['Roots']
    train_end = SM['train_end']
    train_start = SM['train_start']

    if Dimension <= 2:
        plt.figure(figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        # for root in Roots:
        #     plt.scatter(Roots[root][0][0], Roots[root][0][1], c='black', marker = 'o', s=200 )



        plt.title("Datapoints Visualization For Incoming Data Points after Processing")

        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

        SM['Name: Display6'] = True
        SM.sync()
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# dist_with_parent = []
# for key in labels:
#     if labels[key] == 1:
#         dp = data1[key]
#         parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, key)
#         if parent_of_curr_dp_idx is not None:
#             dist_with_parent.append([np.linalg.norm(dp - data1[parent_of_curr_dp_idx]), key])
        
# dist_with_parent.sort(key=itemgetter(0), reverse = True)

# print(dist_with_parent)


# dist_with_parent_1 = []
# for key in labels:
#     if labels[key] == 2:
#         dp = data1[key]
#         parent_of_curr_dp_idx = GetParentfromTree_F(Tree_Structure, key)
#         if parent_of_curr_dp_idx is not None:
#             dist_with_parent_1.append([np.linalg.norm(dp - data1[parent_of_curr_dp_idx]), key])
        
# dist_with_parent_1.sort(key=itemgetter(0), reverse = True)

# print("\n", dist_with_parent_1)



In [ ]:
# Name: Check Which Nodes Have No Parent In Tree

if 'Name: Check Which Nodes Have No Parent In Tree' not in SM:
    Tree_Structure = SM['Tree_Structure']
    # Roots
    Supposed_Roots = CheckWhichNodesHaveNoParentInTree(Tree_Structure)
    print(Supposed_Roots)

    SM['Supposed_Roots'] = Supposed_Roots
    
    SM['Name: Check Which Nodes Have No Parent In Tree'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Operation 1

if 'Name: Operation 1' not in SM:
    data1 = SM['data1']
    algo = SM['algo']
    labels = SM['labels']
    Store_Radius_Density = SM['Store_Radius_Density']
    Store_Parents = SM['Store_Parents']

    nbrs_ddcal_updated = NearestNeighbors(n_neighbors=len(data1), algorithm=algo).fit(data1)
    distances_ddcal_updated, indices_ddcal_updated = nbrs_ddcal_updated.kneighbors(data1)

    for i in range(len(data1)):
        if labels[i] == -1:
            Store_Radius_Density[i] = [0, 0]
            Store_Parents[i] = [0]
            Store_Radius_Density[i] = [0]

    SM['Store_Radius_Density'] = Store_Radius_Density
    SM['Store_Parents'] = Store_Parents
    with open('distances_ddcal_updated.p', 'wb') as pfile1:
        pickle.dump(distances_ddcal_updated, pfile1, protocol=4)
    # SM['distances_ddcal_updated'] = distances_ddcal_updated
    with open('indices_ddcal_updated.p', 'wb') as pfile2:
        pickle.dump(indices_ddcal_updated, pfile2, protocol=4)
    # SM['indices_ddcal_updated'] = indices_ddcal_updated
    # SM['nbrs_ddcal_updated'] = nbrs_ddcal_updated
    
    SM['Name: Operation 1'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Show1

if 'Name: Show1' not in SM:
    Store_Self_Density = SM['Store_Self_Density']
    Store_Parents = SM['Store_Parents']
    Store_Self_Density = SM['Store_Self_Density']
    

    print(len(Store_Self_Density))
    print(len(Store_Parents))
    
    SM['Name: Show1'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Check If Present1

if 'Name: Check If Present1' not in SM:
    Roots = SM['Roots']
    cluster_centers = SM['cluster_centers']
    data1 = SM['data1']
    labels = SM['labels']
    
    def CheckifPresent(cluster_centers, item1):
        boolean = False
        for pair in cluster_centers:
            item2 = pair[0]
            comparison = item1 == item2
            equals_array = comparison.all()
            if equals_array == True:
                boolean = True
                break
        return boolean


    for root in Roots:
        item1 = Roots[root][0]
        if CheckifPresent(cluster_centers, item1) == False:
            idx_item1 = GetIdxFromWholeDataset(data1, item1)
            label_item1 = labels[idx_item1]
            cluster_centers.append([item1.tolist(), label_item1])

    SM['cluster_centers'] = cluster_centers

    SM['Name: Check If Present1'] = True

    # print(cluster_centers)

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Operation2

if 'Name: Operation2' not in SM:
    cluster_centers = SM['cluster_centers']
    labels = SM['labels']
    data1 = SM['data1']
    Roots = SM['Roots']
    Store_Self_Density = SM['Store_Self_Density']
    
    
    # Getting already present labels
    curr_center_idxs = set()
    for item in cluster_centers:
        if item[1] not in curr_center_idxs:
            curr_center_idxs.add(item[1])
    # print(curr_center_idxs)


    # Getting labels which have not been considered as cluster centers
    labels_to_add = set()
    idxs_to_add = dict()
    for key in labels:
        if labels[key] not in curr_center_idxs and labels[key] != -1:
            labels_to_add.add(labels[key])
            if labels[key] not in idxs_to_add:
                idxs_to_add[labels[key]] = key
    print(labels_to_add)
    print(idxs_to_add)


    # Adding the left-over labels to cluster_centers
    for label in list(labels_to_add):
        idx_label = idxs_to_add[label]
        item1 = data1[idx_label]
        cluster_centers.append([item1.tolist(), label])

    # Adding the left-over labels to Roots as well
    for label in list(labels_to_add):
        idx_label = idxs_to_add[label]
        item1 = data1[idx_label]
        Roots[idx_label] = [item1, Store_Self_Density[idx_label]]
        
    SM['Roots'] = Roots
    SM['cluster_centers'] = cluster_centers
    
    SM['Name: Operation2'] = True

else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Outlier Post Processing Last Time

if 'Name: Outlier Post Processing Last Time' not in SM:
    labels = SM['labels']
    cluster_centers = SM['cluster_centers']
    data1 = SM['data1']
    Roots = SM['Roots']
    Tree_Structure = SM['Tree_Structure']
    LLE_lookup = SM['LLE_lookup']
    algo = SM['algo']
    alpha_weight_avg = SM['alpha_weight_avg']
    Store_Radius_Density = SM['Store_Radius_Density']
    with open('distances_ddcal_updated.p', 'rb') as pfile1:
        distances_ddcal_updated = pickle.load(pfile1)
    # distances_ddcal_updated = SM['distances_ddcal_updated']
    with open('indices_ddcal_updated.p', 'rb') as pfile2:
        indices_ddcal_updated = pickle.load(pfile2)
    # indices_ddcal_updated = SM['indices_ddcal_updated']
    Store_Parents = SM['Store_Parents']
    Store_Self_Density = SM['Store_Self_Density']

    Roots, labels, Tree_Structure = Correcting_wrong_clustering(labels, cluster_centers, data1, Roots, Tree_Structure)

    OutlierPostProcessing(LLE_lookup, algo, data1, labels, percent5, alpha_weight_avg, 
                              Store_Radius_Density, distances_ddcal_updated, indices_ddcal_updated, Store_Parents, 
                          Store_Self_Density, Tree_Structure)


    SM['Roots'] = Roots
    SM['labels'] = labels
    SM['Tree_Structure'] = Tree_Structure
    SM['Name: Outlier Post Processing Last Time'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Display7

if 'Name: Display7' not in SM:
    
    Dimension = SM['Dimension']
    labels = SM['labels']
    data1 = SM['data1']
    cluster_num = SM['cluster_num']
    Incoming_points = SM['Incoming_points']
    Roots = SM['Roots']
    train_end = SM['train_end']
    train_start = SM['train_start']

    if Dimension <= 2:
        plt.figure(figsize=(8,8))
        color = ['red', 'aqua', 'yellow', 'magenta', 'black', 'darkgreen', 'blue', 'gray', 'purple', 'lime', 'maroon', 'gold', 'deeppink', 'silver', 'lawngreen', 'pink', 'navy', 'blueviolet', 'turquoise', 'dodgerblue', 'navajowhite', 'khaki', 'darkslateblue', 'darkseagreen', 'mediumvioletred', 'palevioletred', 'cornflowerblue', 'plum', 'steelblue', 'lightcoral']
        if cluster_num <= len(color):
            for i in labels:
                # print("i: ", i, "i-limit_end: ", i-limit_end, "i+train_start: ", i+train_start, "i-(train_end-train_start)", i-(train_end-train_start))
                if labels[i] != -1:
                    try:
                        plt.scatter(data1[i][0], data1[i][1], c=color[labels[i]-1])
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1], c=color[labels[i]-1])

                else:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except IndexError:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)


        else:
            for i in labels:
                if labels[i] == -1:
                    try:
                        plt.scatter(data1[i][0],data1[i][1],c='green',marker='*',s=150)
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0],Incoming_points[i-(train_end-train_start)][1],c='green',marker='*',s=150)

                else:
                    try:
                        plt.scatter(data1[i][0], data1[i][1],c='blue', label='Datapoints')
                    except:
                        plt.scatter(Incoming_points[i-(train_end-train_start)][0], Incoming_points[i-(train_end-train_start)][1],c='blue', label='Datapoints')


        # for root in Roots:
        #     plt.scatter(Roots[root][0][0], Roots[root][0][1], c='gold', marker = 'o', s=200 )



        plt.title("Datapoints Visualization For Incoming Data Points after Processing")

        plt.show()
        plt.clf()
        plt.cla()
        plt.close()

        SM['Name: Display7'] = True
        SM.sync()
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Roots

In [ ]:
# nbrs_ddcal = NearestNeighbors(n_neighbors=len(data1), algorithm=algo).fit(data1)
# distances_ddcal, indices_ddcal = nbrs_ddcal.kneighbors(data1)

# for i in range(len(data1)):
    
#     if labels[i] != -1:
#         parent_of_curr_pt = GetParentfromTree_F1(Tree_Structure, i)
#         Orig_Parent_Dp = data1[parent_of_curr_pt]
#         Inc_Dp = data1[i]
#         opt_k = LLE_lookup[i]
#         neighbour_idxs = indices_ddcal[i][1:opt_k]

#         for neighbour_idx in neighbour_idxs:
#             if labels[neighbour_idx] != labels[i]:
#                 neighbour_dp = data1[neighbour_idx]
                
                
#                 if parent_of_curr_pt == None:
#                     Density_neighbour = Store_Self_Density[neighbour_idx]
#                     Density_Inc_Dp = Store_Self_Density[i]
#                     list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, neighbour_idx)
#                     data_parent = np.array(list_of_parents) #, dtype=np.float64)
#                     weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
#                     numerator = abs(weighted_density - Density_Inc_Dp)
#                     denominator = weighted_density
#                     cardinality_curr_cluster = GetCardinality_F(labels, i)
#                     if cardinality_curr_cluster <= 8 and numerator/denominator <= thresh:
#                         print("CHANGE DESPITE BEING ROOT")
                    
#                 else:
#                     # if parent_of_curr_pt is not None:
#                     # Find distances with the present parent and the probable parent.
#                     dist_orig_parent = np.linalg.norm(Inc_Dp - Orig_Parent_Dp)
#                     dist_prob_parent = np.linalg.norm(Inc_Dp - neighbour_dp)
                    
#                     Density_neighbour = Store_Self_Density[neighbour_idx]
#                     Density_Inc_Dp = Store_Self_Density[i]
#                     list_of_parents = GetListofDensityofParents(Store_Self_Density, Tree_Structure, neighbour_idx)
#                     data_parent = np.array(list_of_parents) #, dtype=np.float64)
#                     weighted_density = numpy_ewma_vectorized_v2(data_parent, alpha_weight_avg)  #O()??
#                     numerator = abs(weighted_density - Density_Inc_Dp)
#                     denominator = weighted_density

#                     if (dist_orig_parent)*0.3 >= dist_prob_parent and numerator/denominator <= thresh:
#                         print("INC_Dp: ", Inc_Dp, "Orig_Parent_Dp: ", Orig_Parent_Dp)
#                         print("(dist_orig_parent)*0.3: ", (dist_orig_parent)*0.3, "dist_prob_parent: ", dist_prob_parent,\
#                               "frac: ", numerator/denominator)
#                         PlotF2(Inc_Dp, Orig_Parent_Dp, neighbour_dp, Dimension, cluster_num, data1, \
#                                labels, Incoming_points, train_end, train_start, Roots)
#                         if neighbour_idx in Tree_Structure:
#                             Tree_Structure[neighbour_idx].append(i)
#                         else:
#                             Tree_Structure[neighbour_idx] = [i]
#                         if parent_of_curr_pt != None:
#                             Tree_Structure[parent_of_curr_pt].remove(i)

#                         ChangeAllLabels_F(labels, Tree_Structure, i, labels[neighbour_idx])
#                         RemoveNodesWithNoChild(Tree_Structure)
#             #             print("")
#             #             labels[i] = labels[neighbour_idx]
#                         break


    

In [ ]:
# Name: Show2

if 'Name: Show2' not in SM:
    data_all = SM['data_all']
    data1 = SM['data1']
    labels = SM['labels']
    
    print(len(data_all))
    print(len(data1))
    print(len(labels))

    # for i in range(len(data_all)):
    #     if i not in labels:
    #         print("F", i)
            
    SM['Name: Show2'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Outlier Detection

if 'Name: Outlier Detection' not in SM:
    Outlier_Detection = SM['Outlier_Detection']
    
    if Outlier_Detection:
        def OutlierDetectionEvaluation(actual_labels, pred_labels, data1, labels_Idxs):
            # confusion matrix in sklearn
            from sklearn.metrics import confusion_matrix
            from sklearn.metrics import classification_report

            # actual values
            actual = actual_labels #[1,0,0,1,0,0,1,0,0,1]
            # predicted values
            predicted = pred_labels #[1,0,0,1,0,0,0,1,0,0]

            if len(actual) != len(predicted):
                trim = len(predicted)
                actual = actual[:trim]

            # confusion matrix
            matrix = confusion_matrix(actual,predicted, labels=[1,0])
            #print('Confusion matrix : \n',matrix)

            # outcome values order in sklearn
            tp, fn, fp, tn = confusion_matrix(actual,predicted,labels=[1,0]).reshape(-1)
            #print('Outcome values : \n', tp, fn, fp, tn)

            # classification report for precision, recall f1-score and accuracy
            matrix = classification_report(actual,predicted,labels=[1,0])
            print('Classification report : \n',matrix)


            print("\n")
            DB_Index = davies_bouldin_score(data1, labels_Idxs)
            print("DB Index =", DB_Index)
            distances = pairwise_distances(data1)
            Dunn_Index = dunn(distances, labels_Idxs)
            print("Dunn Index =", Dunn_Index)
            Sil_Score = silhouette_score(data1, labels_Idxs)
            print('Silhouette Score =', Sil_Score)
            Cal_Har_Score = metrics.calinski_harabasz_score(data1, labels_Idxs)
            print('Calinski-Harabasz Index =', Cal_Har_Score)
            
            SM['Classification report'] = matrix
            SM['DB_Index'] = DB_Index
            SM['Dunn_Index'] = Dunn_Index
            SM['Sil_Score'] = Sil_Score
            SM['Cal_Har_Score'] = Cal_Har_Score

        def Actual_labels(Ground_Truth_data_path):
            with open(Ground_Truth_data_path, 'r') as f:
                actual_labels = np.genfromtxt(f, delimiter=',')
            return actual_labels

        def GetLabelsForOutlierDet(labels):
            get_labels_OutlierDet = []
            for key in labels:
                if labels[key] != -1:
                    get_labels_OutlierDet.append(0)
                else:
                    get_labels_OutlierDet.append(1)
            return get_labels_OutlierDet

        # Function to generate labels for getting DB, Dunn, Silhouette and Calinski-Harabasz Indexes
        def GetLabelsForIdxs(labels):
            get_labels_DB = []
            for key in labels:
                if labels[key] > 0:
                    get_labels_DB.append(labels[key]-1)
                else:
                    get_labels_DB.append(labels[key])
            return get_labels_DB
        
        Ground_Truth_data_path = SM['Ground_Truth_data_path']
        labels = SM['labels']
        data1 = SM['data1']
        
        actual_labels = Actual_labels(Ground_Truth_data_path)
        pred_labels = GetLabelsForOutlierDet(labels)
        get_labels_DB = GetLabelsForIdxs(labels)
        labels_Idxs = np.array(get_labels_DB)
        OutlierDetectionEvaluation(actual_labels, pred_labels, data1, labels_Idxs)
        
    SM['Name: Outlier Detection'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
# Name: Print Time

if 'Name: Print Time' not in SM:
    Total_Time = SM['Total_Time']
    Incoming_points = SM['Incoming_points']

    Avg_Time_Taken = Total_Time/len(Incoming_points)
    print("Average Time Taken is {} seconds ".format(Avg_Time_Taken))
    #print("Average density =", sum(density)/len(data1))
    
    SM['Avg_Time_Taken'] = Avg_Time_Taken
    
    SM['Name: Print Time'] = True
    
else:
    print("Skipped as Already Stored in Existing File!!")

In [ ]:
os.remove("distances_ddcal.p")
os.remove("distances_ddcal_updated.p")
os.remove("indices_ddcal.p")
os.remove("indices_ddcal_updated.p")
# print("File Removed!")

In [ ]:
# list(SM.items())
SM.clear()
print("Current Status of Shelve Dictionaries: ", list(SM.items()))
SM.close()